In [ ]:
from enum import IntEnum, unique
from matplotlib.pyplot import figure

import re 
import os

import statistics as st
import numpy as np
import matplotlib.pyplot as plt

LANES = 28
MODULES = 7
CONNECTORS = 4

re_start_str = r"d([0-9A-F])p([0-9A-F])s([0-9A-F]): ranges \{"
re_lane_str = r" ?(\d+): (\d+),?"
re_end_str = r"\}"

re_str = re_start_str
for i in range(LANES): re_str+=re_lane_str
re_str += re_end_str

cre = re.compile(re_str)

@unique
class Hg(IntEnum): # header grouping
    driver_dac = 1
    pre_dac = 2
    pll_stages = 3
    
    
@unique
class Ilg(IntEnum): # In line group
    lane = 0
    width = 1
    
@unique
class Ds(IntEnum): # DataSelect
    equalisation_off=0
    equalisation_2=2
    equalisation_4=4
    full_scan=5
    scan=6
    scan_172=7
    scan_192=8
    
lane2module_lut = {6:1,7:1,20:1,21:1,5:2,8:2,19:2,22:2,4:3,9:3,18:3,23:3,3:4,10:4,17:4,24:4,2:5,11:5,16:5,25:5,1:6,12:6,15:6,26:6,0:7,13:7,14:7,27:7}

In [ ]:
eq2 = """d3p3s4: ranges {0: 156, 1: 167, 2: 163, 3: 167, 4: 163, 5: 177, 6: 186, 7: 176, 8: 157, 9: 158, 10: 171, 11: 151, 12: 146, 13: 160, 14: 148, 15: 150, 16: 151, 17: 173, 18: 166, 19: 188, 20: 172, 21: 166, 22: 180, 23: 168, 24: 175, 25: 147, 26: 151, 27: 193}
d3p6s4: ranges {0: 170, 1: 169, 2: 167, 3: 167, 4: 171, 5: 179, 6: 191, 7: 179, 8: 160, 9: 162, 10: 174, 11: 152, 12: 153, 13: 168, 14: 154, 15: 155, 16: 157, 17: 175, 18: 171, 19: 189, 20: 176, 21: 170, 22: 185, 23: 173, 24: 179, 25: 157, 26: 155, 27: 199}
d3p9s4: ranges {0: 168, 1: 170, 2: 167, 3: 169, 4: 172, 5: 185, 6: 190, 7: 178, 8: 161, 9: 164, 10: 176, 11: 152, 12: 156, 13: 169, 14: 156, 15: 161, 16: 160, 17: 171, 18: 169, 19: 190, 20: 171, 21: 168, 22: 185, 23: 175, 24: 180, 25: 158, 26: 157, 27: 201}
d3pCs4: ranges {0: 165, 1: 167, 2: 172, 3: 172, 4: 172, 5: 185, 6: 189, 7: 179, 8: 163, 9: 165, 10: 179, 11: 157, 12: 158, 13: 168, 14: 157, 15: 161, 16: 162, 17: 175, 18: 171, 19: 193, 20: 179, 21: 171, 22: 187, 23: 175, 24: 182, 25: 154, 26: 156, 27: 201}
d3pFs4: ranges {0: 168, 1: 171, 2: 168, 3: 171, 4: 172, 5: 185, 6: 185, 7: 179, 8: 165, 9: 165, 10: 174, 11: 157, 12: 154, 13: 170, 14: 156, 15: 158, 16: 162, 17: 176, 18: 171, 19: 193, 20: 172, 21: 171, 22: 187, 23: 175, 24: 180, 25: 159, 26: 153, 27: 200}
d4p9s4: ranges {0: 163, 1: 167, 2: 163, 3: 169, 4: 168, 5: 182, 6: 190, 7: 179, 8: 163, 9: 164, 10: 174, 11: 156, 12: 154, 13: 166, 14: 153, 15: 157, 16: 160, 17: 172, 18: 171, 19: 193, 20: 174, 21: 168, 22: 186, 23: 178, 24: 179, 25: 156, 26: 152, 27: 194}
d4pCs4: ranges {0: 164, 1: 169, 2: 163, 3: 168, 4: 169, 5: 184, 6: 191, 7: 180, 8: 161, 9: 163, 10: 173, 11: 155, 12: 152, 13: 164, 14: 151, 15: 152, 16: 160, 17: 176, 18: 169, 19: 192, 20: 174, 21: 167, 22: 187, 23: 174, 24: 178, 25: 155, 26: 153, 27: 197}
d4pFs4: ranges {0: 161, 1: 168, 2: 165, 3: 169, 4: 168, 5: 180, 6: 191, 7: 178, 8: 165, 9: 165, 10: 173, 11: 158, 12: 152, 13: 164, 14: 154, 15: 158, 16: 155, 17: 172, 18: 165, 19: 188, 20: 174, 21: 167, 22: 187, 23: 172, 24: 180, 25: 152, 26: 151, 27: 195}
d5p9s4: ranges {0: 158, 1: 166, 2: 160, 3: 164, 4: 167, 5: 180, 6: 187, 7: 179, 8: 161, 9: 161, 10: 170, 11: 155, 12: 148, 13: 158, 14: 142, 15: 149, 16: 156, 17: 170, 18: 164, 19: 191, 20: 172, 21: 167, 22: 187, 23: 169, 24: 176, 25: 151, 26: 150, 27: 191}
d5pCs4: ranges {0: 157, 1: 162, 2: 161, 3: 161, 4: 166, 5: 180, 6: 187, 7: 177, 8: 163, 9: 161, 10: 171, 11: 153, 12: 151, 13: 161, 14: 148, 15: 151, 16: 155, 17: 170, 18: 167, 19: 191, 20: 173, 21: 166, 22: 187, 23: 174, 24: 173, 25: 149, 26: 147, 27: 186}
d5pFs4: ranges {0: 158, 1: 164, 2: 162, 3: 165, 4: 167, 5: 180, 6: 183, 7: 174, 8: 163, 9: 159, 10: 169, 11: 154, 12: 151, 13: 159, 14: 144, 15: 150, 16: 155, 17: 171, 18: 164, 19: 191, 20: 171, 21: 167, 22: 184, 23: 168, 24: 177, 25: 153, 26: 149, 27: 188}
d6p8s4: ranges {0: 153, 1: 163, 2: 155, 3: 161, 4: 164, 5: 178, 6: 184, 7: 176, 8: 160, 9: 161, 10: 167, 11: 150, 12: 147, 13: 155, 14: 141, 15: 146, 16: 147, 17: 169, 18: 167, 19: 193, 20: 173, 21: 169, 22: 185, 23: 171, 24: 175, 25: 150, 26: 146, 27: 178}
d6pCs4: ranges {0: 151, 1: 161, 2: 154, 3: 163, 4: 163, 5: 176, 6: 186, 7: 177, 8: 157, 9: 156, 10: 166, 11: 149, 12: 143, 13: 154, 14: 142, 15: 149, 16: 150, 17: 166, 18: 162, 19: 188, 20: 170, 21: 162, 22: 183, 23: 172, 24: 171, 25: 152, 26: 144, 27: 185}
d6pFs4: ranges {0: 155, 1: 163, 2: 158, 3: 164, 4: 161, 5: 175, 6: 186, 7: 174, 8: 160, 9: 159, 10: 167, 11: 150, 12: 146, 13: 154, 14: 137, 15: 147, 16: 151, 17: 167, 18: 160, 19: 186, 20: 165, 21: 164, 22: 183, 23: 166, 24: 169, 25: 148, 26: 143, 27: 185}
d8p8s4: ranges {0: 141, 1: 151, 2: 147, 3: 154, 4: 161, 5: 176, 6: 177, 7: 173, 8: 156, 9: 156, 10: 161, 11: 142, 12: 142, 13: 147, 14: 128, 15: 135, 16: 144, 17: 162, 18: 158, 19: 182, 20: 165, 21: 158, 22: 176, 23: 163, 24: 165, 25: 143, 26: 136, 27: 176}
d8pCs4: ranges {0: 142, 1: 151, 2: 150, 3: 152, 4: 159, 5: 172, 6: 180, 7: 167, 8: 152, 9: 154, 10: 161, 11: 142, 12: 143, 13: 149, 14: 130, 15: 140, 16: 143, 17: 162, 18: 157, 19: 186, 20: 163, 21: 158, 22: 180, 23: 166, 24: 170, 25: 145, 26: 138, 27: 177}
d8pFs4: ranges {0: 142, 1: 154, 2: 152, 3: 155, 4: 159, 5: 170, 6: 183, 7: 170, 8: 158, 9: 155, 10: 159, 11: 143, 12: 140, 13: 148, 14: 129, 15: 140, 16: 142, 17: 162, 18: 156, 19: 184, 20: 164, 21: 161, 22: 178, 23: 163, 24: 169, 25: 143, 26: 136, 27: 178}
dCp8s4: ranges {0: 127, 1: 142, 2: 138, 3: 146, 4: 151, 5: 164, 6: 177, 7: 167, 8: 149, 9: 147, 10: 149, 11: 132, 12: 129, 13: 129, 14: 109, 15: 124, 16: 126, 17: 152, 18: 148, 19: 176, 20: 160, 21: 153, 22: 171, 23: 158, 24: 160, 25: 132, 26: 124, 27: 159}
dCpCs4: ranges {0: 126, 1: 140, 2: 133, 3: 141, 4: 148, 5: 164, 6: 173, 7: 163, 8: 149, 9: 147, 10: 151, 11: 131, 12: 127, 13: 131, 14: 113, 15: 128, 16: 129, 17: 152, 18: 149, 19: 172, 20: 159, 21: 155, 22: 172, 23: 159, 24: 159, 25: 134, 26: 125, 27: 162}
dCpFs4: ranges {0: 123, 1: 139, 2: 139, 3: 147, 4: 152, 5: 163, 6: 174, 7: 164, 8: 151, 9: 147, 10: 152, 11: 132, 12: 128, 13: 131, 14: 112, 15: 127, 16: 130, 17: 149, 18: 146, 19: 174, 20: 158, 21: 154, 22: 173, 23: 160, 24: 159, 25: 136, 26: 124, 27: 162}
"""

eq0 = """d3p3s4: ranges {0: 147, 1: 161, 2: 158, 3: 165, 4: 156, 5: 171, 6: 188, 7: 174, 8: 155, 9: 154, 10: 169, 11: 149, 12: 140, 13: 157, 14: 141, 15: 142, 16: 150, 17: 169, 18: 162, 19: 187, 20: 173, 21: 164, 22: 180, 23: 166, 24: 172, 25: 140, 26: 145, 27: 185}
d3p6s4: ranges {0: 164, 1: 165, 2: 165, 3: 167, 4: 163, 5: 181, 6: 188, 7: 176, 8: 159, 9: 163, 10: 173, 11: 148, 12: 150, 13: 163, 14: 149, 15: 153, 16: 154, 17: 169, 18: 170, 19: 191, 20: 173, 21: 168, 22: 183, 23: 169, 24: 178, 25: 153, 26: 153, 27: 197}
d3p9s4: ranges {0: 162, 1: 169, 2: 166, 3: 166, 4: 167, 5: 183, 6: 187, 7: 176, 8: 160, 9: 163, 10: 170, 11: 150, 12: 151, 13: 165, 14: 151, 15: 158, 16: 158, 17: 170, 18: 172, 19: 193, 20: 175, 21: 174, 22: 186, 23: 174, 24: 181, 25: 151, 26: 153, 27: 199}
d3pCs4: ranges {0: 164, 1: 165, 2: 164, 3: 166, 4: 169, 5: 183, 6: 189, 7: 177, 8: 162, 9: 162, 10: 174, 11: 150, 12: 149, 13: 167, 14: 153, 15: 158, 16: 156, 17: 174, 18: 171, 19: 191, 20: 176, 21: 173, 22: 186, 23: 176, 24: 180, 25: 154, 26: 155, 27: 198}
d3pFs4: ranges {0: 157, 1: 168, 2: 161, 3: 164, 4: 169, 5: 183, 6: 192, 7: 178, 8: 163, 9: 164, 10: 173, 11: 151, 12: 152, 13: 169, 14: 153, 15: 156, 16: 158, 17: 175, 18: 172, 19: 194, 20: 177, 21: 173, 22: 184, 23: 173, 24: 183, 25: 154, 26: 155, 27: 198}
d4p9s4: ranges {0: 152, 1: 171, 2: 153, 3: 162, 4: 166, 5: 178, 6: 189, 7: 178, 8: 160, 9: 161, 10: 166, 11: 150, 12: 150, 13: 154, 14: 142, 15: 151, 16: 155, 17: 170, 18: 167, 19: 194, 20: 176, 21: 171, 22: 188, 23: 172, 24: 177, 25: 153, 26: 149, 27: 190}
d4pCs4: ranges {0: 159, 1: 166, 2: 157, 3: 167, 4: 166, 5: 181, 6: 184, 7: 177, 8: 163, 9: 162, 10: 166, 11: 151, 12: 152, 13: 161, 14: 146, 15: 151, 16: 151, 17: 175, 18: 168, 19: 192, 20: 177, 21: 168, 22: 186, 23: 170, 24: 175, 25: 153, 26: 151, 27: 192}
d4pFs4: ranges {0: 157, 1: 167, 2: 161, 3: 167, 4: 165, 5: 180, 6: 189, 7: 178, 8: 164, 9: 162, 10: 168, 11: 154, 12: 149, 13: 160, 14: 147, 15: 151, 16: 153, 17: 174, 18: 168, 19: 190, 20: 178, 21: 169, 22: 187, 23: 173, 24: 175, 25: 150, 26: 147, 27: 190}
d5p9s4: ranges {0: 150, 1: 155, 2: 154, 3: 162, 4: 164, 5: 174, 6: 188, 7: 177, 8: 160, 9: 158, 10: 165, 11: 145, 12: 144, 13: 148, 14: 137, 15: 141, 16: 150, 17: 168, 18: 164, 19: 191, 20: 172, 21: 167, 22: 185, 23: 170, 24: 173, 25: 145, 26: 143, 27: 184}
d5pCs4: ranges {0: 143, 1: 159, 2: 152, 3: 162, 4: 162, 5: 174, 6: 187, 7: 175, 8: 159, 9: 157, 10: 166, 11: 147, 12: 143, 13: 150, 14: 136, 15: 144, 16: 146, 17: 169, 18: 162, 19: 186, 20: 176, 21: 167, 22: 187, 23: 169, 24: 169, 25: 150, 26: 143, 27: 175}
d5pFs4: ranges {0: 147, 1: 162, 2: 150, 3: 162, 4: 162, 5: 174, 6: 184, 7: 176, 8: 160, 9: 156, 10: 166, 11: 147, 12: 142, 13: 152, 14: 140, 15: 145, 16: 149, 17: 171, 18: 162, 19: 192, 20: 171, 21: 168, 22: 183, 23: 171, 24: 173, 25: 147, 26: 142, 27: 184}
d6p8s4: ranges {0: 142, 1: 152, 2: 146, 3: 160, 4: 160, 5: 172, 6: 185, 7: 174, 8: 160, 9: 151, 10: 159, 11: 141, 12: 132, 13: 148, 14: 129, 15: 136, 16: 144, 17: 163, 18: 162, 19: 191, 20: 174, 21: 166, 22: 184, 23: 165, 24: 169, 25: 146, 26: 136, 27: 179}
d6pCs4: ranges {0: 142, 1: 150, 2: 147, 3: 158, 4: 162, 5: 174, 6: 185, 7: 174, 8: 156, 9: 155, 10: 162, 11: 138, 12: 137, 13: 145, 14: 127, 15: 139, 16: 143, 17: 163, 18: 159, 19: 187, 20: 173, 21: 164, 22: 183, 23: 169, 24: 172, 25: 146, 26: 139, 27: 177}
d6pFs4: ranges {0: 142, 1: 152, 2: 150, 3: 161, 4: 159, 5: 171, 6: 186, 7: 173, 8: 159, 9: 154, 10: 163, 11: 144, 12: 136, 13: 147, 14: 132, 15: 139, 16: 145, 17: 163, 18: 160, 19: 185, 20: 167, 21: 163, 22: 184, 23: 166, 24: 165, 25: 145, 26: 138, 27: 179}
d8p8s4: ranges {0: 126, 1: 144, 2: 135, 3: 149, 4: 152, 5: 169, 6: 179, 7: 169, 8: 151, 9: 148, 10: 155, 11: 134, 12: 130, 13: 135, 14: 116, 15: 129, 16: 137, 17: 155, 18: 153, 19: 181, 20: 166, 21: 160, 22: 176, 23: 163, 24: 163, 25: 140, 26: 128, 27: 163}
d8pCs4: ranges {0: 130, 1: 142, 2: 142, 3: 151, 4: 152, 5: 163, 6: 182, 7: 169, 8: 154, 9: 153, 10: 155, 11: 140, 12: 130, 13: 135, 14: 118, 15: 127, 16: 136, 17: 160, 18: 152, 19: 184, 20: 163, 21: 160, 22: 178, 23: 166, 24: 166, 25: 137, 26: 126, 27: 165}
d8pFs4: ranges {0: 130, 1: 146, 2: 138, 3: 148, 4: 152, 5: 167, 6: 179, 7: 168, 8: 154, 9: 151, 10: 155, 11: 133, 12: 128, 13: 135, 14: 119, 15: 128, 16: 132, 17: 157, 18: 155, 19: 182, 20: 166, 21: 158, 22: 179, 23: 161, 24: 164, 25: 135, 26: 128, 27: 169}
dCp8s4: ranges {0: 115, 1: 130, 2: 126, 3: 139, 4: 145, 5: 163, 6: 173, 7: 163, 8: 146, 9: 142, 10: 147, 11: 121, 12: 114, 13: 120, 14: 100, 15: 115, 16: 120, 17: 149, 18: 142, 19: 171, 20: 159, 21: 154, 22: 170, 23: 156, 24: 154, 25: 127, 26: 116, 27: 152}
dCpCs4: ranges {0: 114, 1: 128, 2: 124, 3: 140, 4: 145, 5: 159, 6: 173, 7: 162, 8: 148, 9: 145, 10: 146, 11: 124, 12: 116, 13: 123, 14: 101, 15: 114, 16: 118, 17: 148, 18: 145, 19: 174, 20: 159, 21: 156, 22: 173, 23: 155, 24: 159, 25: 128, 26: 116, 27: 151}
dCpFs4: ranges {0: 113, 1: 126, 2: 127, 3: 142, 4: 145, 5: 160, 6: 171, 7: 164, 8: 147, 9: 145, 10: 146, 11: 125, 12: 115, 13: 125, 14: 101, 15: 116, 16: 124, 17: 149, 18: 148, 19: 174, 20: 160, 21: 155, 22: 173, 23: 156, 24: 153, 25: 124, 26: 118, 27: 153}
"""

eq4 = """d3p3s4: ranges {0: 166, 1: 170, 2: 167, 3: 169, 4: 165, 5: 178, 6: 189, 7: 179, 8: 158, 9: 158, 10: 173, 11: 154, 12: 153, 13: 164, 14: 154, 15: 157, 16: 156, 17: 171, 18: 168, 19: 189, 20: 175, 21: 167, 22: 185, 23: 173, 24: 177, 25: 150, 26: 149, 27: 194}
d3p6s4: ranges {0: 170, 1: 170, 2: 170, 3: 170, 4: 170, 5: 179, 6: 194, 7: 182, 8: 162, 9: 163, 10: 176, 11: 157, 12: 158, 13: 168, 14: 154, 15: 164, 16: 161, 17: 173, 18: 172, 19: 190, 20: 177, 21: 173, 22: 185, 23: 179, 24: 179, 25: 151, 26: 156, 27: 201}
d3p9s4: ranges {0: 175, 1: 174, 2: 171, 3: 172, 4: 172, 5: 180, 6: 196, 7: 183, 8: 164, 9: 163, 10: 174, 11: 160, 12: 159, 13: 173, 14: 159, 15: 163, 16: 165, 17: 174, 18: 171, 19: 191, 20: 179, 21: 170, 22: 188, 23: 181, 24: 181, 25: 156, 26: 159, 27: 204}
d3pCs4: ranges {0: 175, 1: 177, 2: 173, 3: 174, 4: 171, 5: 181, 6: 195, 7: 182, 8: 163, 9: 166, 10: 176, 11: 157, 12: 164, 13: 171, 14: 155, 15: 163, 16: 163, 17: 175, 18: 172, 19: 192, 20: 179, 21: 169, 22: 188, 23: 178, 24: 180, 25: 155, 26: 158, 27: 203}
d3pFs4: ranges {0: 175, 1: 173, 2: 168, 3: 174, 4: 173, 5: 180, 6: 194, 7: 178, 8: 164, 9: 163, 10: 177, 11: 159, 12: 160, 13: 172, 14: 157, 15: 166, 16: 165, 17: 175, 18: 170, 19: 192, 20: 177, 21: 170, 22: 188, 23: 180, 24: 182, 25: 159, 26: 159, 27: 202}
d4p9s4: ranges {0: 171, 1: 172, 2: 169, 3: 169, 4: 171, 5: 182, 6: 192, 7: 180, 8: 165, 9: 163, 10: 174, 11: 159, 12: 159, 13: 171, 14: 157, 15: 162, 16: 158, 17: 172, 18: 172, 19: 193, 20: 176, 21: 170, 22: 188, 23: 181, 24: 179, 25: 158, 26: 159, 27: 200}
d4pCs4: ranges {0: 170, 1: 173, 2: 170, 3: 169, 4: 172, 5: 183, 6: 197, 7: 181, 8: 166, 9: 162, 10: 173, 11: 158, 12: 156, 13: 172, 14: 157, 15: 164, 16: 162, 17: 174, 18: 172, 19: 192, 20: 180, 21: 170, 22: 188, 23: 179, 24: 177, 25: 156, 26: 157, 27: 202}
d4pFs4: ranges {0: 168, 1: 172, 2: 171, 3: 173, 4: 171, 5: 180, 6: 193, 7: 182, 8: 165, 9: 162, 10: 173, 11: 160, 12: 161, 13: 172, 14: 158, 15: 162, 16: 162, 17: 174, 18: 171, 19: 193, 20: 178, 21: 169, 22: 188, 23: 183, 24: 178, 25: 156, 26: 155, 27: 191}
d5p9s4: ranges {0: 170, 1: 170, 2: 170, 3: 167, 4: 171, 5: 180, 6: 196, 7: 181, 8: 163, 9: 160, 10: 174, 11: 157, 12: 157, 13: 168, 14: 154, 15: 160, 16: 158, 17: 173, 18: 168, 19: 194, 20: 178, 21: 168, 22: 187, 23: 183, 24: 178, 25: 156, 26: 153, 27: 200}
d5pCs4: ranges {0: 162, 1: 173, 2: 168, 3: 170, 4: 170, 5: 180, 6: 193, 7: 180, 8: 166, 9: 162, 10: 172, 11: 158, 12: 158, 13: 164, 14: 153, 15: 160, 16: 159, 17: 172, 18: 170, 19: 190, 20: 180, 21: 169, 22: 187, 23: 180, 24: 177, 25: 155, 26: 155, 27: 197}
d5pFs4: ranges {0: 162, 1: 174, 2: 169, 3: 170, 4: 170, 5: 181, 6: 191, 7: 181, 8: 164, 9: 162, 10: 171, 11: 155, 12: 156, 13: 166, 14: 154, 15: 158, 16: 159, 17: 172, 18: 170, 19: 188, 20: 177, 21: 170, 22: 185, 23: 181, 24: 176, 25: 157, 26: 154, 27: 195}
d6p8s4: ranges {0: 167, 1: 170, 2: 167, 3: 166, 4: 170, 5: 178, 6: 194, 7: 181, 8: 163, 9: 161, 10: 172, 11: 156, 12: 159, 13: 169, 14: 152, 15: 161, 16: 159, 17: 174, 18: 168, 19: 188, 20: 175, 21: 168, 22: 188, 23: 178, 24: 171, 25: 153, 26: 151, 27: 198}
d6pCs4: ranges {0: 166, 1: 172, 2: 163, 3: 169, 4: 168, 5: 178, 6: 192, 7: 179, 8: 162, 9: 159, 10: 167, 11: 151, 12: 154, 13: 166, 14: 149, 15: 156, 16: 159, 17: 174, 18: 168, 19: 190, 20: 177, 21: 166, 22: 187, 23: 180, 24: 178, 25: 153, 26: 154, 27: 197}
d6pFs4: ranges {0: 163, 1: 170, 2: 166, 3: 168, 4: 170, 5: 182, 6: 193, 7: 180, 8: 164, 9: 158, 10: 170, 11: 157, 12: 154, 13: 168, 14: 151, 15: 158, 16: 159, 17: 174, 18: 168, 19: 188, 20: 177, 21: 168, 22: 187, 23: 176, 24: 179, 25: 155, 26: 151, 27: 193}
d8p8s4: ranges {0: 161, 1: 167, 2: 163, 3: 164, 4: 166, 5: 177, 6: 191, 7: 179, 8: 162, 9: 160, 10: 166, 11: 153, 12: 148, 13: 163, 14: 146, 15: 156, 16: 154, 17: 172, 18: 170, 19: 192, 20: 178, 21: 170, 22: 185, 23: 176, 24: 174, 25: 151, 26: 146, 27: 193}
d8pCs4: ranges {0: 163, 1: 168, 2: 164, 3: 166, 4: 170, 5: 176, 6: 190, 7: 177, 8: 162, 9: 157, 10: 170, 11: 155, 12: 151, 13: 162, 14: 148, 15: 156, 16: 153, 17: 169, 18: 165, 19: 189, 20: 174, 21: 164, 22: 189, 23: 176, 24: 174, 25: 151, 26: 145, 27: 192}
d8pFs4: ranges {0: 162, 1: 169, 2: 165, 3: 168, 4: 169, 5: 178, 6: 192, 7: 178, 8: 163, 9: 157, 10: 164, 11: 156, 12: 153, 13: 160, 14: 148, 15: 156, 16: 153, 17: 171, 18: 168, 19: 189, 20: 175, 21: 168, 22: 188, 23: 177, 24: 175, 25: 152, 26: 147, 27: 191}
dCp8s4: ranges {0: 147, 1: 161, 2: 154, 3: 157, 4: 165, 5: 174, 6: 190, 7: 176, 8: 158, 9: 154, 10: 164, 11: 148, 12: 146, 13: 152, 14: 138, 15: 149, 16: 145, 17: 161, 18: 158, 19: 186, 20: 168, 21: 163, 22: 178, 23: 167, 24: 169, 25: 145, 26: 142, 27: 178}
dCpCs4: ranges {0: 147, 1: 161, 2: 154, 3: 162, 4: 167, 5: 170, 6: 189, 7: 177, 8: 161, 9: 154, 10: 163, 11: 146, 12: 145, 13: 151, 14: 137, 15: 150, 16: 146, 17: 163, 18: 162, 19: 185, 20: 168, 21: 162, 22: 179, 23: 167, 24: 164, 25: 145, 26: 143, 27: 178}
dCpFs4: ranges {0: 152, 1: 157, 2: 159, 3: 163, 4: 165, 5: 172, 6: 187, 7: 177, 8: 161, 9: 154, 10: 162, 11: 147, 12: 148, 13: 154, 14: 140, 15: 150, 16: 145, 17: 165, 18: 162, 19: 185, 20: 173, 21: 166, 22: 179, 23: 164, 24: 168, 25: 145, 26: 140, 27: 182}
"""

full = """d3p0s4: ranges {0: 63, 1: 111, 2: 93, 3: 126, 4: 102, 5: 138, 6: 167, 7: 155, 8: 122, 9: 125, 10: 131, 11: 113, 12: 64, 13: 79, 14: 63, 15: 60, 16: 92, 17: 122, 18: 119, 19: 160, 20: 145, 21: 128, 22: 153, 23: 116, 24: 134, 25: 73, 26: 98, 27: 112}
d3p1s4: ranges {0: 125, 1: 152, 2: 141, 3: 157, 4: 146, 5: 166, 6: 188, 7: 174, 8: 150, 9: 149, 10: 160, 11: 147, 12: 123, 13: 132, 14: 120, 15: 119, 16: 135, 17: 160, 18: 152, 19: 177, 20: 165, 21: 150, 22: 176, 23: 151, 24: 162, 25: 129, 26: 134, 27: 168}
d3p2s4: ranges {0: 146, 1: 159, 2: 157, 3: 164, 4: 158, 5: 172, 6: 184, 7: 174, 8: 155, 9: 155, 10: 168, 11: 157, 12: 138, 13: 150, 14: 141, 15: 142, 16: 148, 17: 167, 18: 158, 19: 185, 20: 170, 21: 161, 22: 178, 23: 163, 24: 173, 25: 144, 26: 147, 27: 185}
d3p3s4: ranges {0: 152, 1: 163, 2: 165, 3: 163, 4: 162, 5: 178, 6: 185, 7: 176, 8: 158, 9: 161, 10: 168, 11: 158, 12: 146, 13: 158, 14: 147, 15: 150, 16: 156, 17: 168, 18: 165, 19: 188, 20: 169, 21: 167, 22: 180, 23: 167, 24: 174, 25: 153, 26: 150, 27: 190}
d3p4s4: ranges {0: 159, 1: 170, 2: 166, 3: 166, 4: 167, 5: 179, 6: 192, 7: 181, 8: 160, 9: 162, 10: 172, 11: 161, 12: 150, 13: 163, 14: 152, 15: 155, 16: 157, 17: 171, 18: 166, 19: 186, 20: 171, 21: 169, 22: 182, 23: 168, 24: 178, 25: 151, 26: 155, 27: 197}
d3p5s4: ranges {0: 168, 1: 169, 2: 170, 3: 166, 4: 170, 5: 181, 6: 189, 7: 180, 8: 161, 9: 162, 10: 176, 11: 162, 12: 152, 13: 167, 14: 155, 15: 159, 16: 155, 17: 172, 18: 171, 19: 186, 20: 172, 21: 168, 22: 183, 23: 173, 24: 179, 25: 156, 26: 154, 27: 198}
d3p6s4: ranges {0: 170, 1: 168, 2: 171, 3: 167, 4: 170, 5: 179, 6: 188, 7: 179, 8: 162, 9: 163, 10: 176, 11: 164, 12: 153, 13: 165, 14: 155, 15: 157, 16: 159, 17: 172, 18: 172, 19: 188, 20: 173, 21: 170, 22: 181, 23: 171, 24: 177, 25: 154, 26: 155, 27: 201}
d3p7s4: ranges {0: 165, 1: 170, 2: 173, 3: 169, 4: 172, 5: 180, 6: 191, 7: 182, 8: 163, 9: 161, 10: 178, 11: 162, 12: 155, 13: 167, 14: 155, 15: 161, 16: 159, 17: 172, 18: 172, 19: 190, 20: 173, 21: 170, 22: 183, 23: 170, 24: 181, 25: 157, 26: 156, 27: 202}
d3p8s4: ranges {0: 168, 1: 166, 2: 174, 3: 168, 4: 173, 5: 184, 6: 191, 7: 181, 8: 165, 9: 164, 10: 177, 11: 165, 12: 154, 13: 170, 14: 159, 15: 161, 16: 160, 17: 171, 18: 170, 19: 192, 20: 175, 21: 170, 22: 183, 23: 173, 24: 182, 25: 156, 26: 157, 27: 200}
d3p9s4: ranges {0: 169, 1: 171, 2: 170, 3: 170, 4: 170, 5: 184, 6: 191, 7: 179, 8: 164, 9: 164, 10: 177, 11: 167, 12: 159, 13: 168, 14: 158, 15: 159, 16: 161, 17: 174, 18: 170, 19: 191, 20: 173, 21: 169, 22: 182, 23: 174, 24: 174, 25: 158, 26: 156, 27: 202}
d3pAs4: ranges {0: 168, 1: 169, 2: 171, 3: 170, 4: 172, 5: 181, 6: 190, 7: 180, 8: 164, 9: 168, 10: 173, 11: 162, 12: 158, 13: 171, 14: 155, 15: 156, 16: 157, 17: 171, 18: 171, 19: 194, 20: 176, 21: 170, 22: 184, 23: 174, 24: 182, 25: 157, 26: 155, 27: 202}
d3pBs4: ranges {0: 170, 1: 171, 2: 172, 3: 167, 4: 172, 5: 183, 6: 192, 7: 180, 8: 165, 9: 166, 10: 175, 11: 167, 12: 156, 13: 171, 14: 154, 15: 162, 16: 160, 17: 172, 18: 173, 19: 192, 20: 173, 21: 169, 22: 183, 23: 172, 24: 179, 25: 159, 26: 157, 27: 202}
d3pCs4: ranges {0: 171, 1: 170, 2: 172, 3: 170, 4: 174, 5: 185, 6: 190, 7: 180, 8: 163, 9: 163, 10: 175, 11: 166, 12: 157, 13: 169, 14: 158, 15: 159, 16: 162, 17: 174, 18: 171, 19: 189, 20: 173, 21: 169, 22: 184, 23: 172, 24: 181, 25: 160, 26: 155, 27: 201}
d3pDs4: ranges {0: 168, 1: 171, 2: 173, 3: 168, 4: 171, 5: 182, 6: 190, 7: 180, 8: 166, 9: 166, 10: 173, 11: 166, 12: 158, 13: 170, 14: 156, 15: 161, 16: 160, 17: 175, 18: 171, 19: 191, 20: 174, 21: 167, 22: 185, 23: 175, 24: 180, 25: 159, 26: 155, 27: 199}
d3pEs4: ranges {0: 167, 1: 172, 2: 172, 3: 171, 4: 173, 5: 182, 6: 188, 7: 180, 8: 163, 9: 162, 10: 177, 11: 167, 12: 156, 13: 172, 14: 156, 15: 161, 16: 159, 17: 172, 18: 171, 19: 190, 20: 175, 21: 170, 22: 184, 23: 175, 24: 180, 25: 158, 26: 154, 27: 204}
d4p0s4: ranges {0: 75, 1: 112, 2: 103, 3: 129, 4: 111, 5: 141, 6: 168, 7: 158, 8: 127, 9: 126, 10: 125, 11: 119, 12: 75, 13: 86, 14: 73, 15: 74, 16: 96, 17: 127, 18: 123, 19: 159, 20: 148, 21: 133, 22: 159, 23: 127, 24: 132, 25: 82, 26: 100, 27: 119}
d4p1s4: ranges {0: 119, 1: 145, 2: 140, 3: 151, 4: 148, 5: 163, 6: 179, 7: 168, 8: 146, 9: 146, 10: 153, 11: 143, 12: 119, 13: 131, 14: 115, 15: 121, 16: 137, 17: 159, 18: 150, 19: 175, 20: 162, 21: 155, 22: 176, 23: 152, 24: 161, 25: 128, 26: 131, 27: 164}
d4p2s4: ranges {0: 143, 1: 155, 2: 155, 3: 159, 4: 158, 5: 174, 6: 184, 7: 172, 8: 154, 9: 152, 10: 165, 11: 152, 12: 136, 13: 148, 14: 134, 15: 134, 16: 145, 17: 164, 18: 159, 19: 184, 20: 169, 21: 159, 22: 181, 23: 162, 24: 167, 25: 146, 26: 142, 27: 181}
d4p3s4: ranges {0: 152, 1: 162, 2: 161, 3: 161, 4: 162, 5: 174, 6: 186, 7: 175, 8: 158, 9: 157, 10: 167, 11: 159, 12: 145, 13: 157, 14: 144, 15: 143, 16: 150, 17: 167, 18: 165, 19: 185, 20: 173, 21: 162, 22: 182, 23: 168, 24: 173, 25: 151, 26: 150, 27: 188}
d4p4s4: ranges {0: 158, 1: 168, 2: 166, 3: 166, 4: 167, 5: 180, 6: 186, 7: 178, 8: 160, 9: 159, 10: 169, 11: 160, 12: 147, 13: 161, 14: 148, 15: 149, 16: 155, 17: 172, 18: 162, 19: 188, 20: 170, 21: 166, 22: 184, 23: 170, 24: 177, 25: 151, 26: 149, 27: 192}
d4p5s4: ranges {0: 159, 1: 167, 2: 169, 3: 161, 4: 168, 5: 177, 6: 183, 7: 176, 8: 160, 9: 160, 10: 171, 11: 159, 12: 151, 13: 164, 14: 149, 15: 151, 16: 158, 17: 172, 18: 168, 19: 190, 20: 171, 21: 167, 22: 183, 23: 172, 24: 173, 25: 155, 26: 152, 27: 197}
d4p6s4: ranges {0: 165, 1: 164, 2: 166, 3: 166, 4: 169, 5: 180, 6: 189, 7: 176, 8: 163, 9: 165, 10: 174, 11: 163, 12: 153, 13: 165, 14: 154, 15: 158, 16: 157, 17: 171, 18: 168, 19: 185, 20: 173, 21: 169, 22: 183, 23: 174, 24: 177, 25: 152, 26: 154, 27: 199}
d4p7s4: ranges {0: 161, 1: 170, 2: 166, 3: 164, 4: 169, 5: 180, 6: 188, 7: 178, 8: 160, 9: 163, 10: 174, 11: 163, 12: 152, 13: 165, 14: 151, 15: 156, 16: 156, 17: 171, 18: 169, 19: 189, 20: 172, 21: 168, 22: 184, 23: 174, 24: 175, 25: 155, 26: 153, 27: 197}
d4p8s4: ranges {0: 162, 1: 167, 2: 168, 3: 168, 4: 170, 5: 182, 6: 188, 7: 178, 8: 163, 9: 166, 10: 175, 11: 160, 12: 151, 13: 162, 14: 150, 15: 154, 16: 158, 17: 170, 18: 168, 19: 186, 20: 172, 21: 167, 22: 179, 23: 174, 24: 175, 25: 157, 26: 156, 27: 196}
d4p9s4: ranges {0: 164, 1: 169, 2: 168, 3: 165, 4: 168, 5: 182, 6: 188, 7: 180, 8: 163, 9: 164, 10: 169, 11: 164, 12: 153, 13: 163, 14: 155, 15: 156, 16: 157, 17: 173, 18: 166, 19: 190, 20: 174, 21: 167, 22: 184, 23: 172, 24: 176, 25: 158, 26: 152, 27: 196}
d4pAs4: ranges {0: 163, 1: 163, 2: 166, 3: 169, 4: 169, 5: 181, 6: 189, 7: 178, 8: 162, 9: 164, 10: 169, 11: 162, 12: 152, 13: 165, 14: 155, 15: 153, 16: 157, 17: 170, 18: 169, 19: 191, 20: 172, 21: 168, 22: 185, 23: 172, 24: 175, 25: 158, 26: 152, 27: 196}
d4pBs4: ranges {0: 162, 1: 169, 2: 166, 3: 166, 4: 168, 5: 181, 6: 189, 7: 179, 8: 163, 9: 167, 10: 174, 11: 164, 12: 152, 13: 166, 14: 149, 15: 156, 16: 158, 17: 171, 18: 167, 19: 188, 20: 174, 21: 166, 22: 184, 23: 174, 24: 180, 25: 157, 26: 149, 27: 195}
d4pCs4: ranges {0: 163, 1: 169, 2: 166, 3: 168, 4: 170, 5: 182, 6: 190, 7: 180, 8: 162, 9: 163, 10: 171, 11: 164, 12: 154, 13: 162, 14: 153, 15: 153, 16: 156, 17: 171, 18: 167, 19: 190, 20: 170, 21: 161, 22: 184, 23: 175, 24: 174, 25: 156, 26: 149, 27: 196}
d4pDs4: ranges {0: 161, 1: 168, 2: 168, 3: 169, 4: 170, 5: 179, 6: 191, 7: 181, 8: 165, 9: 163, 10: 172, 11: 164, 12: 153, 13: 163, 14: 149, 15: 154, 16: 157, 17: 173, 18: 168, 19: 190, 20: 171, 21: 167, 22: 185, 23: 172, 24: 177, 25: 157, 26: 149, 27: 197}
d4pEs4: ranges {0: 157, 1: 168, 2: 166, 3: 168, 4: 169, 5: 182, 6: 189, 7: 179, 8: 164, 9: 164, 10: 172, 11: 163, 12: 153, 13: 162, 14: 152, 15: 155, 16: 156, 17: 171, 18: 168, 19: 189, 20: 172, 21: 168, 22: 183, 23: 176, 24: 173, 25: 158, 26: 150, 27: 196}
d5p0s4: ranges {0: 80, 1: 111, 2: 103, 3: 129, 4: 117, 5: 143, 6: 163, 7: 154, 8: 129, 9: 127, 10: 125, 11: 121, 12: 84, 13: 90, 14: 75, 15: 80, 16: 97, 17: 126, 18: 126, 19: 158, 20: 150, 21: 135, 22: 157, 23: 128, 24: 135, 25: 90, 26: 100, 27: 126}
d5p1s4: ranges {0: 117, 1: 140, 2: 140, 3: 144, 4: 148, 5: 163, 6: 174, 7: 166, 8: 146, 9: 144, 10: 152, 11: 142, 12: 121, 13: 124, 14: 111, 15: 116, 16: 131, 17: 155, 18: 147, 19: 176, 20: 162, 21: 152, 22: 173, 23: 150, 24: 156, 25: 126, 26: 127, 27: 161}
d5p2s4: ranges {0: 136, 1: 152, 2: 152, 3: 157, 4: 156, 5: 170, 6: 182, 7: 171, 8: 154, 9: 154, 10: 160, 11: 152, 12: 133, 13: 143, 14: 128, 15: 133, 16: 144, 17: 160, 18: 156, 19: 181, 20: 169, 21: 158, 22: 180, 23: 160, 24: 165, 25: 144, 26: 139, 27: 178}
d5p3s4: ranges {0: 147, 1: 156, 2: 157, 3: 162, 4: 160, 5: 173, 6: 180, 7: 174, 8: 160, 9: 156, 10: 162, 11: 157, 12: 141, 13: 152, 14: 134, 15: 140, 16: 151, 17: 167, 18: 159, 19: 183, 20: 168, 21: 163, 22: 182, 23: 163, 24: 169, 25: 150, 26: 141, 27: 185}
d5p4s4: ranges {0: 151, 1: 162, 2: 162, 3: 162, 4: 164, 5: 178, 6: 186, 7: 176, 8: 158, 9: 158, 10: 170, 11: 161, 12: 146, 13: 158, 14: 145, 15: 145, 16: 152, 17: 166, 18: 164, 19: 185, 20: 167, 21: 164, 22: 183, 23: 167, 24: 172, 25: 154, 26: 142, 27: 191}
d5p5s4: ranges {0: 154, 1: 164, 2: 161, 3: 165, 4: 166, 5: 179, 6: 185, 7: 178, 8: 158, 9: 162, 10: 166, 11: 157, 12: 148, 13: 162, 14: 145, 15: 149, 16: 156, 17: 171, 18: 167, 19: 187, 20: 174, 21: 171, 22: 184, 23: 169, 24: 173, 25: 153, 26: 144, 27: 190}
d5p6s4: ranges {0: 160, 1: 163, 2: 166, 3: 165, 4: 167, 5: 177, 6: 185, 7: 177, 8: 162, 9: 158, 10: 171, 11: 161, 12: 152, 13: 159, 14: 142, 15: 153, 16: 155, 17: 170, 18: 165, 19: 185, 20: 172, 21: 167, 22: 182, 23: 173, 24: 176, 25: 151, 26: 150, 27: 191}
d5p7s4: ranges {0: 159, 1: 163, 2: 164, 3: 165, 4: 166, 5: 180, 6: 189, 7: 178, 8: 159, 9: 161, 10: 169, 11: 162, 12: 149, 13: 162, 14: 145, 15: 149, 16: 156, 17: 171, 18: 166, 19: 189, 20: 170, 21: 165, 22: 183, 23: 172, 24: 174, 25: 154, 26: 149, 27: 190}
d5p8s4: ranges {0: 153, 1: 163, 2: 163, 3: 167, 4: 170, 5: 182, 6: 186, 7: 179, 8: 161, 9: 160, 10: 169, 11: 163, 12: 148, 13: 156, 14: 146, 15: 151, 16: 155, 17: 167, 18: 164, 19: 190, 20: 171, 21: 166, 22: 182, 23: 171, 24: 174, 25: 154, 26: 150, 27: 192}
d5p9s4: ranges {0: 159, 1: 165, 2: 160, 3: 162, 4: 168, 5: 178, 6: 191, 7: 181, 8: 161, 9: 162, 10: 170, 11: 162, 12: 149, 13: 159, 14: 144, 15: 151, 16: 153, 17: 167, 18: 165, 19: 186, 20: 171, 21: 164, 22: 184, 23: 171, 24: 175, 25: 151, 26: 149, 27: 182}
d5pAs4: ranges {0: 150, 1: 162, 2: 161, 3: 166, 4: 165, 5: 179, 6: 187, 7: 178, 8: 162, 9: 161, 10: 170, 11: 163, 12: 147, 13: 155, 14: 148, 15: 146, 16: 151, 17: 169, 18: 166, 19: 188, 20: 172, 21: 167, 22: 183, 23: 172, 24: 174, 25: 155, 26: 149, 27: 191}
d5pBs4: ranges {0: 158, 1: 165, 2: 162, 3: 165, 4: 167, 5: 176, 6: 189, 7: 178, 8: 161, 9: 160, 10: 169, 11: 161, 12: 147, 13: 159, 14: 143, 15: 149, 16: 155, 17: 171, 18: 167, 19: 187, 20: 171, 21: 166, 22: 183, 23: 172, 24: 176, 25: 154, 26: 145, 27: 193}
d5pCs4: ranges {0: 153, 1: 165, 2: 162, 3: 162, 4: 167, 5: 179, 6: 188, 7: 180, 8: 161, 9: 161, 10: 169, 11: 161, 12: 146, 13: 156, 14: 146, 15: 152, 16: 155, 17: 170, 18: 163, 19: 187, 20: 173, 21: 167, 22: 183, 23: 171, 24: 176, 25: 152, 26: 146, 27: 188}
d5pDs4: ranges {0: 158, 1: 164, 2: 163, 3: 167, 4: 166, 5: 179, 6: 189, 7: 174, 8: 163, 9: 159, 10: 168, 11: 162, 12: 147, 13: 161, 14: 141, 15: 153, 16: 150, 17: 168, 18: 164, 19: 189, 20: 170, 21: 163, 22: 185, 23: 173, 24: 174, 25: 153, 26: 149, 27: 192}
d5pEs4: ranges {0: 154, 1: 165, 2: 163, 3: 162, 4: 165, 5: 180, 6: 189, 7: 178, 8: 162, 9: 162, 10: 171, 11: 162, 12: 146, 13: 157, 14: 144, 15: 151, 16: 155, 17: 170, 18: 164, 19: 189, 20: 169, 21: 166, 22: 185, 23: 171, 24: 175, 25: 152, 26: 146, 27: 191}
d6p0s4: ranges {0: 82, 1: 109, 2: 110, 3: 127, 4: 120, 5: 143, 6: 162, 7: 153, 8: 130, 9: 128, 10: 129, 11: 121, 12: 84, 13: 96, 14: 75, 15: 85, 16: 99, 17: 128, 18: 125, 19: 159, 20: 147, 21: 133, 22: 159, 23: 131, 24: 137, 25: 97, 26: 101, 27: 129}
d6p1s4: ranges {0: 111, 1: 138, 2: 139, 3: 147, 4: 147, 5: 160, 6: 176, 7: 166, 8: 149, 9: 144, 10: 149, 11: 136, 12: 120, 13: 124, 14: 110, 15: 118, 16: 127, 17: 152, 18: 147, 19: 173, 20: 162, 21: 149, 22: 172, 23: 155, 24: 153, 25: 127, 26: 125, 27: 159}
d6p2s4: ranges {0: 129, 1: 148, 2: 152, 3: 154, 4: 154, 5: 168, 6: 177, 7: 169, 8: 155, 9: 146, 10: 158, 11: 150, 12: 131, 13: 138, 14: 123, 15: 131, 16: 142, 17: 162, 18: 156, 19: 180, 20: 166, 21: 158, 22: 177, 23: 159, 24: 161, 25: 140, 26: 133, 27: 175}
d6p3s4: ranges {0: 145, 1: 154, 2: 156, 3: 160, 4: 159, 5: 173, 6: 180, 7: 173, 8: 156, 9: 149, 10: 160, 11: 153, 12: 139, 13: 146, 14: 132, 15: 137, 16: 147, 17: 163, 18: 156, 19: 183, 20: 166, 21: 160, 22: 177, 23: 163, 24: 167, 25: 146, 26: 137, 27: 182}
d6p4s4: ranges {0: 149, 1: 161, 2: 156, 3: 158, 4: 161, 5: 173, 6: 185, 7: 173, 8: 160, 9: 156, 10: 164, 11: 159, 12: 143, 13: 152, 14: 140, 15: 141, 16: 149, 17: 167, 18: 160, 19: 186, 20: 169, 21: 165, 22: 179, 23: 169, 24: 169, 25: 151, 26: 144, 27: 186}
d6p5s4: ranges {0: 147, 1: 159, 2: 158, 3: 164, 4: 164, 5: 176, 6: 185, 7: 176, 8: 160, 9: 158, 10: 168, 11: 159, 12: 143, 13: 154, 14: 140, 15: 144, 16: 151, 17: 167, 18: 162, 19: 187, 20: 169, 21: 166, 22: 182, 23: 171, 24: 171, 25: 150, 26: 146, 27: 187}
d6p6s4: ranges {0: 144, 1: 156, 2: 159, 3: 161, 4: 163, 5: 178, 6: 183, 7: 176, 8: 160, 9: 157, 10: 166, 11: 161, 12: 144, 13: 152, 14: 141, 15: 144, 16: 150, 17: 169, 18: 162, 19: 187, 20: 170, 21: 165, 22: 183, 23: 169, 24: 172, 25: 150, 26: 143, 27: 186}
d6p7s4: ranges {0: 147, 1: 158, 2: 159, 3: 164, 4: 166, 5: 174, 6: 188, 7: 175, 8: 160, 9: 158, 10: 166, 11: 156, 12: 144, 13: 156, 14: 136, 15: 146, 16: 149, 17: 167, 18: 163, 19: 183, 20: 167, 21: 166, 22: 183, 23: 171, 24: 170, 25: 150, 26: 144, 27: 186}
d6p8s4: ranges {0: 150, 1: 159, 2: 158, 3: 162, 4: 166, 5: 176, 6: 188, 7: 176, 8: 160, 9: 156, 10: 164, 11: 160, 12: 144, 13: 155, 14: 141, 15: 144, 16: 149, 17: 166, 18: 161, 19: 185, 20: 171, 21: 165, 22: 182, 23: 171, 24: 173, 25: 148, 26: 144, 27: 183}
d6p9s4: ranges {0: 150, 1: 158, 2: 161, 3: 161, 4: 164, 5: 177, 6: 188, 7: 176, 8: 161, 9: 157, 10: 166, 11: 156, 12: 143, 13: 155, 14: 141, 15: 148, 16: 150, 17: 165, 18: 160, 19: 182, 20: 170, 21: 162, 22: 182, 23: 170, 24: 172, 25: 151, 26: 141, 27: 187}
d6pAs4: ranges {0: 148, 1: 161, 2: 159, 3: 162, 4: 163, 5: 176, 6: 184, 7: 174, 8: 158, 9: 157, 10: 162, 11: 153, 12: 144, 13: 154, 14: 142, 15: 143, 16: 150, 17: 166, 18: 161, 19: 185, 20: 170, 21: 164, 22: 185, 23: 169, 24: 173, 25: 149, 26: 144, 27: 186}
d6pBs4: ranges {0: 152, 1: 160, 2: 156, 3: 161, 4: 165, 5: 178, 6: 187, 7: 175, 8: 162, 9: 159, 10: 164, 11: 159, 12: 146, 13: 155, 14: 139, 15: 146, 16: 151, 17: 164, 18: 162, 19: 187, 20: 167, 21: 164, 22: 182, 23: 170, 24: 172, 25: 151, 26: 144, 27: 186}
d6pCs4: ranges {0: 147, 1: 159, 2: 160, 3: 160, 4: 166, 5: 177, 6: 183, 7: 175, 8: 162, 9: 158, 10: 167, 11: 160, 12: 143, 13: 154, 14: 136, 15: 148, 16: 152, 17: 166, 18: 161, 19: 183, 20: 167, 21: 165, 22: 180, 23: 171, 24: 171, 25: 150, 26: 142, 27: 184}
d6pDs4: ranges {0: 151, 1: 160, 2: 159, 3: 162, 4: 163, 5: 175, 6: 184, 7: 173, 8: 160, 9: 160, 10: 167, 11: 158, 12: 144, 13: 154, 14: 142, 15: 146, 16: 150, 17: 166, 18: 161, 19: 186, 20: 171, 21: 164, 22: 183, 23: 167, 24: 172, 25: 150, 26: 140, 27: 182}
d6pEs4: ranges {0: 145, 1: 160, 2: 158, 3: 164, 4: 166, 5: 177, 6: 184, 7: 176, 8: 161, 9: 160, 10: 165, 11: 160, 12: 143, 13: 154, 14: 141, 15: 146, 16: 150, 17: 167, 18: 161, 19: 187, 20: 169, 21: 164, 22: 184, 23: 170, 24: 170, 25: 149, 26: 142, 27: 184}
d7p0s4: ranges {0: 85, 1: 112, 2: 111, 3: 124, 4: 123, 5: 145, 6: 167, 7: 157, 8: 129, 9: 126, 10: 127, 11: 117, 12: 89, 13: 97, 14: 80, 15: 86, 16: 104, 17: 129, 18: 127, 19: 158, 20: 147, 21: 139, 22: 157, 23: 133, 24: 134, 25: 101, 26: 99, 27: 131}
d7p1s4: ranges {0: 114, 1: 136, 2: 137, 3: 144, 4: 147, 5: 161, 6: 178, 7: 168, 8: 146, 9: 142, 10: 144, 11: 139, 12: 118, 13: 125, 14: 106, 15: 115, 16: 126, 17: 149, 18: 143, 19: 173, 20: 160, 21: 152, 22: 172, 23: 149, 24: 154, 25: 124, 26: 121, 27: 154}
d7p2s4: ranges {0: 132, 1: 147, 2: 146, 3: 148, 4: 155, 5: 166, 6: 179, 7: 168, 8: 151, 9: 148, 10: 155, 11: 148, 12: 128, 13: 138, 14: 117, 15: 127, 16: 137, 17: 157, 18: 150, 19: 179, 20: 165, 21: 158, 22: 177, 23: 160, 24: 161, 25: 140, 26: 132, 27: 170}
d7p3s4: ranges {0: 139, 1: 147, 2: 154, 3: 155, 4: 159, 5: 174, 6: 183, 7: 169, 8: 158, 9: 150, 10: 159, 11: 149, 12: 135, 13: 144, 14: 128, 15: 136, 16: 144, 17: 161, 18: 155, 19: 181, 20: 166, 21: 159, 22: 179, 23: 166, 24: 163, 25: 144, 26: 140, 27: 177}
d7p4s4: ranges {0: 139, 1: 154, 2: 157, 3: 157, 4: 162, 5: 170, 6: 179, 7: 171, 8: 159, 9: 155, 10: 161, 11: 154, 12: 142, 13: 150, 14: 133, 15: 139, 16: 146, 17: 162, 18: 157, 19: 185, 20: 165, 21: 160, 22: 182, 23: 164, 24: 168, 25: 149, 26: 140, 27: 179}
d7p5s4: ranges {0: 146, 1: 154, 2: 156, 3: 159, 4: 162, 5: 176, 6: 188, 7: 176, 8: 159, 9: 154, 10: 162, 11: 154, 12: 141, 13: 148, 14: 133, 15: 142, 16: 147, 17: 167, 18: 159, 19: 183, 20: 165, 21: 162, 22: 182, 23: 167, 24: 170, 25: 149, 26: 139, 27: 182}
d7p6s4: ranges {0: 146, 1: 154, 2: 155, 3: 159, 4: 161, 5: 173, 6: 183, 7: 175, 8: 158, 9: 157, 10: 162, 11: 154, 12: 138, 13: 148, 14: 136, 15: 140, 16: 146, 17: 166, 18: 159, 19: 184, 20: 169, 21: 165, 22: 180, 23: 169, 24: 171, 25: 148, 26: 139, 27: 180}
d7p7s4: ranges {0: 145, 1: 156, 2: 159, 3: 156, 4: 163, 5: 176, 6: 184, 7: 174, 8: 157, 9: 153, 10: 162, 11: 155, 12: 141, 13: 149, 14: 135, 15: 139, 16: 146, 17: 161, 18: 159, 19: 183, 20: 165, 21: 164, 22: 182, 23: 166, 24: 171, 25: 148, 26: 138, 27: 183}
d7p8s4: ranges {0: 141, 1: 156, 2: 153, 3: 159, 4: 162, 5: 178, 6: 185, 7: 175, 8: 158, 9: 155, 10: 163, 11: 153, 12: 138, 13: 150, 14: 133, 15: 143, 16: 148, 17: 165, 18: 158, 19: 185, 20: 164, 21: 162, 22: 182, 23: 169, 24: 170, 25: 144, 26: 141, 27: 179}
d7p9s4: ranges {0: 145, 1: 155, 2: 155, 3: 155, 4: 162, 5: 175, 6: 184, 7: 173, 8: 161, 9: 156, 10: 162, 11: 158, 12: 139, 13: 149, 14: 131, 15: 142, 16: 146, 17: 164, 18: 159, 19: 182, 20: 164, 21: 162, 22: 182, 23: 167, 24: 171, 25: 148, 26: 137, 27: 180}
d7pAs4: ranges {0: 147, 1: 156, 2: 158, 3: 158, 4: 163, 5: 173, 6: 185, 7: 173, 8: 158, 9: 157, 10: 163, 11: 156, 12: 140, 13: 150, 14: 133, 15: 142, 16: 146, 17: 166, 18: 159, 19: 185, 20: 168, 21: 161, 22: 182, 23: 167, 24: 170, 25: 147, 26: 142, 27: 177}
d7pBs4: ranges {0: 148, 1: 154, 2: 156, 3: 156, 4: 161, 5: 176, 6: 184, 7: 173, 8: 160, 9: 157, 10: 162, 11: 157, 12: 139, 13: 152, 14: 133, 15: 139, 16: 149, 17: 165, 18: 161, 19: 184, 20: 169, 21: 167, 22: 181, 23: 167, 24: 168, 25: 146, 26: 136, 27: 178}
d7pCs4: ranges {0: 144, 1: 155, 2: 157, 3: 159, 4: 162, 5: 176, 6: 180, 7: 174, 8: 158, 9: 155, 10: 165, 11: 155, 12: 140, 13: 150, 14: 133, 15: 142, 16: 148, 17: 162, 18: 161, 19: 181, 20: 165, 21: 165, 22: 182, 23: 168, 24: 169, 25: 145, 26: 138, 27: 180}
d7pDs4: ranges {0: 147, 1: 157, 2: 155, 3: 159, 4: 159, 5: 173, 6: 180, 7: 174, 8: 159, 9: 156, 10: 163, 11: 155, 12: 141, 13: 152, 14: 133, 15: 144, 16: 147, 17: 162, 18: 160, 19: 184, 20: 168, 21: 163, 22: 181, 23: 169, 24: 169, 25: 148, 26: 141, 27: 178}
d7pEs4: ranges {0: 148, 1: 157, 2: 153, 3: 157, 4: 163, 5: 172, 6: 183, 7: 170, 8: 160, 9: 155, 10: 164, 11: 151, 12: 140, 13: 152, 14: 135, 15: 140, 16: 147, 17: 165, 18: 157, 19: 186, 20: 163, 21: 162, 22: 180, 23: 163, 24: 169, 25: 147, 26: 140, 27: 182}
d8p0s4: ranges {0: 88, 1: 112, 2: 112, 3: 123, 4: 122, 5: 141, 6: 164, 7: 152, 8: 130, 9: 128, 10: 128, 11: 120, 12: 89, 13: 98, 14: 75, 15: 88, 16: 99, 17: 127, 18: 128, 19: 159, 20: 148, 21: 137, 22: 156, 23: 136, 24: 138, 25: 99, 26: 101, 27: 128}
d8p1s4: ranges {0: 112, 1: 134, 2: 129, 3: 141, 4: 143, 5: 163, 6: 172, 7: 165, 8: 145, 9: 140, 10: 145, 11: 135, 12: 116, 13: 123, 14: 107, 15: 116, 16: 121, 17: 150, 18: 145, 19: 171, 20: 161, 21: 153, 22: 170, 23: 152, 24: 154, 25: 128, 26: 120, 27: 152}
d8p2s4: ranges {0: 127, 1: 142, 2: 146, 3: 151, 4: 153, 5: 164, 6: 177, 7: 166, 8: 150, 9: 146, 10: 154, 11: 140, 12: 128, 13: 133, 14: 119, 15: 127, 16: 137, 17: 154, 18: 149, 19: 176, 20: 161, 21: 153, 22: 175, 23: 159, 24: 159, 25: 136, 26: 133, 27: 166}
d8p3s4: ranges {0: 134, 1: 149, 2: 152, 3: 151, 4: 157, 5: 170, 6: 178, 7: 170, 8: 153, 9: 152, 10: 158, 11: 149, 12: 134, 13: 143, 14: 126, 15: 134, 16: 137, 17: 160, 18: 154, 19: 180, 20: 161, 21: 157, 22: 180, 23: 161, 24: 164, 25: 140, 26: 137, 27: 170}
d8p4s4: ranges {0: 140, 1: 153, 2: 149, 3: 158, 4: 159, 5: 170, 6: 181, 7: 170, 8: 158, 9: 154, 10: 159, 11: 149, 12: 138, 13: 141, 14: 128, 15: 137, 16: 145, 17: 160, 18: 157, 19: 181, 20: 164, 21: 160, 22: 177, 23: 167, 24: 167, 25: 143, 26: 134, 27: 176}
d8p5s4: ranges {0: 141, 1: 151, 2: 153, 3: 158, 4: 159, 5: 174, 6: 183, 7: 170, 8: 158, 9: 155, 10: 163, 11: 151, 12: 136, 13: 144, 14: 131, 15: 139, 16: 142, 17: 164, 18: 157, 19: 182, 20: 165, 21: 161, 22: 178, 23: 161, 24: 170, 25: 141, 26: 137, 27: 176}
d8p6s4: ranges {0: 137, 1: 152, 2: 154, 3: 154, 4: 160, 5: 174, 6: 182, 7: 172, 8: 158, 9: 154, 10: 162, 11: 151, 12: 136, 13: 143, 14: 130, 15: 137, 16: 141, 17: 165, 18: 157, 19: 185, 20: 165, 21: 160, 22: 179, 23: 164, 24: 167, 25: 143, 26: 137, 27: 176}
d8p7s4: ranges {0: 132, 1: 152, 2: 156, 3: 158, 4: 160, 5: 172, 6: 184, 7: 176, 8: 157, 9: 154, 10: 161, 11: 153, 12: 137, 13: 142, 14: 132, 15: 138, 16: 143, 17: 163, 18: 157, 19: 178, 20: 165, 21: 160, 22: 179, 23: 168, 24: 167, 25: 143, 26: 138, 27: 178}
d8p8s4: ranges {0: 142, 1: 151, 2: 152, 3: 156, 4: 159, 5: 176, 6: 179, 7: 171, 8: 157, 9: 154, 10: 158, 11: 155, 12: 140, 13: 148, 14: 128, 15: 137, 16: 142, 17: 160, 18: 157, 19: 184, 20: 166, 21: 161, 22: 181, 23: 166, 24: 170, 25: 139, 26: 135, 27: 177}
d8p9s4: ranges {0: 143, 1: 148, 2: 152, 3: 155, 4: 159, 5: 174, 6: 184, 7: 174, 8: 156, 9: 153, 10: 162, 11: 150, 12: 138, 13: 143, 14: 129, 15: 139, 16: 142, 17: 161, 18: 159, 19: 180, 20: 168, 21: 159, 22: 176, 23: 167, 24: 170, 25: 143, 26: 137, 27: 172}
d8pAs4: ranges {0: 136, 1: 150, 2: 149, 3: 156, 4: 160, 5: 172, 6: 185, 7: 174, 8: 159, 9: 154, 10: 157, 11: 144, 12: 133, 13: 143, 14: 133, 15: 138, 16: 141, 17: 158, 18: 157, 19: 181, 20: 166, 21: 159, 22: 177, 23: 164, 24: 166, 25: 145, 26: 137, 27: 177}
d8pBs4: ranges {0: 141, 1: 151, 2: 151, 3: 156, 4: 158, 5: 173, 6: 181, 7: 170, 8: 156, 9: 152, 10: 159, 11: 152, 12: 137, 13: 143, 14: 131, 15: 136, 16: 143, 17: 163, 18: 159, 19: 179, 20: 165, 21: 160, 22: 180, 23: 166, 24: 169, 25: 142, 26: 137, 27: 175}
d8pCs4: ranges {0: 140, 1: 153, 2: 151, 3: 155, 4: 158, 5: 174, 6: 183, 7: 173, 8: 158, 9: 153, 10: 157, 11: 152, 12: 139, 13: 144, 14: 130, 15: 138, 16: 144, 17: 163, 18: 159, 19: 181, 20: 161, 21: 161, 22: 179, 23: 165, 24: 164, 25: 142, 26: 136, 27: 174}
d8pDs4: ranges {0: 143, 1: 151, 2: 152, 3: 156, 4: 160, 5: 174, 6: 187, 7: 176, 8: 157, 9: 153, 10: 160, 11: 152, 12: 139, 13: 147, 14: 133, 15: 137, 16: 141, 17: 163, 18: 158, 19: 180, 20: 163, 21: 161, 22: 180, 23: 164, 24: 169, 25: 145, 26: 137, 27: 178}
d8pEs4: ranges {0: 141, 1: 153, 2: 155, 3: 156, 4: 160, 5: 174, 6: 183, 7: 171, 8: 155, 9: 154, 10: 161, 11: 151, 12: 138, 13: 145, 14: 130, 15: 139, 16: 142, 17: 162, 18: 160, 19: 184, 20: 162, 21: 160, 22: 179, 23: 165, 24: 168, 25: 144, 26: 135, 27: 178}
d9p0s4: ranges {0: 89, 1: 111, 2: 111, 3: 123, 4: 126, 5: 146, 6: 162, 7: 151, 8: 131, 9: 128, 10: 126, 11: 120, 12: 91, 13: 99, 14: 83, 15: 91, 16: 101, 17: 127, 18: 126, 19: 158, 20: 147, 21: 140, 22: 160, 23: 134, 24: 138, 25: 102, 26: 95, 27: 130}
d9p1s4: ranges {0: 105, 1: 134, 2: 131, 3: 141, 4: 140, 5: 158, 6: 172, 7: 161, 8: 145, 9: 141, 10: 144, 11: 132, 12: 113, 13: 119, 14: 100, 15: 113, 16: 124, 17: 147, 18: 141, 19: 168, 20: 157, 21: 150, 22: 168, 23: 149, 24: 148, 25: 122, 26: 117, 27: 150}
d9p2s4: ranges {0: 120, 1: 143, 2: 143, 3: 149, 4: 152, 5: 164, 6: 179, 7: 163, 8: 149, 9: 148, 10: 150, 11: 139, 12: 125, 13: 127, 14: 113, 15: 125, 16: 133, 17: 153, 18: 150, 19: 176, 20: 161, 21: 155, 22: 174, 23: 157, 24: 157, 25: 137, 26: 128, 27: 165}
d9p3s4: ranges {0: 132, 1: 145, 2: 149, 3: 151, 4: 158, 5: 168, 6: 179, 7: 165, 8: 152, 9: 146, 10: 157, 11: 146, 12: 133, 13: 138, 14: 120, 15: 132, 16: 135, 17: 156, 18: 152, 19: 179, 20: 165, 21: 159, 22: 176, 23: 164, 24: 160, 25: 141, 26: 131, 27: 171}
d9p4s4: ranges {0: 136, 1: 147, 2: 148, 3: 154, 4: 159, 5: 168, 6: 181, 7: 171, 8: 155, 9: 151, 10: 158, 11: 147, 12: 136, 13: 139, 14: 123, 15: 133, 16: 140, 17: 159, 18: 155, 19: 178, 20: 164, 21: 159, 22: 177, 23: 161, 24: 163, 25: 140, 26: 134, 27: 168}
d9p5s4: ranges {0: 134, 1: 147, 2: 149, 3: 152, 4: 156, 5: 172, 6: 181, 7: 169, 8: 156, 9: 150, 10: 159, 11: 148, 12: 135, 13: 137, 14: 125, 15: 133, 16: 140, 17: 158, 18: 156, 19: 181, 20: 165, 21: 158, 22: 176, 23: 165, 24: 167, 25: 139, 26: 133, 27: 172}
d9p6s4: ranges {0: 130, 1: 145, 2: 150, 3: 155, 4: 157, 5: 172, 6: 184, 7: 174, 8: 156, 9: 154, 10: 159, 11: 146, 12: 134, 13: 138, 14: 126, 15: 132, 16: 138, 17: 160, 18: 157, 19: 180, 20: 160, 21: 157, 22: 176, 23: 164, 24: 167, 25: 140, 26: 129, 27: 173}
d9p7s4: ranges {0: 136, 1: 147, 2: 149, 3: 153, 4: 155, 5: 171, 6: 185, 7: 172, 8: 156, 9: 151, 10: 156, 11: 145, 12: 129, 13: 140, 14: 120, 15: 134, 16: 137, 17: 158, 18: 156, 19: 182, 20: 164, 21: 160, 22: 177, 23: 165, 24: 166, 25: 140, 26: 133, 27: 172}
d9p8s4: ranges {0: 135, 1: 146, 2: 148, 3: 155, 4: 161, 5: 171, 6: 182, 7: 170, 8: 155, 9: 155, 10: 157, 11: 148, 12: 136, 13: 141, 14: 124, 15: 135, 16: 138, 17: 161, 18: 157, 19: 180, 20: 163, 21: 157, 22: 180, 23: 162, 24: 165, 25: 140, 26: 133, 27: 172}
d9p9s4: ranges {0: 132, 1: 148, 2: 148, 3: 149, 4: 157, 5: 171, 6: 180, 7: 170, 8: 156, 9: 151, 10: 158, 11: 147, 12: 135, 13: 140, 14: 126, 15: 134, 16: 139, 17: 159, 18: 159, 19: 180, 20: 162, 21: 159, 22: 180, 23: 163, 24: 163, 25: 142, 26: 133, 27: 171}
d9pAs4: ranges {0: 135, 1: 146, 2: 149, 3: 150, 4: 156, 5: 171, 6: 180, 7: 168, 8: 157, 9: 151, 10: 158, 11: 146, 12: 136, 13: 140, 14: 121, 15: 136, 16: 138, 17: 158, 18: 157, 19: 180, 20: 164, 21: 159, 22: 177, 23: 163, 24: 166, 25: 141, 26: 133, 27: 172}
d9pBs4: ranges {0: 140, 1: 149, 2: 149, 3: 155, 4: 159, 5: 171, 6: 179, 7: 170, 8: 156, 9: 149, 10: 158, 11: 149, 12: 134, 13: 143, 14: 126, 15: 135, 16: 137, 17: 157, 18: 153, 19: 182, 20: 162, 21: 157, 22: 175, 23: 165, 24: 165, 25: 138, 26: 132, 27: 171}
d9pCs4: ranges {0: 134, 1: 145, 2: 149, 3: 154, 4: 156, 5: 170, 6: 181, 7: 170, 8: 154, 9: 151, 10: 155, 11: 150, 12: 133, 13: 141, 14: 126, 15: 135, 16: 138, 17: 158, 18: 155, 19: 177, 20: 165, 21: 159, 22: 178, 23: 163, 24: 168, 25: 139, 26: 133, 27: 171}
d9pDs4: ranges {0: 135, 1: 145, 2: 148, 3: 150, 4: 156, 5: 172, 6: 180, 7: 169, 8: 157, 9: 151, 10: 157, 11: 150, 12: 134, 13: 142, 14: 125, 15: 135, 16: 138, 17: 159, 18: 157, 19: 183, 20: 165, 21: 158, 22: 177, 23: 164, 24: 162, 25: 141, 26: 131, 27: 175}
d9pEs4: ranges {0: 134, 1: 147, 2: 149, 3: 155, 4: 157, 5: 168, 6: 179, 7: 167, 8: 155, 9: 149, 10: 159, 11: 151, 12: 135, 13: 145, 14: 128, 15: 134, 16: 137, 17: 160, 18: 157, 19: 182, 20: 162, 21: 161, 22: 177, 23: 163, 24: 167, 25: 139, 26: 134, 27: 172}
dAp0s4: ranges {0: 90, 1: 111, 2: 109, 3: 123, 4: 127, 5: 145, 6: 161, 7: 149, 8: 133, 9: 128, 10: 125, 11: 118, 12: 89, 13: 100, 14: 82, 15: 90, 16: 102, 17: 128, 18: 126, 19: 158, 20: 145, 21: 139, 22: 158, 23: 137, 24: 136, 25: 104, 26: 97, 27: 132}
dAp1s4: ranges {0: 103, 1: 132, 2: 130, 3: 141, 4: 140, 5: 155, 6: 168, 7: 162, 8: 143, 9: 136, 10: 141, 11: 128, 12: 114, 13: 114, 14: 98, 15: 112, 16: 119, 17: 142, 18: 142, 19: 164, 20: 154, 21: 147, 22: 166, 23: 146, 24: 149, 25: 118, 26: 113, 27: 150}
dAp2s4: ranges {0: 122, 1: 138, 2: 142, 3: 144, 4: 150, 5: 164, 6: 174, 7: 166, 8: 147, 9: 144, 10: 151, 11: 136, 12: 126, 13: 128, 14: 111, 15: 124, 16: 131, 17: 153, 18: 147, 19: 171, 20: 159, 21: 153, 22: 173, 23: 154, 24: 154, 25: 132, 26: 127, 27: 157}
dAp3s4: ranges {0: 120, 1: 140, 2: 144, 3: 151, 4: 155, 5: 167, 6: 177, 7: 166, 8: 151, 9: 148, 10: 154, 11: 145, 12: 128, 13: 134, 14: 116, 15: 129, 16: 136, 17: 155, 18: 150, 19: 178, 20: 162, 21: 156, 22: 177, 23: 158, 24: 160, 25: 135, 26: 130, 27: 167}
dAp4s4: ranges {0: 131, 1: 143, 2: 145, 3: 151, 4: 157, 5: 170, 6: 179, 7: 171, 8: 148, 9: 151, 10: 156, 11: 144, 12: 132, 13: 136, 14: 121, 15: 130, 16: 134, 17: 157, 18: 151, 19: 176, 20: 162, 21: 153, 22: 174, 23: 164, 24: 158, 25: 139, 26: 130, 27: 169}
dAp5s4: ranges {0: 131, 1: 140, 2: 146, 3: 150, 4: 156, 5: 168, 6: 176, 7: 169, 8: 155, 9: 149, 10: 157, 11: 144, 12: 130, 13: 136, 14: 120, 15: 132, 16: 137, 17: 157, 18: 152, 19: 178, 20: 163, 21: 156, 22: 176, 23: 157, 24: 159, 25: 136, 26: 130, 27: 167}
dAp6s4: ranges {0: 130, 1: 147, 2: 146, 3: 149, 4: 156, 5: 169, 6: 176, 7: 169, 8: 153, 9: 148, 10: 156, 11: 145, 12: 128, 13: 137, 14: 119, 15: 131, 16: 136, 17: 157, 18: 152, 19: 178, 20: 164, 21: 155, 22: 175, 23: 162, 24: 164, 25: 138, 26: 130, 27: 167}
dAp7s4: ranges {0: 126, 1: 142, 2: 146, 3: 150, 4: 157, 5: 167, 6: 178, 7: 168, 8: 152, 9: 150, 10: 157, 11: 144, 12: 131, 13: 133, 14: 121, 15: 130, 16: 133, 17: 157, 18: 150, 19: 176, 20: 161, 21: 154, 22: 177, 23: 159, 24: 163, 25: 138, 26: 129, 27: 165}
dAp8s4: ranges {0: 130, 1: 146, 2: 144, 3: 148, 4: 155, 5: 169, 6: 181, 7: 171, 8: 155, 9: 151, 10: 155, 11: 145, 12: 133, 13: 138, 14: 118, 15: 132, 16: 135, 17: 157, 18: 147, 19: 181, 20: 160, 21: 155, 22: 176, 23: 161, 24: 163, 25: 136, 26: 129, 27: 169}
dAp9s4: ranges {0: 129, 1: 146, 2: 141, 3: 151, 4: 154, 5: 167, 6: 180, 7: 166, 8: 154, 9: 151, 10: 158, 11: 145, 12: 130, 13: 137, 14: 121, 15: 132, 16: 136, 17: 157, 18: 150, 19: 179, 20: 159, 21: 156, 22: 175, 23: 160, 24: 161, 25: 138, 26: 130, 27: 168}
dApAs4: ranges {0: 134, 1: 145, 2: 146, 3: 149, 4: 153, 5: 171, 6: 175, 7: 167, 8: 153, 9: 148, 10: 156, 11: 147, 12: 131, 13: 135, 14: 121, 15: 132, 16: 135, 17: 157, 18: 153, 19: 178, 20: 163, 21: 155, 22: 173, 23: 162, 24: 164, 25: 137, 26: 132, 27: 168}
dApBs4: ranges {0: 127, 1: 142, 2: 144, 3: 151, 4: 154, 5: 169, 6: 175, 7: 169, 8: 156, 9: 150, 10: 153, 11: 146, 12: 132, 13: 137, 14: 122, 15: 131, 16: 135, 17: 157, 18: 153, 19: 179, 20: 164, 21: 155, 22: 173, 23: 162, 24: 163, 25: 135, 26: 129, 27: 170}
dApCs4: ranges {0: 130, 1: 143, 2: 139, 3: 150, 4: 157, 5: 168, 6: 177, 7: 169, 8: 152, 9: 149, 10: 155, 11: 146, 12: 128, 13: 136, 14: 121, 15: 132, 16: 137, 17: 158, 18: 154, 19: 179, 20: 159, 21: 156, 22: 175, 23: 157, 24: 159, 25: 139, 26: 128, 27: 169}
dApDs4: ranges {0: 133, 1: 146, 2: 145, 3: 147, 4: 156, 5: 168, 6: 179, 7: 167, 8: 153, 9: 150, 10: 157, 11: 143, 12: 125, 13: 137, 14: 122, 15: 135, 16: 136, 17: 158, 18: 153, 19: 179, 20: 163, 21: 158, 22: 175, 23: 159, 24: 163, 25: 138, 26: 132, 27: 170}
dApEs4: ranges {0: 128, 1: 145, 2: 144, 3: 154, 4: 154, 5: 169, 6: 179, 7: 169, 8: 151, 9: 151, 10: 153, 11: 147, 12: 131, 13: 135, 14: 122, 15: 132, 16: 135, 17: 155, 18: 152, 19: 178, 20: 161, 21: 156, 22: 176, 23: 161, 24: 161, 25: 140, 26: 127, 27: 170}
dBp0s4: ranges {0: 90, 1: 113, 2: 112, 3: 123, 4: 127, 5: 145, 6: 162, 7: 151, 8: 128, 9: 128, 10: 123, 11: 118, 12: 93, 13: 98, 14: 80, 15: 92, 16: 103, 17: 123, 18: 124, 19: 155, 20: 145, 21: 137, 22: 154, 23: 133, 24: 134, 25: 101, 26: 97, 27: 131}
dBp1s4: ranges {0: 102, 1: 129, 2: 128, 3: 138, 4: 141, 5: 157, 6: 169, 7: 161, 8: 143, 9: 138, 10: 139, 11: 128, 12: 107, 13: 117, 14: 97, 15: 111, 16: 117, 17: 144, 18: 141, 19: 167, 20: 155, 21: 150, 22: 166, 23: 149, 24: 150, 25: 121, 26: 112, 27: 147}
dBp2s4: ranges {0: 115, 1: 132, 2: 139, 3: 144, 4: 151, 5: 162, 6: 172, 7: 165, 8: 148, 9: 142, 10: 145, 11: 139, 12: 124, 13: 123, 14: 107, 15: 121, 16: 128, 17: 149, 18: 145, 19: 173, 20: 159, 21: 152, 22: 171, 23: 154, 24: 155, 25: 131, 26: 123, 27: 158}
dBp3s4: ranges {0: 124, 1: 141, 2: 140, 3: 144, 4: 153, 5: 162, 6: 175, 7: 168, 8: 149, 9: 145, 10: 151, 11: 143, 12: 125, 13: 131, 14: 115, 15: 129, 16: 129, 17: 154, 18: 149, 19: 174, 20: 159, 21: 155, 22: 171, 23: 154, 24: 158, 25: 134, 26: 127, 27: 164}
dBp4s4: ranges {0: 126, 1: 137, 2: 143, 3: 146, 4: 152, 5: 167, 6: 175, 7: 164, 8: 150, 9: 147, 10: 153, 11: 142, 12: 126, 13: 131, 14: 116, 15: 129, 16: 131, 17: 152, 18: 149, 19: 175, 20: 159, 21: 154, 22: 174, 23: 160, 24: 157, 25: 134, 26: 125, 27: 159}
dBp5s4: ranges {0: 127, 1: 139, 2: 144, 3: 151, 4: 154, 5: 169, 6: 178, 7: 167, 8: 152, 9: 146, 10: 152, 11: 142, 12: 128, 13: 131, 14: 119, 15: 127, 16: 131, 17: 153, 18: 149, 19: 178, 20: 160, 21: 155, 22: 172, 23: 159, 24: 158, 25: 132, 26: 129, 27: 165}
dBp6s4: ranges {0: 121, 1: 141, 2: 145, 3: 145, 4: 153, 5: 169, 6: 177, 7: 167, 8: 151, 9: 150, 10: 150, 11: 143, 12: 129, 13: 130, 14: 115, 15: 127, 16: 132, 17: 154, 18: 148, 19: 173, 20: 159, 21: 155, 22: 172, 23: 158, 24: 163, 25: 134, 26: 130, 27: 165}
dBp7s4: ranges {0: 130, 1: 142, 2: 145, 3: 145, 4: 151, 5: 165, 6: 177, 7: 165, 8: 151, 9: 149, 10: 152, 11: 139, 12: 129, 13: 134, 14: 119, 15: 129, 16: 133, 17: 153, 18: 149, 19: 176, 20: 158, 21: 155, 22: 174, 23: 159, 24: 163, 25: 134, 26: 129, 27: 165}
dBp8s4: ranges {0: 127, 1: 142, 2: 143, 3: 150, 4: 152, 5: 168, 6: 177, 7: 168, 8: 153, 9: 148, 10: 152, 11: 144, 12: 128, 13: 134, 14: 118, 15: 130, 16: 131, 17: 154, 18: 148, 19: 175, 20: 156, 21: 154, 22: 172, 23: 159, 24: 158, 25: 135, 26: 128, 27: 164}
dBp9s4: ranges {0: 127, 1: 143, 2: 144, 3: 150, 4: 152, 5: 166, 6: 180, 7: 169, 8: 152, 9: 149, 10: 152, 11: 142, 12: 127, 13: 133, 14: 112, 15: 131, 16: 131, 17: 153, 18: 149, 19: 176, 20: 163, 21: 159, 22: 173, 23: 160, 24: 161, 25: 134, 26: 127, 27: 165}
dBpAs4: ranges {0: 127, 1: 140, 2: 143, 3: 150, 4: 150, 5: 169, 6: 176, 7: 166, 8: 151, 9: 148, 10: 152, 11: 141, 12: 130, 13: 133, 14: 115, 15: 127, 16: 133, 17: 152, 18: 151, 19: 174, 20: 159, 21: 157, 22: 171, 23: 157, 24: 163, 25: 132, 26: 127, 27: 166}
dBpBs4: ranges {0: 128, 1: 143, 2: 144, 3: 147, 4: 151, 5: 166, 6: 179, 7: 168, 8: 154, 9: 149, 10: 154, 11: 140, 12: 127, 13: 134, 14: 115, 15: 129, 16: 135, 17: 155, 18: 148, 19: 176, 20: 158, 21: 155, 22: 171, 23: 159, 24: 161, 25: 135, 26: 128, 27: 166}
dBpCs4: ranges {0: 129, 1: 142, 2: 144, 3: 150, 4: 153, 5: 164, 6: 178, 7: 165, 8: 152, 9: 148, 10: 155, 11: 140, 12: 126, 13: 136, 14: 116, 15: 128, 16: 132, 17: 152, 18: 147, 19: 174, 20: 160, 21: 155, 22: 174, 23: 159, 24: 158, 25: 136, 26: 122, 27: 166}
dBpDs4: ranges {0: 127, 1: 142, 2: 144, 3: 152, 4: 153, 5: 165, 6: 177, 7: 168, 8: 152, 9: 149, 10: 154, 11: 143, 12: 129, 13: 135, 14: 116, 15: 131, 16: 131, 17: 152, 18: 151, 19: 174, 20: 157, 21: 154, 22: 172, 23: 161, 24: 161, 25: 136, 26: 125, 27: 164}
dBpEs4: ranges {0: 125, 1: 139, 2: 143, 3: 149, 4: 152, 5: 168, 6: 179, 7: 167, 8: 153, 9: 148, 10: 155, 11: 142, 12: 129, 13: 131, 14: 117, 15: 130, 16: 132, 17: 153, 18: 149, 19: 173, 20: 162, 21: 154, 22: 172, 23: 161, 24: 161, 25: 132, 26: 128, 27: 167}
dCp0s4: ranges {0: 85, 1: 110, 2: 113, 3: 121, 4: 128, 5: 142, 6: 157, 7: 149, 8: 129, 9: 126, 10: 126, 11: 116, 12: 94, 13: 98, 14: 81, 15: 92, 16: 104, 17: 124, 18: 125, 19: 151, 20: 145, 21: 137, 22: 153, 23: 131, 24: 134, 25: 105, 26: 99, 27: 132}
dCp1s4: ranges {0: 106, 1: 128, 2: 125, 3: 137, 4: 140, 5: 156, 6: 169, 7: 162, 8: 142, 9: 136, 10: 138, 11: 129, 12: 110, 13: 118, 14: 97, 15: 112, 16: 114, 17: 141, 18: 137, 19: 164, 20: 155, 21: 149, 22: 163, 23: 147, 24: 148, 25: 120, 26: 113, 27: 149}
dCp2s4: ranges {0: 115, 1: 132, 2: 137, 3: 143, 4: 148, 5: 162, 6: 168, 7: 160, 8: 149, 9: 145, 10: 146, 11: 135, 12: 118, 13: 124, 14: 104, 15: 120, 16: 127, 17: 150, 18: 145, 19: 167, 20: 157, 21: 153, 22: 168, 23: 153, 24: 152, 25: 126, 26: 120, 27: 157}
dCp3s4: ranges {0: 123, 1: 139, 2: 138, 3: 146, 4: 152, 5: 164, 6: 174, 7: 165, 8: 151, 9: 145, 10: 148, 11: 140, 12: 123, 13: 129, 14: 111, 15: 125, 16: 129, 17: 150, 18: 148, 19: 172, 20: 159, 21: 152, 22: 172, 23: 158, 24: 157, 25: 127, 26: 125, 27: 159}
dCp4s4: ranges {0: 124, 1: 142, 2: 141, 3: 146, 4: 151, 5: 165, 6: 175, 7: 165, 8: 149, 9: 145, 10: 151, 11: 139, 12: 125, 13: 128, 14: 113, 15: 125, 16: 130, 17: 151, 18: 148, 19: 173, 20: 159, 21: 155, 22: 173, 23: 155, 24: 156, 25: 130, 26: 124, 27: 162}
dCp5s4: ranges {0: 121, 1: 137, 2: 138, 3: 144, 4: 152, 5: 164, 6: 176, 7: 166, 8: 151, 9: 147, 10: 153, 11: 141, 12: 125, 13: 128, 14: 115, 15: 126, 16: 129, 17: 152, 18: 148, 19: 175, 20: 160, 21: 154, 22: 174, 23: 158, 24: 158, 25: 132, 26: 123, 27: 160}
dCp6s4: ranges {0: 119, 1: 138, 2: 142, 3: 147, 4: 152, 5: 165, 6: 176, 7: 170, 8: 152, 9: 150, 10: 146, 11: 139, 12: 124, 13: 126, 14: 108, 15: 124, 16: 128, 17: 151, 18: 145, 19: 175, 20: 159, 21: 152, 22: 172, 23: 158, 24: 160, 25: 129, 26: 126, 27: 162}
dCp7s4: ranges {0: 123, 1: 139, 2: 138, 3: 147, 4: 149, 5: 163, 6: 174, 7: 164, 8: 152, 9: 147, 10: 149, 11: 138, 12: 126, 13: 128, 14: 107, 15: 125, 16: 128, 17: 151, 18: 148, 19: 173, 20: 160, 21: 153, 22: 173, 23: 159, 24: 157, 25: 130, 26: 120, 27: 159}
dCp8s4: ranges {0: 124, 1: 134, 2: 139, 3: 146, 4: 153, 5: 163, 6: 174, 7: 165, 8: 150, 9: 147, 10: 153, 11: 140, 12: 126, 13: 128, 14: 113, 15: 125, 16: 127, 17: 149, 18: 149, 19: 174, 20: 156, 21: 152, 22: 171, 23: 156, 24: 159, 25: 132, 26: 124, 27: 159}
dCp9s4: ranges {0: 125, 1: 140, 2: 138, 3: 148, 4: 150, 5: 167, 6: 178, 7: 166, 8: 149, 9: 142, 10: 152, 11: 142, 12: 124, 13: 131, 14: 109, 15: 128, 16: 129, 17: 151, 18: 150, 19: 173, 20: 160, 21: 154, 22: 170, 23: 157, 24: 151, 25: 134, 26: 126, 27: 162}
dCpAs4: ranges {0: 124, 1: 139, 2: 139, 3: 146, 4: 151, 5: 167, 6: 175, 7: 165, 8: 151, 9: 143, 10: 151, 11: 139, 12: 121, 13: 130, 14: 113, 15: 125, 16: 130, 17: 149, 18: 147, 19: 173, 20: 157, 21: 155, 22: 170, 23: 161, 24: 160, 25: 132, 26: 124, 27: 160}
dCpBs4: ranges {0: 122, 1: 137, 2: 142, 3: 146, 4: 150, 5: 162, 6: 175, 7: 166, 8: 151, 9: 147, 10: 152, 11: 140, 12: 128, 13: 127, 14: 114, 15: 127, 16: 128, 17: 154, 18: 149, 19: 172, 20: 158, 21: 155, 22: 171, 23: 156, 24: 158, 25: 133, 26: 124, 27: 163}
dCpCs4: ranges {0: 122, 1: 138, 2: 143, 3: 146, 4: 151, 5: 166, 6: 174, 7: 165, 8: 150, 9: 144, 10: 151, 11: 139, 12: 125, 13: 130, 14: 114, 15: 125, 16: 129, 17: 150, 18: 148, 19: 172, 20: 160, 21: 156, 22: 172, 23: 158, 24: 157, 25: 130, 26: 126, 27: 161}
dCpDs4: ranges {0: 122, 1: 138, 2: 143, 3: 146, 4: 150, 5: 162, 6: 173, 7: 164, 8: 150, 9: 147, 10: 145, 11: 141, 12: 126, 13: 131, 14: 112, 15: 128, 16: 129, 17: 151, 18: 147, 19: 174, 20: 159, 21: 156, 22: 173, 23: 157, 24: 158, 25: 133, 26: 125, 27: 163}
dCpEs4: ranges {0: 117, 1: 140, 2: 141, 3: 147, 4: 151, 5: 165, 6: 174, 7: 167, 8: 151, 9: 147, 10: 150, 11: 139, 12: 126, 13: 129, 14: 115, 15: 122, 16: 127, 17: 152, 18: 147, 19: 174, 20: 158, 21: 154, 22: 172, 23: 158, 24: 160, 25: 133, 26: 121, 27: 161}
dDp0s4: ranges {0: 90, 1: 112, 2: 114, 3: 122, 4: 128, 5: 143, 6: 158, 7: 148, 8: 131, 9: 127, 10: 124, 11: 117, 12: 93, 13: 100, 14: 81, 15: 93, 16: 104, 17: 124, 18: 127, 19: 153, 20: 144, 21: 138, 22: 157, 23: 133, 24: 133, 25: 102, 26: 94, 27: 131}
dDp1s4: ranges {0: 105, 1: 126, 2: 129, 3: 135, 4: 139, 5: 154, 6: 164, 7: 159, 8: 143, 9: 135, 10: 139, 11: 127, 12: 111, 13: 114, 14: 94, 15: 111, 16: 116, 17: 137, 18: 138, 19: 163, 20: 152, 21: 148, 22: 165, 23: 148, 24: 148, 25: 117, 26: 110, 27: 144}
dDp2s4: ranges {0: 118, 1: 133, 2: 134, 3: 141, 4: 143, 5: 158, 6: 175, 7: 165, 8: 147, 9: 142, 10: 143, 11: 137, 12: 119, 13: 125, 14: 105, 15: 120, 16: 123, 17: 148, 18: 143, 19: 170, 20: 155, 21: 153, 22: 167, 23: 152, 24: 153, 25: 124, 26: 117, 27: 154}
dDp3s4: ranges {0: 120, 1: 133, 2: 137, 3: 145, 4: 148, 5: 161, 6: 175, 7: 165, 8: 148, 9: 144, 10: 148, 11: 141, 12: 124, 13: 125, 14: 110, 15: 122, 16: 124, 17: 150, 18: 145, 19: 172, 20: 155, 21: 152, 22: 171, 23: 155, 24: 154, 25: 130, 26: 119, 27: 155}
dDp4s4: ranges {0: 120, 1: 136, 2: 141, 3: 142, 4: 151, 5: 164, 6: 175, 7: 165, 8: 147, 9: 146, 10: 149, 11: 139, 12: 123, 13: 125, 14: 109, 15: 122, 16: 128, 17: 149, 18: 145, 19: 174, 20: 159, 21: 153, 22: 172, 23: 154, 24: 159, 25: 129, 26: 123, 27: 159}
dDp5s4: ranges {0: 120, 1: 133, 2: 139, 3: 141, 4: 150, 5: 162, 6: 179, 7: 167, 8: 151, 9: 142, 10: 151, 11: 136, 12: 123, 13: 124, 14: 109, 15: 121, 16: 128, 17: 148, 18: 147, 19: 174, 20: 158, 21: 153, 22: 169, 23: 159, 24: 157, 25: 128, 26: 122, 27: 159}
dDp6s4: ranges {0: 116, 1: 134, 2: 138, 3: 145, 4: 149, 5: 161, 6: 175, 7: 164, 8: 146, 9: 146, 10: 150, 11: 138, 12: 121, 13: 123, 14: 109, 15: 122, 16: 128, 17: 148, 18: 146, 19: 170, 20: 159, 21: 152, 22: 172, 23: 157, 24: 158, 25: 128, 26: 119, 27: 161}
dDp7s4: ranges {0: 120, 1: 138, 2: 135, 3: 144, 4: 150, 5: 164, 6: 172, 7: 165, 8: 149, 9: 144, 10: 149, 11: 138, 12: 123, 13: 125, 14: 110, 15: 124, 16: 126, 17: 149, 18: 148, 19: 173, 20: 158, 21: 151, 22: 170, 23: 157, 24: 157, 25: 128, 26: 122, 27: 158}
dDp8s4: ranges {0: 120, 1: 136, 2: 138, 3: 141, 4: 151, 5: 163, 6: 176, 7: 170, 8: 149, 9: 147, 10: 145, 11: 139, 12: 124, 13: 129, 14: 110, 15: 121, 16: 120, 17: 151, 18: 147, 19: 172, 20: 153, 21: 154, 22: 167, 23: 156, 24: 156, 25: 131, 26: 122, 27: 152}
dDp9s4: ranges {0: 119, 1: 138, 2: 138, 3: 142, 4: 149, 5: 162, 6: 175, 7: 162, 8: 149, 9: 145, 10: 150, 11: 138, 12: 123, 13: 128, 14: 110, 15: 121, 16: 125, 17: 150, 18: 147, 19: 174, 20: 159, 21: 152, 22: 172, 23: 157, 24: 157, 25: 129, 26: 122, 27: 153}
dDpAs4: ranges {0: 117, 1: 135, 2: 139, 3: 143, 4: 148, 5: 165, 6: 175, 7: 160, 8: 148, 9: 145, 10: 149, 11: 138, 12: 122, 13: 123, 14: 113, 15: 124, 16: 128, 17: 151, 18: 143, 19: 172, 20: 158, 21: 155, 22: 171, 23: 155, 24: 157, 25: 131, 26: 123, 27: 159}
dDpBs4: ranges {0: 121, 1: 137, 2: 134, 3: 144, 4: 149, 5: 162, 6: 171, 7: 164, 8: 151, 9: 146, 10: 149, 11: 137, 12: 123, 13: 124, 14: 107, 15: 124, 16: 128, 17: 150, 18: 143, 19: 171, 20: 157, 21: 152, 22: 170, 23: 157, 24: 154, 25: 126, 26: 119, 27: 158}
dDpCs4: ranges {0: 120, 1: 140, 2: 137, 3: 145, 4: 148, 5: 163, 6: 175, 7: 162, 8: 148, 9: 146, 10: 148, 11: 135, 12: 120, 13: 129, 14: 110, 15: 123, 16: 127, 17: 147, 18: 146, 19: 174, 20: 158, 21: 152, 22: 170, 23: 156, 24: 158, 25: 130, 26: 123, 27: 161}
dDpDs4: ranges {0: 123, 1: 137, 2: 136, 3: 145, 4: 151, 5: 166, 6: 174, 7: 162, 8: 148, 9: 141, 10: 149, 11: 136, 12: 123, 13: 129, 14: 108, 15: 124, 16: 124, 17: 148, 18: 144, 19: 173, 20: 159, 21: 152, 22: 172, 23: 155, 24: 157, 25: 129, 26: 124, 27: 156}
dDpEs4: ranges {0: 117, 1: 137, 2: 136, 3: 143, 4: 148, 5: 162, 6: 175, 7: 163, 8: 148, 9: 143, 10: 148, 11: 139, 12: 118, 13: 123, 14: 108, 15: 125, 16: 128, 17: 147, 18: 144, 19: 174, 20: 157, 21: 154, 22: 170, 23: 155, 24: 153, 25: 127, 26: 121, 27: 160}
dEp0s4: ranges {0: 92, 1: 111, 2: 112, 3: 120, 4: 126, 5: 141, 6: 159, 7: 147, 8: 129, 9: 126, 10: 126, 11: 117, 12: 94, 13: 101, 14: 82, 15: 93, 16: 101, 17: 124, 18: 125, 19: 152, 20: 144, 21: 137, 22: 157, 23: 133, 24: 137, 25: 105, 26: 98, 27: 133}
dEp1s4: ranges {0: 104, 1: 127, 2: 125, 3: 135, 4: 139, 5: 153, 6: 168, 7: 156, 8: 143, 9: 136, 10: 139, 11: 127, 12: 110, 13: 114, 14: 93, 15: 111, 16: 112, 17: 138, 18: 134, 19: 162, 20: 151, 21: 146, 22: 162, 23: 148, 24: 143, 25: 116, 26: 107, 27: 146}
dEp2s4: ranges {0: 112, 1: 132, 2: 133, 3: 140, 4: 145, 5: 159, 6: 168, 7: 162, 8: 148, 9: 142, 10: 144, 11: 133, 12: 119, 13: 118, 14: 105, 15: 115, 16: 118, 17: 147, 18: 143, 19: 169, 20: 158, 21: 151, 22: 168, 23: 149, 24: 149, 25: 124, 26: 116, 27: 155}
dEp3s4: ranges {0: 118, 1: 131, 2: 135, 3: 142, 4: 147, 5: 162, 6: 175, 7: 165, 8: 147, 9: 143, 10: 148, 11: 135, 12: 120, 13: 124, 14: 108, 15: 122, 16: 124, 17: 148, 18: 143, 19: 168, 20: 156, 21: 151, 22: 170, 23: 154, 24: 155, 25: 130, 26: 122, 27: 158}
dEp4s4: ranges {0: 119, 1: 135, 2: 136, 3: 141, 4: 147, 5: 161, 6: 174, 7: 166, 8: 144, 9: 144, 10: 148, 11: 138, 12: 121, 13: 124, 14: 108, 15: 123, 16: 124, 17: 147, 18: 143, 19: 170, 20: 159, 21: 154, 22: 171, 23: 154, 24: 155, 25: 128, 26: 121, 27: 154}
dEp5s4: ranges {0: 118, 1: 133, 2: 135, 3: 141, 4: 147, 5: 162, 6: 168, 7: 165, 8: 148, 9: 145, 10: 146, 11: 135, 12: 122, 13: 124, 14: 109, 15: 122, 16: 125, 17: 149, 18: 146, 19: 171, 20: 156, 21: 154, 22: 171, 23: 156, 24: 156, 25: 128, 26: 117, 27: 156}
dEp6s4: ranges {0: 118, 1: 135, 2: 136, 3: 140, 4: 149, 5: 160, 6: 173, 7: 164, 8: 147, 9: 145, 10: 150, 11: 138, 12: 123, 13: 124, 14: 104, 15: 123, 16: 124, 17: 147, 18: 143, 19: 171, 20: 157, 21: 152, 22: 172, 23: 156, 24: 154, 25: 128, 26: 121, 27: 155}
dEp7s4: ranges {0: 120, 1: 134, 2: 137, 3: 139, 4: 150, 5: 161, 6: 173, 7: 167, 8: 149, 9: 144, 10: 149, 11: 138, 12: 121, 13: 124, 14: 109, 15: 121, 16: 124, 17: 149, 18: 144, 19: 168, 20: 157, 21: 150, 22: 169, 23: 151, 24: 156, 25: 127, 26: 119, 27: 155}
dEp8s4: ranges {0: 118, 1: 136, 2: 138, 3: 142, 4: 147, 5: 162, 6: 170, 7: 164, 8: 148, 9: 141, 10: 149, 11: 135, 12: 121, 13: 124, 14: 110, 15: 120, 16: 126, 17: 147, 18: 147, 19: 171, 20: 157, 21: 151, 22: 171, 23: 153, 24: 156, 25: 126, 26: 119, 27: 158}
dEp9s4: ranges {0: 118, 1: 135, 2: 137, 3: 140, 4: 148, 5: 161, 6: 171, 7: 160, 8: 147, 9: 143, 10: 146, 11: 137, 12: 119, 13: 125, 14: 105, 15: 124, 16: 123, 17: 145, 18: 144, 19: 169, 20: 157, 21: 152, 22: 171, 23: 155, 24: 157, 25: 129, 26: 122, 27: 158}
dEpAs4: ranges {0: 116, 1: 134, 2: 137, 3: 143, 4: 148, 5: 161, 6: 171, 7: 164, 8: 147, 9: 145, 10: 148, 11: 138, 12: 121, 13: 127, 14: 106, 15: 121, 16: 125, 17: 148, 18: 142, 19: 169, 20: 158, 21: 153, 22: 170, 23: 155, 24: 153, 25: 128, 26: 119, 27: 155}
dEpBs4: ranges {0: 120, 1: 134, 2: 135, 3: 142, 4: 148, 5: 160, 6: 172, 7: 162, 8: 148, 9: 143, 10: 146, 11: 138, 12: 119, 13: 124, 14: 110, 15: 123, 16: 125, 17: 147, 18: 143, 19: 171, 20: 157, 21: 154, 22: 169, 23: 151, 24: 157, 25: 128, 26: 122, 27: 160}
dEpCs4: ranges {0: 119, 1: 137, 2: 133, 3: 139, 4: 149, 5: 159, 6: 170, 7: 163, 8: 148, 9: 144, 10: 149, 11: 137, 12: 123, 13: 126, 14: 110, 15: 120, 16: 127, 17: 146, 18: 145, 19: 168, 20: 156, 21: 152, 22: 170, 23: 156, 24: 154, 25: 126, 26: 122, 27: 156}
dEpDs4: ranges {0: 121, 1: 135, 2: 134, 3: 143, 4: 148, 5: 162, 6: 171, 7: 162, 8: 149, 9: 143, 10: 150, 11: 133, 12: 119, 13: 126, 14: 106, 15: 121, 16: 126, 17: 148, 18: 142, 19: 171, 20: 157, 21: 151, 22: 171, 23: 156, 24: 155, 25: 129, 26: 119, 27: 159}
dEpEs4: ranges {0: 116, 1: 136, 2: 137, 3: 143, 4: 150, 5: 160, 6: 176, 7: 167, 8: 147, 9: 144, 10: 146, 11: 133, 12: 123, 13: 124, 14: 107, 15: 120, 16: 127, 17: 149, 18: 143, 19: 170, 20: 158, 21: 154, 22: 169, 23: 157, 24: 157, 25: 129, 26: 120, 27: 159}
"""

undersampled_scan = """
d1p3s4: ranges {0: 154, 1: 162, 2: 160, 3: 163, 4: 153, 5: 172, 6: 184, 7: 172, 8: 149, 9: 155, 10: 166, 11: 156, 12: 136, 13: 146, 14: 147, 15: 147, 16: 143, 17: 165, 18: 160, 19: 184, 20: 170, 21: 164, 22: 176, 23: 161, 24: 174, 25: 136, 26: 146, 27: 184}
d1p6s4: ranges {0: 160, 1: 165, 2: 165, 3: 165, 4: 162, 5: 174, 6: 185, 7: 173, 8: 156, 9: 158, 10: 172, 11: 161, 12: 140, 13: 155, 14: 154, 15: 153, 16: 152, 17: 168, 18: 163, 19: 187, 20: 171, 21: 167, 22: 182, 23: 164, 24: 177, 25: 147, 26: 148, 27: 192}
d1p9s4: ranges {0: 164, 1: 167, 2: 166, 3: 168, 4: 166, 5: 176, 6: 187, 7: 180, 8: 158, 9: 155, 10: 175, 11: 165, 12: 149, 13: 160, 14: 151, 15: 157, 16: 154, 17: 172, 18: 169, 19: 188, 20: 174, 21: 168, 22: 181, 23: 168, 24: 178, 25: 149, 26: 148, 27: 196}
d1pCs4: ranges {0: 163, 1: 165, 2: 170, 3: 168, 4: 168, 5: 179, 6: 187, 7: 176, 8: 159, 9: 160, 10: 176, 11: 164, 12: 150, 13: 164, 14: 155, 15: 159, 16: 154, 17: 170, 18: 167, 19: 186, 20: 167, 21: 172, 22: 179, 23: 167, 24: 180, 25: 150, 26: 152, 27: 196}
d1pFs4: ranges {0: 167, 1: 169, 2: 169, 3: 170, 4: 169, 5: 180, 6: 187, 7: 175, 8: 158, 9: 161, 10: 176, 11: 164, 12: 150, 13: 162, 14: 155, 15: 160, 16: 155, 17: 173, 18: 170, 19: 187, 20: 173, 21: 167, 22: 185, 23: 168, 24: 181, 25: 154, 26: 152, 27: 197}
d2p0s4: ranges {0: 40, 1: 103, 2: 85, 3: 123, 4: 53, 5: 126, 6: 163, 7: 147, 8: 110, 9: 116, 10: 126, 11: 116, 12: 38, 13: 62, 14: 54, 15: 28, 16: 78, 17: 113, 18: 116, 19: 161, 20: 142, 21: 123, 22: 151, 23: 104, 24: 133, 25: 46, 26: 100, 27: 97}
d2p3s4: ranges {0: 153, 1: 166, 2: 165, 3: 163, 4: 160, 5: 175, 6: 184, 7: 175, 8: 154, 9: 159, 10: 170, 11: 160, 12: 144, 13: 154, 14: 150, 15: 151, 16: 150, 17: 167, 18: 163, 19: 185, 20: 167, 21: 165, 22: 178, 23: 165, 24: 178, 25: 144, 26: 146, 27: 193}
d2p6s4: ranges {0: 167, 1: 168, 2: 168, 3: 168, 4: 169, 5: 180, 6: 187, 7: 175, 8: 158, 9: 161, 10: 174, 11: 161, 12: 150, 13: 165, 14: 157, 15: 160, 16: 156, 17: 169, 18: 170, 19: 187, 20: 170, 21: 170, 22: 182, 23: 170, 24: 182, 25: 152, 26: 149, 27: 200}
d2p9s4: ranges {0: 171, 1: 171, 2: 170, 3: 169, 4: 168, 5: 183, 6: 193, 7: 182, 8: 163, 9: 161, 10: 176, 11: 163, 12: 154, 13: 165, 14: 155, 15: 159, 16: 157, 17: 170, 18: 167, 19: 187, 20: 172, 21: 174, 22: 184, 23: 172, 24: 179, 25: 152, 26: 152, 27: 202}
d2pCs4: ranges {0: 172, 1: 170, 2: 174, 3: 169, 4: 173, 5: 180, 6: 191, 7: 180, 8: 162, 9: 166, 10: 178, 11: 164, 12: 152, 13: 168, 14: 157, 15: 162, 16: 159, 17: 173, 18: 172, 19: 193, 20: 175, 21: 174, 22: 184, 23: 176, 24: 184, 25: 156, 26: 157, 27: 205}
d2pFs4: ranges {0: 167, 1: 171, 2: 174, 3: 170, 4: 172, 5: 181, 6: 189, 7: 180, 8: 163, 9: 165, 10: 177, 11: 163, 12: 153, 13: 166, 14: 160, 15: 161, 16: 158, 17: 176, 18: 173, 19: 192, 20: 176, 21: 174, 22: 185, 23: 176, 24: 180, 25: 154, 26: 156, 27: 203}
d3p0s4: ranges {0: 61, 1: 111, 2: 94, 3: 125, 4: 102, 5: 138, 6: 168, 7: 150, 8: 121, 9: 120, 10: 126, 11: 117, 12: 66, 13: 77, 14: 63, 15: 60, 16: 89, 17: 127, 18: 123, 19: 159, 20: 150, 21: 132, 22: 154, 23: 118, 24: 135, 25: 72, 26: 98, 27: 113}
d3p3s4: ranges {0: 157, 1: 163, 2: 164, 3: 165, 4: 160, 5: 177, 6: 183, 7: 173, 8: 158, 9: 158, 10: 170, 11: 159, 12: 146, 13: 160, 14: 146, 15: 148, 16: 151, 17: 172, 18: 165, 19: 188, 20: 170, 21: 164, 22: 180, 23: 169, 24: 177, 25: 149, 26: 152, 27: 192}
d3p6s4: ranges {0: 168, 1: 169, 2: 171, 3: 166, 4: 168, 5: 180, 6: 188, 7: 179, 8: 160, 9: 163, 10: 174, 11: 162, 12: 152, 13: 166, 14: 154, 15: 158, 16: 159, 17: 171, 18: 170, 19: 190, 20: 174, 21: 172, 22: 185, 23: 172, 24: 180, 25: 154, 26: 152, 27: 201}
d3p9s4: ranges {0: 168, 1: 166, 2: 172, 3: 170, 4: 172, 5: 180, 6: 190, 7: 179, 8: 162, 9: 165, 10: 175, 11: 162, 12: 155, 13: 168, 14: 156, 15: 161, 16: 159, 17: 172, 18: 174, 19: 192, 20: 172, 21: 171, 22: 187, 23: 177, 24: 181, 25: 157, 26: 155, 27: 202}
d3pCs4: ranges {0: 169, 1: 168, 2: 173, 3: 169, 4: 172, 5: 182, 6: 188, 7: 181, 8: 166, 9: 164, 10: 176, 11: 166, 12: 153, 13: 171, 14: 156, 15: 159, 16: 158, 17: 174, 18: 173, 19: 193, 20: 175, 21: 168, 22: 189, 23: 176, 24: 179, 25: 156, 26: 154, 27: 200}
d3pFs4: ranges {0: 163, 1: 169, 2: 171, 3: 171, 4: 172, 5: 185, 6: 188, 7: 178, 8: 164, 9: 163, 10: 173, 11: 166, 12: 156, 13: 164, 14: 156, 15: 160, 16: 161, 17: 178, 18: 169, 19: 194, 20: 176, 21: 169, 22: 186, 23: 172, 24: 178, 25: 158, 26: 157, 27: 200}
d4p0s4: ranges {0: 74, 1: 110, 2: 96, 3: 125, 4: 109, 5: 143, 6: 166, 7: 152, 8: 124, 9: 124, 10: 128, 11: 117, 12: 75, 13: 86, 14: 71, 15: 74, 16: 95, 17: 126, 18: 129, 19: 161, 20: 152, 21: 138, 22: 158, 23: 124, 24: 136, 25: 83, 26: 98, 27: 119}
d4p3s4: ranges {0: 151, 1: 163, 2: 162, 3: 164, 4: 163, 5: 174, 6: 185, 7: 173, 8: 159, 9: 157, 10: 168, 11: 159, 12: 143, 13: 158, 14: 143, 15: 146, 16: 150, 17: 169, 18: 162, 19: 188, 20: 167, 21: 162, 22: 180, 23: 166, 24: 173, 25: 150, 26: 147, 27: 188}
d4p6s4: ranges {0: 163, 1: 165, 2: 166, 3: 161, 4: 167, 5: 181, 6: 190, 7: 177, 8: 160, 9: 160, 10: 171, 11: 163, 12: 150, 13: 162, 14: 151, 15: 156, 16: 158, 17: 171, 18: 168, 19: 189, 20: 173, 21: 169, 22: 181, 23: 172, 24: 176, 25: 156, 26: 153, 27: 197}
d4p9s4: ranges {0: 163, 1: 169, 2: 166, 3: 168, 4: 169, 5: 182, 6: 189, 7: 175, 8: 161, 9: 161, 10: 173, 11: 161, 12: 152, 13: 162, 14: 150, 15: 155, 16: 157, 17: 170, 18: 169, 19: 188, 20: 170, 21: 167, 22: 186, 23: 175, 24: 179, 25: 154, 26: 153, 27: 194}
d4pCs4: ranges {0: 163, 1: 167, 2: 166, 3: 169, 4: 170, 5: 179, 6: 188, 7: 179, 8: 162, 9: 163, 10: 174, 11: 165, 12: 155, 13: 165, 14: 149, 15: 154, 16: 156, 17: 171, 18: 167, 19: 190, 20: 173, 21: 165, 22: 186, 23: 171, 24: 176, 25: 156, 26: 150, 27: 194}
d4pFs4: ranges {0: 162, 1: 166, 2: 167, 3: 168, 4: 168, 5: 181, 6: 186, 7: 179, 8: 165, 9: 163, 10: 171, 11: 160, 12: 153, 13: 165, 14: 148, 15: 153, 16: 157, 17: 172, 18: 167, 19: 186, 20: 172, 21: 166, 22: 187, 23: 175, 24: 177, 25: 153, 26: 149, 27: 193}
d5p0s4: ranges {0: 80, 1: 111, 2: 106, 3: 126, 4: 116, 5: 145, 6: 163, 7: 153, 8: 127, 9: 126, 10: 126, 11: 120, 12: 82, 13: 91, 14: 73, 15: 79, 16: 98, 17: 127, 18: 124, 19: 158, 20: 152, 21: 135, 22: 156, 23: 131, 24: 137, 25: 89, 26: 97, 27: 127}
d5p3s4: ranges {0: 147, 1: 158, 2: 160, 3: 161, 4: 160, 5: 175, 6: 183, 7: 172, 8: 159, 9: 157, 10: 163, 11: 153, 12: 142, 13: 152, 14: 138, 15: 139, 16: 150, 17: 163, 18: 161, 19: 185, 20: 166, 21: 162, 22: 182, 23: 165, 24: 166, 25: 150, 26: 143, 27: 187}
d5p6s4: ranges {0: 156, 1: 161, 2: 161, 3: 164, 4: 165, 5: 178, 6: 188, 7: 175, 8: 159, 9: 161, 10: 166, 11: 160, 12: 149, 13: 159, 14: 147, 15: 147, 16: 155, 17: 172, 18: 165, 19: 188, 20: 170, 21: 166, 22: 185, 23: 170, 24: 176, 25: 153, 26: 147, 27: 192}
d5p9s4: ranges {0: 156, 1: 164, 2: 164, 3: 166, 4: 167, 5: 176, 6: 187, 7: 173, 8: 162, 9: 157, 10: 168, 11: 160, 12: 148, 13: 159, 14: 144, 15: 150, 16: 155, 17: 171, 18: 165, 19: 186, 20: 169, 21: 166, 22: 185, 23: 170, 24: 173, 25: 152, 26: 150, 27: 189}
d5pCs4: ranges {0: 148, 1: 166, 2: 162, 3: 166, 4: 166, 5: 178, 6: 189, 7: 182, 8: 162, 9: 157, 10: 168, 11: 162, 12: 148, 13: 157, 14: 142, 15: 149, 16: 153, 17: 169, 18: 164, 19: 190, 20: 169, 21: 163, 22: 182, 23: 172, 24: 175, 25: 153, 26: 146, 27: 189}
d5pFs4: ranges {0: 155, 1: 161, 2: 163, 3: 163, 4: 169, 5: 179, 6: 189, 7: 179, 8: 161, 9: 161, 10: 167, 11: 162, 12: 147, 13: 160, 14: 144, 15: 149, 16: 152, 17: 170, 18: 165, 19: 190, 20: 172, 21: 167, 22: 186, 23: 173, 24: 175, 25: 153, 26: 147, 27: 184}
d6p0s4: ranges {0: 77, 1: 111, 2: 108, 3: 128, 4: 121, 5: 145, 6: 164, 7: 152, 8: 128, 9: 127, 10: 127, 11: 119, 12: 83, 13: 93, 14: 76, 15: 86, 16: 102, 17: 128, 18: 128, 19: 157, 20: 149, 21: 133, 22: 159, 23: 132, 24: 136, 25: 100, 26: 99, 27: 128}
d6p3s4: ranges {0: 143, 1: 153, 2: 156, 3: 158, 4: 159, 5: 173, 6: 183, 7: 171, 8: 154, 9: 154, 10: 158, 11: 152, 12: 139, 13: 147, 14: 131, 15: 137, 16: 145, 17: 163, 18: 159, 19: 178, 20: 164, 21: 161, 22: 179, 23: 166, 24: 168, 25: 148, 26: 139, 27: 183}
d6p6s4: ranges {0: 151, 1: 160, 2: 160, 3: 162, 4: 164, 5: 177, 6: 188, 7: 175, 8: 160, 9: 160, 10: 164, 11: 160, 12: 144, 13: 156, 14: 140, 15: 145, 16: 150, 17: 170, 18: 163, 19: 185, 20: 169, 21: 166, 22: 185, 23: 169, 24: 174, 25: 153, 26: 142, 27: 186}
d6p9s4: ranges {0: 152, 1: 159, 2: 157, 3: 163, 4: 162, 5: 175, 6: 183, 7: 174, 8: 159, 9: 155, 10: 166, 11: 156, 12: 144, 13: 151, 14: 142, 15: 145, 16: 150, 17: 170, 18: 164, 19: 188, 20: 169, 21: 165, 22: 183, 23: 169, 24: 169, 25: 147, 26: 144, 27: 188}
d6pCs4: ranges {0: 152, 1: 156, 2: 159, 3: 159, 4: 161, 5: 175, 6: 185, 7: 177, 8: 162, 9: 154, 10: 167, 11: 157, 12: 144, 13: 156, 14: 139, 15: 146, 16: 152, 17: 169, 18: 163, 19: 186, 20: 172, 21: 165, 22: 183, 23: 169, 24: 173, 25: 151, 26: 145, 27: 186}
d6pFs4: ranges {0: 144, 1: 156, 2: 158, 3: 162, 4: 164, 5: 180, 6: 185, 7: 174, 8: 161, 9: 159, 10: 167, 11: 157, 12: 143, 13: 156, 14: 140, 15: 147, 16: 146, 17: 169, 18: 162, 19: 185, 20: 167, 21: 162, 22: 181, 23: 166, 24: 174, 25: 149, 26: 142, 27: 185}
d7p0s4: ranges {0: 85, 1: 113, 2: 108, 3: 124, 4: 123, 5: 144, 6: 160, 7: 153, 8: 131, 9: 124, 10: 128, 11: 117, 12: 89, 13: 94, 14: 76, 15: 89, 16: 103, 17: 127, 18: 128, 19: 156, 20: 148, 21: 138, 22: 157, 23: 130, 24: 136, 25: 95, 26: 100, 27: 130}
d7p3s4: ranges {0: 138, 1: 152, 2: 152, 3: 158, 4: 157, 5: 173, 6: 184, 7: 171, 8: 159, 9: 149, 10: 159, 11: 152, 12: 138, 13: 144, 14: 131, 15: 137, 16: 145, 17: 163, 18: 155, 19: 182, 20: 167, 21: 160, 22: 177, 23: 164, 24: 160, 25: 145, 26: 137, 27: 178}
d7p6s4: ranges {0: 138, 1: 155, 2: 155, 3: 156, 4: 162, 5: 174, 6: 184, 7: 174, 8: 157, 9: 155, 10: 164, 11: 156, 12: 143, 13: 149, 14: 136, 15: 142, 16: 145, 17: 164, 18: 162, 19: 185, 20: 167, 21: 161, 22: 181, 23: 168, 24: 170, 25: 148, 26: 141, 27: 182}
d7p9s4: ranges {0: 145, 1: 154, 2: 154, 3: 160, 4: 161, 5: 175, 6: 180, 7: 173, 8: 158, 9: 152, 10: 163, 11: 155, 12: 139, 13: 149, 14: 133, 15: 141, 16: 143, 17: 161, 18: 160, 19: 186, 20: 168, 21: 162, 22: 180, 23: 165, 24: 168, 25: 147, 26: 139, 27: 182}
d7pCs4: ranges {0: 143, 1: 156, 2: 157, 3: 158, 4: 161, 5: 171, 6: 184, 7: 172, 8: 161, 9: 155, 10: 160, 11: 149, 12: 141, 13: 150, 14: 136, 15: 141, 16: 146, 17: 164, 18: 161, 19: 184, 20: 166, 21: 162, 22: 178, 23: 165, 24: 168, 25: 147, 26: 140, 27: 179}
d7pFs4: ranges {0: 146, 1: 154, 2: 154, 3: 158, 4: 163, 5: 171, 6: 186, 7: 177, 8: 159, 9: 153, 10: 163, 11: 156, 12: 140, 13: 149, 14: 136, 15: 138, 16: 147, 17: 162, 18: 157, 19: 181, 20: 166, 21: 164, 22: 182, 23: 167, 24: 169, 25: 145, 26: 138, 27: 179}
d8p0s4: ranges {0: 86, 1: 113, 2: 109, 3: 120, 4: 125, 5: 144, 6: 160, 7: 152, 8: 129, 9: 126, 10: 128, 11: 119, 12: 89, 13: 96, 14: 80, 15: 89, 16: 102, 17: 127, 18: 125, 19: 155, 20: 150, 21: 135, 22: 157, 23: 135, 24: 132, 25: 99, 26: 98, 27: 131}
d8p3s4: ranges {0: 135, 1: 149, 2: 151, 3: 152, 4: 157, 5: 168, 6: 178, 7: 168, 8: 154, 9: 152, 10: 160, 11: 149, 12: 131, 13: 142, 14: 124, 15: 136, 16: 135, 17: 160, 18: 157, 19: 178, 20: 166, 21: 157, 22: 177, 23: 164, 24: 157, 25: 140, 26: 134, 27: 176}
d8p6s4: ranges {0: 142, 1: 152, 2: 150, 3: 160, 4: 160, 5: 171, 6: 184, 7: 173, 8: 157, 9: 154, 10: 163, 11: 151, 12: 137, 13: 143, 14: 128, 15: 138, 16: 142, 17: 165, 18: 158, 19: 180, 20: 164, 21: 161, 22: 180, 23: 161, 24: 164, 25: 145, 26: 136, 27: 177}
d8p9s4: ranges {0: 142, 1: 148, 2: 151, 3: 156, 4: 157, 5: 170, 6: 181, 7: 173, 8: 158, 9: 154, 10: 161, 11: 152, 12: 136, 13: 145, 14: 129, 15: 137, 16: 144, 17: 161, 18: 159, 19: 181, 20: 164, 21: 159, 22: 179, 23: 164, 24: 169, 25: 140, 26: 137, 27: 173}
d8pCs4: ranges {0: 142, 1: 153, 2: 153, 3: 156, 4: 158, 5: 171, 6: 182, 7: 171, 8: 155, 9: 153, 10: 161, 11: 151, 12: 136, 13: 146, 14: 130, 15: 139, 16: 138, 17: 163, 18: 157, 19: 181, 20: 165, 21: 160, 22: 180, 23: 162, 24: 168, 25: 143, 26: 134, 27: 163}
d8pFs4: ranges {0: 140, 1: 152, 2: 150, 3: 156, 4: 160, 5: 172, 6: 184, 7: 171, 8: 157, 9: 152, 10: 159, 11: 149, 12: 138, 13: 146, 14: 130, 15: 138, 16: 142, 17: 164, 18: 158, 19: 182, 20: 164, 21: 161, 22: 177, 23: 166, 24: 166, 25: 144, 26: 137, 27: 175}
d9p0s4: ranges {0: 89, 1: 112, 2: 111, 3: 123, 4: 124, 5: 144, 6: 162, 7: 154, 8: 129, 9: 126, 10: 129, 11: 117, 12: 89, 13: 99, 14: 78, 15: 88, 16: 104, 17: 125, 18: 127, 19: 155, 20: 147, 21: 136, 22: 158, 23: 138, 24: 135, 25: 101, 26: 97, 27: 130}
d9p3s4: ranges {0: 132, 1: 145, 2: 145, 3: 152, 4: 153, 5: 166, 6: 180, 7: 165, 8: 152, 9: 150, 10: 157, 11: 148, 12: 130, 13: 140, 14: 123, 15: 130, 16: 139, 17: 155, 18: 152, 19: 177, 20: 165, 21: 158, 22: 176, 23: 159, 24: 161, 25: 140, 26: 131, 27: 168}
d9p6s4: ranges {0: 135, 1: 147, 2: 147, 3: 152, 4: 157, 5: 169, 6: 183, 7: 168, 8: 156, 9: 151, 10: 160, 11: 146, 12: 134, 13: 139, 14: 125, 15: 134, 16: 138, 17: 156, 18: 155, 19: 181, 20: 163, 21: 158, 22: 177, 23: 160, 24: 165, 25: 141, 26: 133, 27: 171}
d9p9s4: ranges {0: 131, 1: 147, 2: 148, 3: 151, 4: 155, 5: 169, 6: 179, 7: 168, 8: 155, 9: 150, 10: 155, 11: 149, 12: 130, 13: 142, 14: 125, 15: 133, 16: 138, 17: 161, 18: 155, 19: 180, 20: 164, 21: 163, 22: 177, 23: 166, 24: 165, 25: 138, 26: 131, 27: 174}
d9pCs4: ranges {0: 136, 1: 148, 2: 151, 3: 149, 4: 156, 5: 169, 6: 184, 7: 173, 8: 154, 9: 149, 10: 160, 11: 151, 12: 135, 13: 135, 14: 122, 15: 135, 16: 141, 17: 158, 18: 154, 19: 181, 20: 162, 21: 159, 22: 177, 23: 163, 24: 165, 25: 140, 26: 133, 27: 174}
d9pFs4: ranges {0: 134, 1: 149, 2: 150, 3: 152, 4: 158, 5: 168, 6: 179, 7: 169, 8: 156, 9: 150, 10: 159, 11: 150, 12: 135, 13: 141, 14: 126, 15: 136, 16: 136, 17: 159, 18: 157, 19: 176, 20: 161, 21: 157, 22: 176, 23: 161, 24: 165, 25: 139, 26: 134, 27: 170}
dAp0s4: ranges {0: 88, 1: 112, 2: 111, 3: 125, 4: 126, 5: 142, 6: 160, 7: 150, 8: 130, 9: 127, 10: 126, 11: 116, 12: 90, 13: 99, 14: 83, 15: 92, 16: 99, 17: 124, 18: 122, 19: 155, 20: 145, 21: 140, 22: 154, 23: 135, 24: 137, 25: 103, 26: 97, 27: 130}
dAp3s4: ranges {0: 124, 1: 144, 2: 145, 3: 151, 4: 152, 5: 166, 6: 176, 7: 165, 8: 150, 9: 147, 10: 151, 11: 144, 12: 131, 13: 132, 14: 117, 15: 130, 16: 135, 17: 156, 18: 149, 19: 174, 20: 162, 21: 155, 22: 174, 23: 160, 24: 158, 25: 135, 26: 130, 27: 167}
dAp6s4: ranges {0: 129, 1: 142, 2: 145, 3: 151, 4: 154, 5: 168, 6: 178, 7: 169, 8: 154, 9: 148, 10: 152, 11: 145, 12: 128, 13: 134, 14: 120, 15: 129, 16: 135, 17: 155, 18: 152, 19: 177, 20: 162, 21: 158, 22: 176, 23: 161, 24: 163, 25: 135, 26: 128, 27: 167}
dAp9s4: ranges {0: 127, 1: 143, 2: 144, 3: 149, 4: 152, 5: 167, 6: 178, 7: 167, 8: 155, 9: 150, 10: 154, 11: 145, 12: 132, 13: 135, 14: 122, 15: 126, 16: 136, 17: 156, 18: 153, 19: 177, 20: 164, 21: 158, 22: 175, 23: 160, 24: 161, 25: 135, 26: 130, 27: 170}
dApCs4: ranges {0: 127, 1: 146, 2: 147, 3: 150, 4: 155, 5: 166, 6: 173, 7: 164, 8: 152, 9: 151, 10: 157, 11: 144, 12: 132, 13: 133, 14: 120, 15: 134, 16: 133, 17: 157, 18: 154, 19: 176, 20: 163, 21: 157, 22: 176, 23: 160, 24: 161, 25: 138, 26: 130, 27: 169}
dApFs4: ranges {0: 126, 1: 139, 2: 146, 3: 153, 4: 154, 5: 169, 6: 182, 7: 172, 8: 154, 9: 148, 10: 155, 11: 146, 12: 132, 13: 133, 14: 120, 15: 132, 16: 136, 17: 158, 18: 153, 19: 178, 20: 161, 21: 155, 22: 173, 23: 162, 24: 162, 25: 135, 26: 131, 27: 169}
dBp0s4: ranges {0: 88, 1: 111, 2: 110, 3: 126, 4: 127, 5: 144, 6: 161, 7: 155, 8: 129, 9: 127, 10: 125, 11: 118, 12: 94, 13: 99, 14: 82, 15: 90, 16: 103, 17: 124, 18: 123, 19: 155, 20: 143, 21: 137, 22: 157, 23: 136, 24: 132, 25: 105, 26: 97, 27: 129}
dBp3s4: ranges {0: 127, 1: 140, 2: 143, 3: 149, 4: 151, 5: 165, 6: 169, 7: 165, 8: 150, 9: 145, 10: 150, 11: 140, 12: 123, 13: 132, 14: 115, 15: 126, 16: 132, 17: 153, 18: 149, 19: 172, 20: 160, 21: 154, 22: 174, 23: 158, 24: 158, 25: 132, 26: 128, 27: 163}
dBp6s4: ranges {0: 122, 1: 141, 2: 141, 3: 148, 4: 152, 5: 164, 6: 170, 7: 164, 8: 152, 9: 149, 10: 152, 11: 143, 12: 124, 13: 130, 14: 116, 15: 127, 16: 133, 17: 153, 18: 149, 19: 172, 20: 158, 21: 155, 22: 174, 23: 158, 24: 157, 25: 133, 26: 128, 27: 163}
dBp9s4: ranges {0: 125, 1: 140, 2: 140, 3: 146, 4: 151, 5: 165, 6: 172, 7: 168, 8: 152, 9: 148, 10: 156, 11: 141, 12: 127, 13: 132, 14: 116, 15: 128, 16: 130, 17: 153, 18: 149, 19: 172, 20: 159, 21: 156, 22: 171, 23: 157, 24: 162, 25: 135, 26: 125, 27: 167}
dBpCs4: ranges {0: 122, 1: 140, 2: 144, 3: 150, 4: 152, 5: 165, 6: 178, 7: 167, 8: 148, 9: 149, 10: 152, 11: 142, 12: 125, 13: 128, 14: 117, 15: 127, 16: 131, 17: 153, 18: 150, 19: 176, 20: 159, 21: 156, 22: 172, 23: 160, 24: 162, 25: 133, 26: 126, 27: 161}
dBpFs4: ranges {0: 128, 1: 140, 2: 143, 3: 149, 4: 152, 5: 167, 6: 174, 7: 166, 8: 153, 9: 147, 10: 149, 11: 139, 12: 129, 13: 134, 14: 116, 15: 129, 16: 134, 17: 152, 18: 152, 19: 177, 20: 162, 21: 154, 22: 172, 23: 160, 24: 158, 25: 136, 26: 128, 27: 157}
dCp0s4: ranges {0: 90, 1: 109, 2: 114, 3: 121, 4: 124, 5: 143, 6: 159, 7: 150, 8: 132, 9: 125, 10: 124, 11: 115, 12: 90, 13: 101, 14: 81, 15: 93, 16: 103, 17: 124, 18: 126, 19: 153, 20: 142, 21: 136, 22: 156, 23: 131, 24: 136, 25: 102, 26: 97, 27: 131}
dCp3s4: ranges {0: 123, 1: 140, 2: 142, 3: 141, 4: 152, 5: 163, 6: 176, 7: 170, 8: 148, 9: 145, 10: 150, 11: 140, 12: 126, 13: 128, 14: 110, 15: 123, 16: 128, 17: 152, 18: 148, 19: 175, 20: 159, 21: 155, 22: 171, 23: 155, 24: 156, 25: 133, 26: 123, 27: 158}
dCp6s4: ranges {0: 122, 1: 135, 2: 139, 3: 146, 4: 148, 5: 161, 6: 174, 7: 165, 8: 150, 9: 143, 10: 149, 11: 136, 12: 120, 13: 128, 14: 112, 15: 126, 16: 130, 17: 150, 18: 148, 19: 171, 20: 159, 21: 152, 22: 170, 23: 159, 24: 159, 25: 132, 26: 123, 27: 162}
dCp9s4: ranges {0: 116, 1: 138, 2: 140, 3: 148, 4: 152, 5: 163, 6: 176, 7: 164, 8: 149, 9: 145, 10: 149, 11: 142, 12: 126, 13: 127, 14: 110, 15: 124, 16: 128, 17: 151, 18: 144, 19: 175, 20: 158, 21: 155, 22: 172, 23: 160, 24: 159, 25: 133, 26: 124, 27: 160}
dCpCs4: ranges {0: 116, 1: 141, 2: 141, 3: 146, 4: 151, 5: 161, 6: 171, 7: 164, 8: 149, 9: 145, 10: 151, 11: 137, 12: 126, 13: 126, 14: 112, 15: 127, 16: 130, 17: 149, 18: 148, 19: 173, 20: 160, 21: 152, 22: 172, 23: 157, 24: 157, 25: 134, 26: 123, 27: 164}
dCpFs4: ranges {0: 126, 1: 140, 2: 140, 3: 148, 4: 151, 5: 163, 6: 171, 7: 164, 8: 151, 9: 149, 10: 150, 11: 140, 12: 125, 13: 129, 14: 111, 15: 126, 16: 127, 17: 151, 18: 148, 19: 175, 20: 159, 21: 153, 22: 173, 23: 156, 24: 159, 25: 133, 26: 126, 27: 163}
dDp0s4: ranges {0: 89, 1: 109, 2: 111, 3: 123, 4: 127, 5: 144, 6: 158, 7: 149, 8: 128, 9: 120, 10: 122, 11: 118, 12: 91, 13: 98, 14: 81, 15: 93, 16: 103, 17: 126, 18: 126, 19: 153, 20: 145, 21: 136, 22: 154, 23: 133, 24: 135, 25: 105, 26: 96, 27: 132}
dDp3s4: ranges {0: 119, 1: 136, 2: 138, 3: 145, 4: 149, 5: 164, 6: 174, 7: 164, 8: 148, 9: 145, 10: 143, 11: 140, 12: 124, 13: 125, 14: 108, 15: 123, 16: 126, 17: 148, 18: 145, 19: 174, 20: 154, 21: 151, 22: 171, 23: 156, 24: 153, 25: 130, 26: 119, 27: 157}
dDp6s4: ranges {0: 121, 1: 134, 2: 137, 3: 143, 4: 151, 5: 162, 6: 174, 7: 164, 8: 146, 9: 146, 10: 151, 11: 138, 12: 119, 13: 128, 14: 111, 15: 124, 16: 129, 17: 151, 18: 146, 19: 172, 20: 157, 21: 153, 22: 172, 23: 157, 24: 154, 25: 128, 26: 122, 27: 158}
dDp9s4: ranges {0: 116, 1: 135, 2: 140, 3: 143, 4: 148, 5: 165, 6: 175, 7: 163, 8: 148, 9: 143, 10: 149, 11: 138, 12: 124, 13: 121, 14: 110, 15: 126, 16: 127, 17: 147, 18: 146, 19: 174, 20: 159, 21: 152, 22: 172, 23: 154, 24: 158, 25: 129, 26: 120, 27: 158}
dDpCs4: ranges {0: 124, 1: 136, 2: 136, 3: 141, 4: 149, 5: 164, 6: 173, 7: 167, 8: 148, 9: 144, 10: 152, 11: 137, 12: 122, 13: 126, 14: 111, 15: 124, 16: 127, 17: 152, 18: 145, 19: 173, 20: 158, 21: 151, 22: 169, 23: 152, 24: 156, 25: 129, 26: 121, 27: 156}
dDpFs4: ranges {0: 121, 1: 136, 2: 138, 3: 145, 4: 151, 5: 161, 6: 173, 7: 163, 8: 147, 9: 145, 10: 150, 11: 139, 12: 124, 13: 126, 14: 110, 15: 124, 16: 130, 17: 149, 18: 145, 19: 174, 20: 158, 21: 153, 22: 172, 23: 157, 24: 157, 25: 130, 26: 120, 27: 158}
dEp0s4: ranges {0: 87, 1: 111, 2: 111, 3: 121, 4: 126, 5: 144, 6: 155, 7: 149, 8: 127, 9: 124, 10: 127, 11: 118, 12: 96, 13: 98, 14: 81, 15: 92, 16: 106, 17: 125, 18: 126, 19: 154, 20: 142, 21: 135, 22: 152, 23: 137, 24: 135, 25: 104, 26: 97, 27: 132}
dEp3s4: ranges {0: 118, 1: 133, 2: 135, 3: 142, 4: 147, 5: 160, 6: 178, 7: 167, 8: 147, 9: 141, 10: 145, 11: 136, 12: 119, 13: 123, 14: 104, 15: 121, 16: 126, 17: 148, 18: 145, 19: 168, 20: 157, 21: 152, 22: 169, 23: 153, 24: 157, 25: 130, 26: 118, 27: 154}
dEp6s4: ranges {0: 120, 1: 135, 2: 137, 3: 141, 4: 147, 5: 163, 6: 173, 7: 161, 8: 148, 9: 145, 10: 149, 11: 135, 12: 120, 13: 125, 14: 108, 15: 122, 16: 126, 17: 149, 18: 143, 19: 172, 20: 157, 21: 152, 22: 168, 23: 156, 24: 156, 25: 127, 26: 119, 27: 154}
dEp9s4: ranges {0: 114, 1: 135, 2: 136, 3: 141, 4: 147, 5: 162, 6: 169, 7: 164, 8: 146, 9: 142, 10: 148, 11: 137, 12: 120, 13: 120, 14: 109, 15: 119, 16: 126, 17: 148, 18: 146, 19: 172, 20: 157, 21: 154, 22: 170, 23: 154, 24: 158, 25: 128, 26: 123, 27: 154}
dEpCs4: ranges {0: 120, 1: 133, 2: 136, 3: 144, 4: 147, 5: 160, 6: 177, 7: 168, 8: 147, 9: 145, 10: 149, 11: 129, 12: 117, 13: 126, 14: 109, 15: 121, 16: 124, 17: 149, 18: 145, 19: 171, 20: 156, 21: 154, 22: 166, 23: 154, 24: 157, 25: 127, 26: 118, 27: 158}
dEpFs4: ranges {0: 116, 1: 136, 2: 136, 3: 140, 4: 149, 5: 161, 6: 173, 7: 163, 8: 147, 9: 143, 10: 148, 11: 136, 12: 119, 13: 121, 14: 110, 15: 120, 16: 125, 17: 149, 18: 144, 19: 172, 20: 159, 21: 154, 22: 168, 23: 156, 24: 157, 25: 128, 26: 122, 27: 156}
dFp0s4: ranges {0: 89, 1: 110, 2: 114, 3: 114, 4: 127, 5: 143, 6: 157, 7: 149, 8: 129, 9: 123, 10: 126, 11: 117, 12: 96, 13: 99, 14: 83, 15: 94, 16: 103, 17: 123, 18: 125, 19: 153, 20: 142, 21: 139, 22: 153, 23: 134, 24: 134, 25: 102, 26: 98, 27: 132}
dFp3s4: ranges {0: 118, 1: 132, 2: 137, 3: 139, 4: 147, 5: 161, 6: 170, 7: 163, 8: 147, 9: 143, 10: 146, 11: 136, 12: 119, 13: 123, 14: 104, 15: 118, 16: 121, 17: 147, 18: 144, 19: 168, 20: 157, 21: 151, 22: 168, 23: 154, 24: 152, 25: 125, 26: 118, 27: 155}
dFp6s4: ranges {0: 117, 1: 132, 2: 137, 3: 139, 4: 147, 5: 161, 6: 169, 7: 163, 8: 147, 9: 143, 10: 146, 11: 137, 12: 119, 13: 123, 14: 106, 15: 120, 16: 125, 17: 145, 18: 143, 19: 170, 20: 157, 21: 151, 22: 170, 23: 151, 24: 156, 25: 125, 26: 118, 27: 156}
dFp9s4: ranges {0: 117, 1: 130, 2: 133, 3: 141, 4: 147, 5: 160, 6: 173, 7: 167, 8: 147, 9: 144, 10: 148, 11: 133, 12: 119, 13: 123, 14: 104, 15: 119, 16: 126, 17: 144, 18: 145, 19: 170, 20: 155, 21: 150, 22: 168, 23: 155, 24: 154, 25: 127, 26: 117, 27: 155}
dFpCs4: ranges {0: 120, 1: 132, 2: 138, 3: 140, 4: 146, 5: 162, 6: 175, 7: 163, 8: 147, 9: 140, 10: 146, 11: 136, 12: 120, 13: 124, 14: 108, 15: 121, 16: 123, 17: 146, 18: 143, 19: 169, 20: 157, 21: 151, 22: 171, 23: 151, 24: 155, 25: 129, 26: 122, 27: 154}
dFpFs4: ranges {0: 119, 1: 133, 2: 135, 3: 136, 4: 149, 5: 159, 6: 166, 7: 162, 8: 148, 9: 144, 10: 146, 11: 135, 12: 117, 13: 123, 14: 108, 15: 122, 16: 119, 17: 148, 18: 143, 19: 169, 20: 158, 21: 152, 22: 167, 23: 152, 24: 158, 25: 126, 26: 117, 27: 155}
"""

undersampled_scan_172 = """
d1p0s4: ranges {0: None, 1: 39, 2: 5, 3: 95, 4: None, 5: 6, 6: 138, 7: 109, 8: 45, 9: 90, 10: 94, 11: 79, 12: None, 13: None, 14: None, 15: None, 16: 6, 17: 13, 18: 32, 19: 143, 20: 91, 21: 88, 22: 111, 23: 10, 24: 96, 25: None, 26: 80, 27: 24}
d1p3s4: ranges {0: 144, 1: 155, 2: 142, 3: 154, 4: 139, 5: 158, 6: 173, 7: 158, 8: 134, 9: 143, 10: 154, 11: 133, 12: 121, 13: 138, 14: 131, 15: 130, 16: 128, 17: 154, 18: 147, 19: 170, 20: 155, 21: 152, 22: 162, 23: 146, 24: 162, 25: 121, 26: 130, 27: 179}
d1p6s4: ranges {0: 152, 1: 157, 2: 142, 3: 157, 4: 147, 5: 163, 6: 171, 7: 164, 8: 141, 9: 147, 10: 161, 11: 135, 12: 129, 13: 149, 14: 140, 15: 139, 16: 135, 17: 158, 18: 151, 19: 173, 20: 161, 21: 155, 22: 168, 23: 151, 24: 166, 25: 131, 26: 135, 27: 189}
d1p9s4: ranges {0: 158, 1: 158, 2: 153, 3: 158, 4: 152, 5: 163, 6: 174, 7: 166, 8: 141, 9: 147, 10: 164, 11: 135, 12: 132, 13: 153, 14: 142, 15: 139, 16: 140, 17: 158, 18: 156, 19: 174, 20: 161, 21: 157, 22: 171, 23: 153, 24: 164, 25: 138, 26: 135, 27: 196}
d1pCs4: ranges {0: 155, 1: 155, 2: 152, 3: 161, 4: 156, 5: 166, 6: 176, 7: 166, 8: 144, 9: 149, 10: 164, 11: 136, 12: 133, 13: 153, 14: 140, 15: 145, 16: 143, 17: 161, 18: 158, 19: 175, 20: 160, 21: 157, 22: 170, 23: 153, 24: 165, 25: 138, 26: 138, 27: 195}
d1pFs4: ranges {0: 158, 1: 160, 2: 153, 3: 160, 4: 154, 5: 167, 6: 180, 7: 165, 8: 144, 9: 152, 10: 163, 11: 140, 12: 133, 13: 156, 14: 143, 15: 141, 16: 137, 17: 160, 18: 156, 19: 172, 20: 161, 21: 158, 22: 169, 23: 157, 24: 163, 25: 139, 26: 132, 27: 197}
d2p0s4: ranges {0: 38, 1: 93, 2: 59, 3: 113, 4: 37, 5: 113, 6: 150, 7: 135, 8: 94, 9: 108, 10: 107, 11: 87, 12: 17, 13: 54, 14: 42, 15: 10, 16: 67, 17: 90, 18: 105, 19: 149, 20: 133, 21: 110, 22: 136, 23: 88, 24: 122, 25: 26, 26: 84, 27: 88}
d2p3s4: ranges {0: 152, 1: 155, 2: 147, 3: 156, 4: 145, 5: 160, 6: 176, 7: 164, 8: 138, 9: 146, 10: 158, 11: 134, 12: 128, 13: 144, 14: 135, 15: 137, 16: 138, 17: 158, 18: 150, 19: 172, 20: 161, 21: 152, 22: 167, 23: 150, 24: 167, 25: 133, 26: 138, 27: 185}
d2p6s4: ranges {0: 152, 1: 156, 2: 150, 3: 158, 4: 156, 5: 167, 6: 176, 7: 163, 8: 143, 9: 149, 10: 163, 11: 137, 12: 135, 13: 151, 14: 142, 15: 146, 16: 140, 17: 159, 18: 156, 19: 176, 20: 161, 21: 156, 22: 169, 23: 153, 24: 168, 25: 136, 26: 140, 27: 195}
d2p9s4: ranges {0: 161, 1: 160, 2: 156, 3: 160, 4: 158, 5: 168, 6: 179, 7: 166, 8: 145, 9: 150, 10: 164, 11: 138, 12: 136, 13: 158, 14: 145, 15: 147, 16: 141, 17: 161, 18: 162, 19: 174, 20: 162, 21: 160, 22: 174, 23: 158, 24: 170, 25: 138, 26: 142, 27: 202}
d2pCs4: ranges {0: 155, 1: 159, 2: 155, 3: 162, 4: 157, 5: 168, 6: 178, 7: 169, 8: 146, 9: 149, 10: 167, 11: 141, 12: 140, 13: 157, 14: 144, 15: 146, 16: 144, 17: 163, 18: 160, 19: 180, 20: 163, 21: 160, 22: 174, 23: 160, 24: 168, 25: 140, 26: 142, 27: 204}
d2pFs4: ranges {0: 158, 1: 163, 2: 154, 3: 161, 4: 158, 5: 169, 6: 179, 7: 167, 8: 148, 9: 154, 10: 167, 11: 140, 12: 140, 13: 160, 14: 146, 15: 150, 16: 144, 17: 164, 18: 161, 19: 180, 20: 163, 21: 161, 22: 174, 23: 161, 24: 170, 25: 144, 26: 141, 27: 203}
d3p0s4: ranges {0: 51, 1: 98, 2: 79, 3: 117, 4: 83, 5: 123, 6: 153, 7: 138, 8: 106, 9: 111, 10: 114, 11: 92, 12: 44, 13: 66, 14: 49, 15: 39, 16: 73, 17: 107, 18: 109, 19: 145, 20: 138, 21: 120, 22: 143, 23: 108, 24: 122, 25: 57, 26: 83, 27: 101}
d3p3s4: ranges {0: 149, 1: 155, 2: 148, 3: 157, 4: 148, 5: 162, 6: 176, 7: 162, 8: 141, 9: 149, 10: 159, 11: 134, 12: 133, 13: 150, 14: 136, 15: 133, 16: 139, 17: 157, 18: 153, 19: 173, 20: 160, 21: 154, 22: 171, 23: 151, 24: 163, 25: 135, 26: 133, 27: 189}
d3p6s4: ranges {0: 159, 1: 157, 2: 151, 3: 157, 4: 153, 5: 165, 6: 175, 7: 165, 8: 145, 9: 149, 10: 162, 11: 137, 12: 141, 13: 159, 14: 139, 15: 143, 16: 144, 17: 156, 18: 156, 19: 177, 20: 160, 21: 157, 22: 174, 23: 161, 24: 170, 25: 141, 26: 139, 27: 199}
d3p9s4: ranges {0: 157, 1: 159, 2: 154, 3: 159, 4: 156, 5: 168, 6: 179, 7: 167, 8: 145, 9: 153, 10: 164, 11: 142, 12: 140, 13: 157, 14: 142, 15: 145, 16: 145, 17: 161, 18: 159, 19: 179, 20: 163, 21: 159, 22: 174, 23: 162, 24: 170, 25: 144, 26: 142, 27: 197}
d3pCs4: ranges {0: 159, 1: 157, 2: 151, 3: 159, 4: 155, 5: 170, 6: 179, 7: 167, 8: 147, 9: 153, 10: 164, 11: 140, 12: 141, 13: 162, 14: 144, 15: 146, 16: 147, 17: 160, 18: 161, 19: 177, 20: 164, 21: 159, 22: 177, 23: 164, 24: 170, 25: 145, 26: 140, 27: 193}
d3pFs4: ranges {0: 161, 1: 157, 2: 158, 3: 158, 4: 158, 5: 169, 6: 177, 7: 167, 8: 146, 9: 153, 10: 165, 11: 143, 12: 139, 13: 159, 14: 140, 15: 145, 16: 145, 17: 159, 18: 162, 19: 177, 20: 164, 21: 156, 22: 175, 23: 163, 24: 163, 25: 144, 26: 142, 27: 198}
d4p0s4: ranges {0: 66, 1: 100, 2: 86, 3: 115, 4: 97, 5: 126, 6: 153, 7: 138, 8: 111, 9: 113, 10: 114, 11: 93, 12: 53, 13: 81, 14: 55, 15: 56, 16: 78, 17: 112, 18: 108, 19: 147, 20: 139, 21: 122, 22: 144, 23: 112, 24: 122, 25: 73, 26: 83, 27: 121}
d4p3s4: ranges {0: 144, 1: 154, 2: 144, 3: 153, 4: 149, 5: 162, 6: 175, 7: 163, 8: 138, 9: 145, 10: 151, 11: 134, 12: 130, 13: 147, 14: 126, 15: 128, 16: 135, 17: 153, 18: 151, 19: 172, 20: 158, 21: 153, 22: 172, 23: 155, 24: 160, 25: 138, 26: 130, 27: 183}
d4p6s4: ranges {0: 150, 1: 160, 2: 149, 3: 155, 4: 154, 5: 169, 6: 177, 7: 166, 8: 145, 9: 148, 10: 158, 11: 137, 12: 137, 13: 158, 14: 138, 15: 139, 16: 141, 17: 161, 18: 155, 19: 177, 20: 163, 21: 157, 22: 173, 23: 161, 24: 169, 25: 143, 26: 137, 27: 193}
d4p9s4: ranges {0: 150, 1: 156, 2: 151, 3: 154, 4: 159, 5: 166, 6: 178, 7: 170, 8: 145, 9: 150, 10: 162, 11: 139, 12: 137, 13: 155, 14: 137, 15: 141, 16: 143, 17: 160, 18: 158, 19: 180, 20: 161, 21: 158, 22: 173, 23: 162, 24: 168, 25: 144, 26: 138, 27: 186}
d4pCs4: ranges {0: 152, 1: 155, 2: 153, 3: 157, 4: 156, 5: 166, 6: 179, 7: 165, 8: 143, 9: 151, 10: 161, 11: 140, 12: 137, 13: 156, 14: 137, 15: 138, 16: 138, 17: 157, 18: 155, 19: 179, 20: 162, 21: 156, 22: 173, 23: 164, 24: 163, 25: 141, 26: 133, 27: 187}
d4pFs4: ranges {0: 153, 1: 156, 2: 148, 3: 160, 4: 155, 5: 166, 6: 174, 7: 164, 8: 147, 9: 151, 10: 161, 11: 140, 12: 138, 13: 155, 14: 135, 15: 141, 16: 143, 17: 158, 18: 156, 19: 178, 20: 162, 21: 157, 22: 175, 23: 164, 24: 166, 25: 143, 26: 136, 27: 182}
d5p0s4: ranges {0: 63, 1: 95, 2: 88, 3: 118, 4: 105, 5: 129, 6: 154, 7: 140, 8: 109, 9: 115, 10: 116, 11: 96, 12: 63, 13: 79, 14: 62, 15: 63, 16: 84, 17: 112, 18: 113, 19: 146, 20: 138, 21: 124, 22: 147, 23: 121, 24: 126, 25: 78, 26: 84, 27: 112}
d5p3s4: ranges {0: 142, 1: 148, 2: 142, 3: 152, 4: 150, 5: 161, 6: 174, 7: 162, 8: 142, 9: 144, 10: 153, 11: 129, 12: 127, 13: 144, 14: 126, 15: 125, 16: 135, 17: 147, 18: 150, 19: 172, 20: 161, 21: 152, 22: 170, 23: 154, 24: 156, 25: 136, 26: 129, 27: 176}
d5p6s4: ranges {0: 149, 1: 154, 2: 150, 3: 157, 4: 155, 5: 165, 6: 177, 7: 166, 8: 145, 9: 152, 10: 156, 11: 137, 12: 136, 13: 149, 14: 136, 15: 133, 16: 140, 17: 159, 18: 152, 19: 175, 20: 162, 21: 153, 22: 175, 23: 160, 24: 164, 25: 140, 26: 135, 27: 180}
d5p9s4: ranges {0: 149, 1: 154, 2: 148, 3: 154, 4: 152, 5: 164, 6: 178, 7: 161, 8: 145, 9: 151, 10: 156, 11: 139, 12: 135, 13: 152, 14: 132, 15: 132, 16: 139, 17: 155, 18: 152, 19: 178, 20: 164, 21: 156, 22: 174, 23: 159, 24: 166, 25: 142, 26: 132, 27: 187}
d5pCs4: ranges {0: 144, 1: 150, 2: 148, 3: 155, 4: 152, 5: 165, 6: 175, 7: 167, 8: 143, 9: 151, 10: 156, 11: 138, 12: 133, 13: 148, 14: 129, 15: 138, 16: 139, 17: 157, 18: 154, 19: 177, 20: 163, 21: 153, 22: 174, 23: 158, 24: 162, 25: 142, 26: 135, 27: 189}
d5pFs4: ranges {0: 145, 1: 151, 2: 144, 3: 156, 4: 155, 5: 165, 6: 176, 7: 164, 8: 143, 9: 151, 10: 156, 11: 136, 12: 133, 13: 151, 14: 134, 15: 139, 16: 140, 17: 156, 18: 154, 19: 173, 20: 162, 21: 156, 22: 176, 23: 160, 24: 164, 25: 142, 26: 134, 27: 182}
d6p0s4: ranges {0: 74, 1: 100, 2: 91, 3: 117, 4: 105, 5: 131, 6: 154, 7: 143, 8: 114, 9: 115, 10: 116, 11: 95, 12: 66, 13: 87, 14: 61, 15: 69, 16: 84, 17: 116, 18: 114, 19: 148, 20: 139, 21: 124, 22: 146, 23: 120, 24: 123, 25: 84, 26: 83, 27: 124}
d6p3s4: ranges {0: 135, 1: 142, 2: 136, 3: 147, 4: 145, 5: 157, 6: 171, 7: 160, 8: 137, 9: 143, 10: 151, 11: 127, 12: 122, 13: 138, 14: 119, 15: 125, 16: 133, 17: 150, 18: 148, 19: 170, 20: 158, 21: 145, 22: 171, 23: 151, 24: 153, 25: 130, 26: 125, 27: 177}
d6p6s4: ranges {0: 137, 1: 152, 2: 143, 3: 151, 4: 148, 5: 163, 6: 173, 7: 166, 8: 142, 9: 147, 10: 152, 11: 133, 12: 131, 13: 141, 14: 126, 15: 133, 16: 137, 17: 157, 18: 153, 19: 175, 20: 161, 21: 149, 22: 171, 23: 158, 24: 160, 25: 138, 26: 129, 27: 178}
d6p9s4: ranges {0: 138, 1: 147, 2: 139, 3: 151, 4: 152, 5: 160, 6: 174, 7: 163, 8: 143, 9: 146, 10: 154, 11: 135, 12: 131, 13: 143, 14: 128, 15: 132, 16: 135, 17: 156, 18: 153, 19: 174, 20: 161, 21: 149, 22: 173, 23: 156, 24: 163, 25: 136, 26: 129, 27: 181}
d6pCs4: ranges {0: 142, 1: 150, 2: 141, 3: 153, 4: 151, 5: 165, 6: 174, 7: 163, 8: 144, 9: 147, 10: 153, 11: 134, 12: 130, 13: 145, 14: 127, 15: 133, 16: 135, 17: 153, 18: 152, 19: 177, 20: 160, 21: 153, 22: 172, 23: 156, 24: 163, 25: 137, 26: 130, 27: 182}
d6pFs4: ranges {0: 136, 1: 150, 2: 144, 3: 153, 4: 153, 5: 160, 6: 173, 7: 164, 8: 143, 9: 144, 10: 152, 11: 132, 12: 131, 13: 145, 14: 127, 15: 134, 16: 133, 17: 155, 18: 150, 19: 176, 20: 160, 21: 151, 22: 173, 23: 159, 24: 161, 25: 136, 26: 128, 27: 180}
d7p0s4: ranges {0: 79, 1: 100, 2: 93, 3: 119, 4: 108, 5: 131, 6: 152, 7: 139, 8: 115, 9: 116, 10: 116, 11: 95, 12: 73, 13: 86, 14: 65, 15: 72, 16: 86, 17: 119, 18: 119, 19: 147, 20: 139, 21: 129, 22: 149, 23: 121, 24: 125, 25: 87, 26: 84, 27: 125}
d7p3s4: ranges {0: 133, 1: 138, 2: 134, 3: 146, 4: 145, 5: 158, 6: 167, 7: 158, 8: 136, 9: 142, 10: 148, 11: 125, 12: 123, 13: 139, 14: 115, 15: 118, 16: 131, 17: 151, 18: 147, 19: 171, 20: 157, 21: 150, 22: 171, 23: 153, 24: 153, 25: 131, 26: 122, 27: 177}
d7p6s4: ranges {0: 139, 1: 147, 2: 140, 3: 148, 4: 153, 5: 162, 6: 171, 7: 162, 8: 141, 9: 146, 10: 151, 11: 130, 12: 128, 13: 139, 14: 121, 15: 128, 16: 131, 17: 150, 18: 151, 19: 175, 20: 158, 21: 153, 22: 172, 23: 157, 24: 162, 25: 134, 26: 122, 27: 172}
d7p9s4: ranges {0: 141, 1: 143, 2: 139, 3: 145, 4: 148, 5: 159, 6: 173, 7: 160, 8: 141, 9: 145, 10: 150, 11: 132, 12: 125, 13: 140, 14: 120, 15: 127, 16: 131, 17: 151, 18: 149, 19: 170, 20: 155, 21: 151, 22: 170, 23: 149, 24: 153, 25: 133, 26: 126, 27: 177}
d7pCs4: ranges {0: 137, 1: 144, 2: 138, 3: 146, 4: 149, 5: 161, 6: 173, 7: 163, 8: 141, 9: 145, 10: 151, 11: 132, 12: 124, 13: 138, 14: 120, 15: 128, 16: 126, 17: 151, 18: 148, 19: 174, 20: 158, 21: 152, 22: 168, 23: 154, 24: 159, 25: 131, 26: 126, 27: 179}
d7pFs4: ranges {0: 137, 1: 143, 2: 137, 3: 150, 4: 151, 5: 159, 6: 173, 7: 161, 8: 138, 9: 146, 10: 151, 11: 131, 12: 126, 13: 143, 14: 122, 15: 128, 16: 133, 17: 150, 18: 147, 19: 171, 20: 157, 21: 152, 22: 169, 23: 153, 24: 160, 25: 130, 26: 124, 27: 177}
d8p0s4: ranges {0: 79, 1: 99, 2: 94, 3: 116, 4: 110, 5: 131, 6: 154, 7: 138, 8: 115, 9: 114, 10: 116, 11: 95, 12: 74, 13: 90, 14: 62, 15: 76, 16: 90, 17: 115, 18: 115, 19: 149, 20: 134, 21: 127, 22: 146, 23: 124, 24: 124, 25: 87, 26: 82, 27: 125}
d8p3s4: ranges {0: 121, 1: 140, 2: 134, 3: 143, 4: 146, 5: 156, 6: 171, 7: 157, 8: 136, 9: 137, 10: 146, 11: 125, 12: 119, 13: 131, 14: 112, 15: 120, 16: 126, 17: 145, 18: 144, 19: 171, 20: 154, 21: 146, 22: 169, 23: 151, 24: 152, 25: 128, 26: 117, 27: 169}
d8p6s4: ranges {0: 131, 1: 140, 2: 137, 3: 148, 4: 146, 5: 159, 6: 174, 7: 160, 8: 140, 9: 140, 10: 148, 11: 128, 12: 122, 13: 138, 14: 114, 15: 122, 16: 131, 17: 148, 18: 146, 19: 173, 20: 159, 21: 150, 22: 170, 23: 153, 24: 157, 25: 127, 26: 122, 27: 172}
d8p9s4: ranges {0: 128, 1: 138, 2: 136, 3: 147, 4: 148, 5: 159, 6: 169, 7: 162, 8: 141, 9: 141, 10: 148, 11: 129, 12: 122, 13: 134, 14: 114, 15: 127, 16: 129, 17: 147, 18: 147, 19: 175, 20: 156, 21: 151, 22: 169, 23: 153, 24: 157, 25: 131, 26: 120, 27: 177}
d8pCs4: ranges {0: 135, 1: 138, 2: 136, 3: 146, 4: 147, 5: 157, 6: 169, 7: 163, 8: 139, 9: 144, 10: 151, 11: 129, 12: 124, 13: 138, 14: 116, 15: 125, 16: 128, 17: 146, 18: 148, 19: 174, 20: 155, 21: 148, 22: 171, 23: 154, 24: 158, 25: 131, 26: 120, 27: 173}
d8pFs4: ranges {0: 130, 1: 141, 2: 136, 3: 148, 4: 148, 5: 157, 6: 172, 7: 158, 8: 140, 9: 142, 10: 148, 11: 126, 12: 124, 13: 136, 14: 115, 15: 123, 16: 126, 17: 150, 18: 145, 19: 176, 20: 156, 21: 154, 22: 171, 23: 158, 24: 156, 25: 132, 26: 121, 27: 170}
d9p0s4: ranges {0: 77, 1: 100, 2: 94, 3: 116, 4: 114, 5: 130, 6: 151, 7: 141, 8: 115, 9: 115, 10: 116, 11: 94, 12: 75, 13: 89, 14: 68, 15: 78, 16: 90, 17: 115, 18: 113, 19: 142, 20: 138, 21: 126, 22: 145, 23: 121, 24: 124, 25: 91, 26: 84, 27: 126}
d9p3s4: ranges {0: 119, 1: 136, 2: 130, 3: 142, 4: 142, 5: 156, 6: 168, 7: 157, 8: 133, 9: 136, 10: 144, 11: 124, 12: 116, 13: 130, 14: 109, 15: 115, 16: 124, 17: 142, 18: 141, 19: 165, 20: 154, 21: 143, 22: 165, 23: 148, 24: 149, 25: 123, 26: 119, 27: 166}
d9p6s4: ranges {0: 125, 1: 139, 2: 132, 3: 140, 4: 146, 5: 156, 6: 169, 7: 157, 8: 138, 9: 140, 10: 142, 11: 126, 12: 121, 13: 132, 14: 113, 15: 121, 16: 126, 17: 143, 18: 145, 19: 166, 20: 152, 21: 144, 22: 166, 23: 152, 24: 154, 25: 130, 26: 118, 27: 172}
d9p9s4: ranges {0: 131, 1: 142, 2: 131, 3: 145, 4: 145, 5: 155, 6: 170, 7: 156, 8: 139, 9: 140, 10: 147, 11: 124, 12: 119, 13: 134, 14: 112, 15: 122, 16: 124, 17: 145, 18: 140, 19: 170, 20: 152, 21: 146, 22: 168, 23: 151, 24: 151, 25: 128, 26: 120, 27: 173}
d9pCs4: ranges {0: 131, 1: 138, 2: 134, 3: 147, 4: 148, 5: 156, 6: 167, 7: 158, 8: 135, 9: 137, 10: 147, 11: 124, 12: 120, 13: 136, 14: 114, 15: 123, 16: 125, 17: 142, 18: 141, 19: 169, 20: 151, 21: 143, 22: 165, 23: 152, 24: 151, 25: 125, 26: 119, 27: 170}
d9pFs4: ranges {0: 125, 1: 138, 2: 132, 3: 144, 4: 144, 5: 155, 6: 169, 7: 157, 8: 138, 9: 139, 10: 145, 11: 125, 12: 121, 13: 133, 14: 112, 15: 121, 16: 124, 17: 146, 18: 142, 19: 168, 20: 154, 21: 146, 22: 169, 23: 150, 24: 155, 25: 124, 26: 120, 27: 164}
dAp0s4: ranges {0: 74, 1: 101, 2: 96, 3: 119, 4: 110, 5: 130, 6: 150, 7: 138, 8: 115, 9: 112, 10: 117, 11: 94, 12: 75, 13: 91, 14: 69, 15: 79, 16: 89, 17: 113, 18: 114, 19: 141, 20: 138, 21: 123, 22: 143, 23: 124, 24: 122, 25: 90, 26: 84, 27: 127}
dAp3s4: ranges {0: 123, 1: 134, 2: 127, 3: 138, 4: 143, 5: 153, 6: 165, 7: 154, 8: 137, 9: 136, 10: 143, 11: 124, 12: 116, 13: 130, 14: 105, 15: 115, 16: 121, 17: 140, 18: 141, 19: 163, 20: 152, 21: 146, 22: 164, 23: 146, 24: 147, 25: 123, 26: 115, 27: 164}
dAp6s4: ranges {0: 119, 1: 135, 2: 131, 3: 141, 4: 141, 5: 152, 6: 167, 7: 155, 8: 138, 9: 138, 10: 141, 11: 120, 12: 117, 13: 126, 14: 108, 15: 116, 16: 123, 17: 141, 18: 140, 19: 168, 20: 152, 21: 144, 22: 166, 23: 151, 24: 151, 25: 122, 26: 115, 27: 163}
dAp9s4: ranges {0: 125, 1: 132, 2: 129, 3: 141, 4: 145, 5: 154, 6: 169, 7: 157, 8: 137, 9: 138, 10: 139, 11: 120, 12: 118, 13: 130, 14: 111, 15: 113, 16: 122, 17: 143, 18: 141, 19: 165, 20: 152, 21: 142, 22: 165, 23: 148, 24: 154, 25: 117, 26: 113, 27: 165}
dApCs4: ranges {0: 128, 1: 133, 2: 129, 3: 138, 4: 143, 5: 155, 6: 167, 7: 155, 8: 136, 9: 138, 10: 143, 11: 122, 12: 118, 13: 131, 14: 107, 15: 120, 16: 122, 17: 141, 18: 140, 19: 165, 20: 150, 21: 142, 22: 165, 23: 148, 24: 152, 25: 122, 26: 113, 27: 158}
dApFs4: ranges {0: 126, 1: 135, 2: 130, 3: 144, 4: 143, 5: 151, 6: 166, 7: 155, 8: 138, 9: 136, 10: 142, 11: 121, 12: 119, 13: 132, 14: 108, 15: 115, 16: 120, 17: 139, 18: 143, 19: 167, 20: 151, 21: 144, 22: 165, 23: 150, 24: 148, 25: 125, 26: 115, 27: 170}
dBp0s4: ranges {0: 76, 1: 100, 2: 95, 3: 116, 4: 114, 5: 131, 6: 150, 7: 137, 8: 115, 9: 114, 10: 116, 11: 92, 12: 76, 13: 90, 14: 69, 15: 79, 16: 85, 17: 113, 18: 115, 19: 145, 20: 134, 21: 127, 22: 140, 23: 125, 24: 123, 25: 93, 26: 83, 27: 125}
dBp3s4: ranges {0: 117, 1: 131, 2: 128, 3: 140, 4: 141, 5: 150, 6: 164, 7: 155, 8: 130, 9: 135, 10: 141, 11: 118, 12: 111, 13: 125, 14: 99, 15: 114, 16: 119, 17: 140, 18: 136, 19: 164, 20: 152, 21: 142, 22: 164, 23: 146, 24: 146, 25: 119, 26: 113, 27: 156}
dBp6s4: ranges {0: 122, 1: 133, 2: 123, 3: 138, 4: 143, 5: 151, 6: 168, 7: 154, 8: 135, 9: 132, 10: 142, 11: 117, 12: 113, 13: 126, 14: 105, 15: 113, 16: 120, 17: 139, 18: 140, 19: 166, 20: 152, 21: 144, 22: 165, 23: 146, 24: 151, 25: 124, 26: 109, 27: 162}
dBp9s4: ranges {0: 121, 1: 129, 2: 127, 3: 137, 4: 142, 5: 152, 6: 167, 7: 156, 8: 136, 9: 137, 10: 140, 11: 120, 12: 110, 13: 128, 14: 104, 15: 116, 16: 119, 17: 138, 18: 140, 19: 167, 20: 151, 21: 138, 22: 163, 23: 146, 24: 150, 25: 122, 26: 113, 27: 162}
dBpCs4: ranges {0: 122, 1: 134, 2: 126, 3: 141, 4: 144, 5: 155, 6: 167, 7: 155, 8: 136, 9: 136, 10: 142, 11: 118, 12: 114, 13: 128, 14: 106, 15: 114, 16: 121, 17: 138, 18: 142, 19: 169, 20: 149, 21: 146, 22: 165, 23: 147, 24: 148, 25: 124, 26: 112, 27: 156}
dBpFs4: ranges {0: 119, 1: 131, 2: 124, 3: 142, 4: 140, 5: 152, 6: 166, 7: 155, 8: 136, 9: 137, 10: 141, 11: 121, 12: 115, 13: 128, 14: 105, 15: 114, 16: 120, 17: 141, 18: 141, 19: 169, 20: 152, 21: 147, 22: 168, 23: 152, 24: 148, 25: 123, 26: 112, 27: 163}
dCp0s4: ranges {0: 77, 1: 102, 2: 97, 3: 118, 4: 112, 5: 130, 6: 148, 7: 138, 8: 115, 9: 116, 10: 115, 11: 94, 12: 76, 13: 92, 14: 70, 15: 78, 16: 89, 17: 112, 18: 112, 19: 142, 20: 135, 21: 127, 22: 142, 23: 122, 24: 123, 25: 89, 26: 81, 27: 128}
dCp3s4: ranges {0: 116, 1: 129, 2: 124, 3: 137, 4: 141, 5: 153, 6: 160, 7: 156, 8: 132, 9: 134, 10: 139, 11: 119, 12: 111, 13: 123, 14: 101, 15: 111, 16: 115, 17: 135, 18: 136, 19: 161, 20: 147, 21: 143, 22: 160, 23: 146, 24: 143, 25: 122, 26: 107, 27: 157}
dCp6s4: ranges {0: 117, 1: 130, 2: 124, 3: 137, 4: 141, 5: 150, 6: 165, 7: 155, 8: 134, 9: 136, 10: 139, 11: 117, 12: 109, 13: 123, 14: 101, 15: 110, 16: 115, 17: 138, 18: 136, 19: 162, 20: 149, 21: 140, 22: 163, 23: 143, 24: 149, 25: 117, 26: 105, 27: 157}
dCp9s4: ranges {0: 116, 1: 130, 2: 125, 3: 133, 4: 141, 5: 150, 6: 166, 7: 154, 8: 131, 9: 133, 10: 138, 11: 117, 12: 112, 13: 126, 14: 100, 15: 114, 16: 116, 17: 138, 18: 137, 19: 165, 20: 146, 21: 140, 22: 162, 23: 144, 24: 144, 25: 120, 26: 111, 27: 154}
dCpCs4: ranges {0: 120, 1: 129, 2: 127, 3: 138, 4: 139, 5: 152, 6: 164, 7: 154, 8: 135, 9: 133, 10: 139, 11: 117, 12: 113, 13: 126, 14: 102, 15: 113, 16: 116, 17: 138, 18: 136, 19: 161, 20: 146, 21: 139, 22: 162, 23: 145, 24: 143, 25: 118, 26: 109, 27: 159}
dCpFs4: ranges {0: 117, 1: 130, 2: 122, 3: 138, 4: 138, 5: 149, 6: 164, 7: 152, 8: 132, 9: 136, 10: 142, 11: 115, 12: 111, 13: 124, 14: 102, 15: 114, 16: 117, 17: 133, 18: 136, 19: 161, 20: 147, 21: 142, 22: 161, 23: 144, 24: 146, 25: 117, 26: 111, 27: 154}
dDp0s4: ranges {0: 84, 1: 101, 2: 100, 3: 116, 4: 113, 5: 132, 6: 147, 7: 138, 8: 112, 9: 117, 10: 115, 11: 94, 12: 78, 13: 96, 14: 70, 15: 79, 16: 90, 17: 115, 18: 113, 19: 142, 20: 135, 21: 128, 22: 143, 23: 125, 24: 118, 25: 91, 26: 86, 27: 129}
dDp3s4: ranges {0: 113, 1: 125, 2: 122, 3: 133, 4: 136, 5: 151, 6: 161, 7: 155, 8: 132, 9: 134, 10: 134, 11: 116, 12: 108, 13: 120, 14: 97, 15: 108, 16: 109, 17: 136, 18: 135, 19: 162, 20: 146, 21: 141, 22: 160, 23: 142, 24: 141, 25: 113, 26: 106, 27: 154}
dDp6s4: ranges {0: 106, 1: 125, 2: 123, 3: 135, 4: 138, 5: 150, 6: 165, 7: 153, 8: 132, 9: 136, 10: 138, 11: 114, 12: 107, 13: 117, 14: 97, 15: 111, 16: 113, 17: 137, 18: 136, 19: 164, 20: 149, 21: 138, 22: 160, 23: 146, 24: 147, 25: 114, 26: 109, 27: 156}
dDp9s4: ranges {0: 115, 1: 127, 2: 123, 3: 135, 4: 138, 5: 151, 6: 165, 7: 153, 8: 131, 9: 136, 10: 134, 11: 115, 12: 109, 13: 123, 14: 98, 15: 111, 16: 113, 17: 137, 18: 135, 19: 161, 20: 146, 21: 139, 22: 160, 23: 142, 24: 145, 25: 116, 26: 106, 27: 155}
dDpCs4: ranges {0: 110, 1: 127, 2: 122, 3: 134, 4: 136, 5: 150, 6: 164, 7: 154, 8: 132, 9: 133, 10: 133, 11: 114, 12: 107, 13: 120, 14: 98, 15: 110, 16: 112, 17: 135, 18: 136, 19: 162, 20: 148, 21: 141, 22: 162, 23: 141, 24: 145, 25: 117, 26: 107, 27: 159}
dDpFs4: ranges {0: 115, 1: 129, 2: 123, 3: 136, 4: 138, 5: 150, 6: 164, 7: 155, 8: 129, 9: 133, 10: 139, 11: 113, 12: 107, 13: 118, 14: 99, 15: 110, 16: 113, 17: 137, 18: 137, 19: 162, 20: 149, 21: 140, 22: 161, 23: 144, 24: 147, 25: 118, 26: 110, 27: 155}
dEp0s4: ranges {0: 78, 1: 101, 2: 98, 3: 113, 4: 114, 5: 131, 6: 146, 7: 137, 8: 110, 9: 115, 10: 111, 11: 94, 12: 79, 13: 92, 14: 71, 15: 79, 16: 90, 17: 112, 18: 113, 19: 143, 20: 135, 21: 123, 22: 138, 23: 120, 24: 118, 25: 92, 26: 81, 27: 131}
dEp3s4: ranges {0: 111, 1: 126, 2: 117, 3: 132, 4: 134, 5: 151, 6: 161, 7: 150, 8: 131, 9: 134, 10: 135, 11: 113, 12: 107, 13: 119, 14: 94, 15: 107, 16: 111, 17: 135, 18: 134, 19: 159, 20: 149, 21: 138, 22: 159, 23: 140, 24: 144, 25: 114, 26: 105, 27: 154}
dEp6s4: ranges {0: 105, 1: 125, 2: 116, 3: 135, 4: 137, 5: 147, 6: 161, 7: 153, 8: 130, 9: 134, 10: 136, 11: 115, 12: 105, 13: 115, 14: 92, 15: 107, 16: 113, 17: 135, 18: 135, 19: 161, 20: 147, 21: 139, 22: 158, 23: 145, 24: 144, 25: 116, 26: 104, 27: 158}
dEp9s4: ranges {0: 112, 1: 127, 2: 120, 3: 136, 4: 135, 5: 150, 6: 163, 7: 152, 8: 131, 9: 133, 10: 136, 11: 113, 12: 108, 13: 119, 14: 96, 15: 106, 16: 112, 17: 136, 18: 134, 19: 157, 20: 146, 21: 143, 22: 160, 23: 141, 24: 140, 25: 116, 26: 107, 27: 155}
dEpCs4: ranges {0: 107, 1: 128, 2: 118, 3: 132, 4: 138, 5: 147, 6: 161, 7: 150, 8: 131, 9: 134, 10: 138, 11: 113, 12: 107, 13: 117, 14: 98, 15: 108, 16: 112, 17: 136, 18: 133, 19: 159, 20: 146, 21: 139, 22: 162, 23: 141, 24: 142, 25: 116, 26: 105, 27: 156}
dEpFs4: ranges {0: 115, 1: 123, 2: 120, 3: 136, 4: 135, 5: 149, 6: 160, 7: 153, 8: 130, 9: 132, 10: 137, 11: 114, 12: 106, 13: 123, 14: 99, 15: 109, 16: 112, 17: 135, 18: 135, 19: 160, 20: 147, 21: 140, 22: 159, 23: 141, 24: 145, 25: 117, 26: 105, 27: 154}
dFp0s4: ranges {0: 80, 1: 101, 2: 95, 3: 114, 4: 114, 5: 131, 6: 148, 7: 135, 8: 114, 9: 113, 10: 109, 11: 92, 12: 75, 13: 92, 14: 68, 15: 82, 16: 87, 17: 115, 18: 115, 19: 142, 20: 135, 21: 123, 22: 143, 23: 124, 24: 123, 25: 93, 26: 82, 27: 129}
dFp3s4: ranges {0: 100, 1: 124, 2: 121, 3: 131, 4: 135, 5: 149, 6: 162, 7: 150, 8: 131, 9: 131, 10: 134, 11: 111, 12: 104, 13: 115, 14: 93, 15: 109, 16: 109, 17: 134, 18: 133, 19: 159, 20: 148, 21: 138, 22: 159, 23: 141, 24: 144, 25: 114, 26: 103, 27: 152}
dFp6s4: ranges {0: 110, 1: 123, 2: 116, 3: 133, 4: 136, 5: 150, 6: 162, 7: 152, 8: 125, 9: 133, 10: 135, 11: 113, 12: 103, 13: 119, 14: 94, 15: 107, 16: 110, 17: 133, 18: 135, 19: 157, 20: 145, 21: 142, 22: 159, 23: 146, 24: 143, 25: 113, 26: 103, 27: 156}
dFp9s4: ranges {0: 110, 1: 125, 2: 122, 3: 131, 4: 136, 5: 147, 6: 161, 7: 151, 8: 131, 9: 134, 10: 128, 11: 113, 12: 105, 13: 118, 14: 95, 15: 108, 16: 109, 17: 133, 18: 133, 19: 161, 20: 144, 21: 139, 22: 159, 23: 144, 24: 145, 25: 116, 26: 103, 27: 153}
dFpCs4: ranges {0: 106, 1: 121, 2: 120, 3: 131, 4: 135, 5: 148, 6: 163, 7: 150, 8: 129, 9: 132, 10: 134, 11: 112, 12: 104, 13: 116, 14: 93, 15: 107, 16: 112, 17: 132, 18: 134, 19: 161, 20: 147, 21: 139, 22: 158, 23: 143, 24: 143, 25: 115, 26: 104, 27: 153}
dFpFs4: ranges {0: 112, 1: 123, 2: 121, 3: 132, 4: 137, 5: 149, 6: 160, 7: 151, 8: 130, 9: 131, 10: 137, 11: 112, 12: 106, 13: 119, 14: 93, 15: 109, 16: 109, 17: 133, 18: 133, 19: 163, 20: 147, 21: 137, 22: 158, 23: 142, 24: 143, 25: 115, 26: 102, 27: 155}
"""

undersampled_scan_192 = """
d1p0s4: ranges {0: None, 1: 58, 2: 47, 3: 70, 4: None, 5: 24, 6: 161, 7: 137, 8: 92, 9: 118, 10: 122, 11: 107, 12: None, 13: 12, 14: 6, 15: None, 16: 16, 17: 20, 18: 59, 19: 155, 20: 89, 21: 117, 22: 135, 23: 45, 24: 119, 25: None, 26: 112, 27: 34}                           [1356/1846]
d1p3s4: ranges {0: 166, 1: 172, 2: 171, 3: 176, 4: 167, 5: 179, 6: 188, 7: 181, 8: 163, 9: 165, 10: 178, 11: 162, 12: 146, 13: 154, 14: 160, 15: 160, 16: 152, 17: 171, 18: 170, 19: 191, 20: 180, 21: 175, 22: 184, 23: 169, 24: 181, 25: 147, 26: 157, 27: 186}
d1p6s4: ranges {0: 170, 1: 175, 2: 170, 3: 177, 4: 173, 5: 184, 6: 194, 7: 188, 8: 168, 9: 172, 10: 185, 11: 164, 12: 155, 13: 166, 14: 165, 15: 167, 16: 160, 17: 175, 18: 170, 19: 189, 20: 180, 21: 177, 22: 191, 23: 175, 24: 183, 25: 157, 26: 160, 27: 194}
d1p9s4: ranges {0: 174, 1: 176, 2: 176, 3: 183, 4: 180, 5: 186, 6: 195, 7: 191, 8: 171, 9: 175, 10: 186, 11: 165, 12: 159, 13: 172, 14: 165, 15: 169, 16: 164, 17: 179, 18: 175, 19: 194, 20: 182, 21: 177, 22: 188, 23: 176, 24: 185, 25: 162, 26: 162, 27: 197}
d1pCs4: ranges {0: 178, 1: 178, 2: 179, 3: 179, 4: 179, 5: 189, 6: 194, 7: 192, 8: 170, 9: 176, 10: 187, 11: 167, 12: 154, 13: 172, 14: 166, 15: 164, 16: 164, 17: 181, 18: 177, 19: 194, 20: 185, 21: 183, 22: 193, 23: 178, 24: 186, 25: 163, 26: 161, 27: 198}
d1pFs4: ranges {0: 178, 1: 174, 2: 174, 3: 180, 4: 180, 5: 187, 6: 195, 7: 194, 8: 172, 9: 175, 10: 184, 11: 166, 12: 156, 13: 173, 14: 166, 15: 169, 16: 165, 17: 181, 18: 175, 19: 193, 20: 183, 21: 181, 22: 192, 23: 178, 24: 185, 25: 158, 26: 161, 27: 200}
d2p0s4: ranges {0: 63, 1: 118, 2: 95, 3: 136, 4: 68, 5: 135, 6: 170, 7: 160, 8: 129, 9: 130, 10: 134, 11: 116, 12: 51, 13: 72, 14: 66, 15: 37, 16: 90, 17: 120, 18: 128, 19: 164, 20: 153, 21: 142, 22: 158, 23: 116, 24: 140, 25: 55, 26: 108, 27: 100}
d2p3s4: ranges {0: 168, 1: 172, 2: 175, 3: 179, 4: 173, 5: 185, 6: 193, 7: 185, 8: 166, 9: 174, 10: 183, 11: 162, 12: 154, 13: 166, 14: 160, 15: 166, 16: 160, 17: 179, 18: 175, 19: 191, 20: 177, 21: 181, 22: 190, 23: 177, 24: 185, 25: 155, 26: 160, 27: 196}
d2p6s4: ranges {0: 180, 1: 177, 2: 182, 3: 180, 4: 179, 5: 187, 6: 197, 7: 191, 8: 171, 9: 174, 10: 187, 11: 168, 12: 160, 13: 173, 14: 164, 15: 171, 16: 167, 17: 181, 18: 181, 19: 195, 20: 183, 21: 180, 22: 188, 23: 180, 24: 184, 25: 163, 26: 165, 27: 203}
d2p9s4: ranges {0: 181, 1: 179, 2: 180, 3: 181, 4: 182, 5: 191, 6: 195, 7: 192, 8: 176, 9: 177, 10: 186, 11: 170, 12: 164, 13: 177, 14: 168, 15: 175, 16: 168, 17: 181, 18: 180, 19: 197, 20: 181, 21: 184, 22: 192, 23: 182, 24: 186, 25: 166, 26: 166, 27: 205}
d2pCs4: ranges {0: 181, 1: 177, 2: 182, 3: 182, 4: 187, 5: 191, 6: 197, 7: 190, 8: 176, 9: 177, 10: 187, 11: 170, 12: 163, 13: 180, 14: 169, 15: 177, 16: 168, 17: 183, 18: 181, 19: 197, 20: 182, 21: 187, 22: 192, 23: 182, 24: 186, 25: 166, 26: 168, 27: 207}
d2pFs4: ranges {0: 181, 1: 180, 2: 183, 3: 184, 4: 185, 5: 193, 6: 198, 7: 194, 8: 177, 9: 179, 10: 187, 11: 171, 12: 168, 13: 178, 14: 173, 15: 177, 16: 172, 17: 185, 18: 183, 19: 197, 20: 187, 21: 183, 22: 188, 23: 183, 24: 186, 25: 167, 26: 166, 27: 206}
d3p0s4: ranges {0: 81, 1: 119, 2: 107, 3: 138, 4: 111, 5: 146, 6: 173, 7: 166, 8: 138, 9: 136, 10: 136, 11: 122, 12: 77, 13: 89, 14: 76, 15: 77, 16: 104, 17: 133, 18: 133, 19: 169, 20: 161, 21: 147, 22: 165, 23: 133, 24: 142, 25: 82, 26: 112, 27: 116}
d3p3s4: ranges {0: 167, 1: 174, 2: 171, 3: 177, 4: 175, 5: 185, 6: 191, 7: 189, 8: 170, 9: 174, 10: 181, 11: 160, 12: 158, 13: 166, 14: 161, 15: 159, 16: 161, 17: 180, 18: 175, 19: 196, 20: 184, 21: 175, 22: 190, 23: 181, 24: 184, 25: 158, 26: 163, 27: 195}
d3p6s4: ranges {0: 178, 1: 179, 2: 182, 3: 179, 4: 183, 5: 191, 6: 196, 7: 192, 8: 173, 9: 174, 10: 187, 11: 166, 12: 166, 13: 176, 14: 167, 15: 169, 16: 167, 17: 181, 18: 181, 19: 196, 20: 184, 21: 185, 22: 194, 23: 183, 24: 183, 25: 165, 26: 166, 27: 203}
d3p9s4: ranges {0: 178, 1: 182, 2: 179, 3: 184, 4: 183, 5: 192, 6: 194, 7: 196, 8: 175, 9: 178, 10: 185, 11: 168, 12: 167, 13: 177, 14: 169, 15: 172, 16: 167, 17: 185, 18: 180, 19: 198, 20: 188, 21: 185, 22: 193, 23: 184, 24: 187, 25: 165, 26: 167, 27: 203}
d3pCs4: ranges {0: 180, 1: 173, 2: 180, 3: 181, 4: 186, 5: 191, 6: 197, 7: 194, 8: 179, 9: 176, 10: 183, 11: 165, 12: 165, 13: 177, 14: 169, 15: 172, 16: 172, 17: 183, 18: 181, 19: 201, 20: 184, 21: 180, 22: 197, 23: 184, 24: 183, 25: 166, 26: 165, 27: 204}
d3pFs4: ranges {0: 180, 1: 183, 2: 181, 3: 185, 4: 184, 5: 192, 6: 195, 7: 194, 8: 176, 9: 177, 10: 186, 11: 171, 12: 161, 13: 177, 14: 167, 15: 171, 16: 171, 17: 186, 18: 181, 19: 201, 20: 188, 21: 187, 22: 191, 23: 185, 24: 184, 25: 167, 26: 167, 27: 203}
d4p0s4: ranges {0: 87, 1: 119, 2: 112, 3: 138, 4: 121, 5: 151, 6: 175, 7: 164, 8: 138, 9: 136, 10: 139, 11: 123, 12: 87, 13: 98, 14: 83, 15: 81, 16: 106, 17: 136, 18: 135, 19: 165, 20: 162, 21: 149, 22: 167, 23: 140, 24: 143, 25: 95, 26: 111, 27: 122}
d4p3s4: ranges {0: 159, 1: 172, 2: 170, 3: 177, 4: 175, 5: 185, 6: 193, 7: 190, 8: 172, 9: 166, 10: 179, 11: 164, 12: 153, 13: 166, 14: 152, 15: 160, 16: 163, 17: 177, 18: 172, 19: 194, 20: 181, 21: 173, 22: 188, 23: 175, 24: 179, 25: 160, 26: 158, 27: 192}
d4p6s4: ranges {0: 170, 1: 178, 2: 177, 3: 179, 4: 181, 5: 187, 6: 195, 7: 192, 8: 172, 9: 177, 10: 184, 11: 163, 12: 162, 13: 169, 14: 162, 15: 166, 16: 167, 17: 181, 18: 177, 19: 198, 20: 182, 21: 180, 22: 193, 23: 182, 24: 185, 25: 165, 26: 166, 27: 198}
d4p9s4: ranges {0: 172, 1: 179, 2: 177, 3: 181, 4: 178, 5: 189, 6: 196, 7: 192, 8: 178, 9: 176, 10: 185, 11: 168, 12: 163, 13: 172, 14: 163, 15: 170, 16: 165, 17: 180, 18: 180, 19: 197, 20: 184, 21: 185, 22: 195, 23: 184, 24: 185, 25: 166, 26: 163, 27: 199}
d4pCs4: ranges {0: 174, 1: 177, 2: 176, 3: 177, 4: 181, 5: 190, 6: 192, 7: 193, 8: 175, 9: 174, 10: 184, 11: 170, 12: 160, 13: 173, 14: 160, 15: 164, 16: 169, 17: 180, 18: 179, 19: 195, 20: 185, 21: 178, 22: 192, 23: 182, 24: 185, 25: 164, 26: 160, 27: 197}
d4pFs4: ranges {0: 174, 1: 177, 2: 171, 3: 182, 4: 179, 5: 190, 6: 196, 7: 193, 8: 175, 9: 175, 10: 180, 11: 170, 12: 160, 13: 173, 14: 161, 15: 165, 16: 166, 17: 182, 18: 177, 19: 197, 20: 185, 21: 183, 22: 196, 23: 183, 24: 184, 25: 163, 26: 161, 27: 198}
d5p0s4: ranges {0: 91, 1: 124, 2: 114, 3: 138, 4: 128, 5: 153, 6: 175, 7: 167, 8: 140, 9: 137, 10: 139, 11: 124, 12: 93, 13: 103, 14: 85, 15: 94, 16: 112, 17: 136, 18: 136, 19: 165, 20: 163, 21: 156, 22: 163, 23: 137, 24: 141, 25: 106, 26: 110, 27: 126}
d5p3s4: ranges {0: 158, 1: 167, 2: 164, 3: 173, 4: 173, 5: 184, 6: 193, 7: 188, 8: 172, 9: 166, 10: 178, 11: 164, 12: 152, 13: 161, 14: 144, 15: 154, 16: 159, 17: 176, 18: 172, 19: 191, 20: 179, 21: 176, 22: 192, 23: 177, 24: 177, 25: 161, 26: 151, 27: 183}
d5p6s4: ranges {0: 162, 1: 171, 2: 171, 3: 176, 4: 179, 5: 186, 6: 194, 7: 189, 8: 170, 9: 174, 10: 178, 11: 162, 12: 159, 13: 168, 14: 156, 15: 161, 16: 165, 17: 179, 18: 174, 19: 191, 20: 177, 21: 176, 22: 191, 23: 183, 24: 184, 25: 162, 26: 161, 27: 189}
d5p9s4: ranges {0: 168, 1: 168, 2: 172, 3: 179, 4: 179, 5: 187, 6: 197, 7: 192, 8: 172, 9: 175, 10: 179, 11: 167, 12: 158, 13: 168, 14: 154, 15: 161, 16: 162, 17: 178, 18: 174, 19: 196, 20: 183, 21: 178, 22: 194, 23: 182, 24: 180, 25: 164, 26: 158, 27: 191}
d5pCs4: ranges {0: 167, 1: 173, 2: 172, 3: 178, 4: 178, 5: 188, 6: 196, 7: 190, 8: 177, 9: 171, 10: 180, 11: 166, 12: 157, 13: 166, 14: 155, 15: 164, 16: 162, 17: 178, 18: 175, 19: 197, 20: 183, 21: 178, 22: 194, 23: 182, 24: 181, 25: 163, 26: 159, 27: 192}
d5pFs4: ranges {0: 166, 1: 172, 2: 167, 3: 182, 4: 178, 5: 187, 6: 194, 7: 190, 8: 175, 9: 172, 10: 178, 11: 167, 12: 159, 13: 166, 14: 156, 15: 164, 16: 162, 17: 178, 18: 176, 19: 192, 20: 184, 21: 179, 22: 192, 23: 181, 24: 178, 25: 165, 26: 159, 27: 192}
d6p0s4: ranges {0: 95, 1: 125, 2: 119, 3: 140, 4: 134, 5: 152, 6: 170, 7: 169, 8: 144, 9: 141, 10: 136, 11: 122, 12: 96, 13: 105, 14: 90, 15: 99, 16: 111, 17: 138, 18: 138, 19: 165, 20: 160, 21: 152, 22: 167, 23: 141, 24: 144, 25: 106, 26: 107, 27: 130}
d6p3s4: ranges {0: 152, 1: 167, 2: 163, 3: 168, 4: 172, 5: 181, 6: 190, 7: 188, 8: 167, 9: 167, 10: 173, 11: 158, 12: 148, 13: 157, 14: 145, 15: 151, 16: 156, 17: 171, 18: 169, 19: 190, 20: 181, 21: 172, 22: 188, 23: 172, 24: 171, 25: 158, 26: 154, 27: 183}
d6p6s4: ranges {0: 159, 1: 173, 2: 167, 3: 176, 4: 174, 5: 185, 6: 195, 7: 190, 8: 169, 9: 168, 10: 179, 11: 161, 12: 154, 13: 160, 14: 151, 15: 158, 16: 162, 17: 176, 18: 172, 19: 192, 20: 184, 21: 177, 22: 190, 23: 182, 24: 176, 25: 156, 26: 158, 27: 185}
d6p9s4: ranges {0: 161, 1: 170, 2: 167, 3: 173, 4: 173, 5: 186, 6: 193, 7: 190, 8: 173, 9: 170, 10: 176, 11: 162, 12: 157, 13: 163, 14: 152, 15: 156, 16: 161, 17: 177, 18: 173, 19: 194, 20: 181, 21: 182, 22: 191, 23: 178, 24: 179, 25: 160, 26: 152, 27: 188}
d6pCs4: ranges {0: 160, 1: 170, 2: 165, 3: 173, 4: 174, 5: 183, 6: 192, 7: 187, 8: 175, 9: 172, 10: 177, 11: 162, 12: 151, 13: 162, 14: 149, 15: 158, 16: 160, 17: 173, 18: 173, 19: 193, 20: 181, 21: 175, 22: 188, 23: 179, 24: 178, 25: 161, 26: 152, 27: 187}
d6pFs4: ranges {0: 162, 1: 168, 2: 166, 3: 174, 4: 176, 5: 184, 6: 195, 7: 189, 8: 170, 9: 167, 10: 176, 11: 161, 12: 155, 13: 164, 14: 152, 15: 159, 16: 160, 17: 174, 18: 171, 19: 191, 20: 177, 21: 176, 22: 193, 23: 180, 24: 178, 25: 161, 26: 153, 27: 188}
d7p0s4: ranges {0: 96, 1: 122, 2: 117, 3: 139, 4: 133, 5: 153, 6: 169, 7: 166, 8: 146, 9: 138, 10: 139, 11: 121, 12: 100, 13: 107, 14: 93, 15: 97, 16: 111, 17: 137, 18: 137, 19: 163, 20: 156, 21: 151, 22: 168, 23: 146, 24: 143, 25: 111, 26: 111, 27: 133}
d7p3s4: ranges {0: 148, 1: 160, 2: 162, 3: 168, 4: 170, 5: 180, 6: 189, 7: 185, 8: 170, 9: 162, 10: 170, 11: 156, 12: 143, 13: 153, 14: 140, 15: 153, 16: 154, 17: 169, 18: 167, 19: 186, 20: 177, 21: 169, 22: 186, 23: 173, 24: 170, 25: 151, 26: 151, 27: 178}
d7p6s4: ranges {0: 155, 1: 166, 2: 165, 3: 172, 4: 176, 5: 181, 6: 190, 7: 186, 8: 171, 9: 169, 10: 174, 11: 158, 12: 149, 13: 156, 14: 147, 15: 153, 16: 157, 17: 175, 18: 168, 19: 192, 20: 178, 21: 176, 22: 188, 23: 176, 24: 178, 25: 157, 26: 155, 27: 179}
d7p9s4: ranges {0: 151, 1: 167, 2: 166, 3: 172, 4: 173, 5: 181, 6: 192, 7: 188, 8: 171, 9: 166, 10: 175, 11: 159, 12: 152, 13: 157, 14: 146, 15: 151, 16: 157, 17: 175, 18: 171, 19: 191, 20: 181, 21: 173, 22: 187, 23: 175, 24: 170, 25: 155, 26: 150, 27: 183}
d7pCs4: ranges {0: 156, 1: 168, 2: 163, 3: 170, 4: 172, 5: 183, 6: 192, 7: 188, 8: 170, 9: 167, 10: 175, 11: 158, 12: 153, 13: 160, 14: 145, 15: 151, 16: 156, 17: 174, 18: 168, 19: 192, 20: 179, 21: 175, 22: 191, 23: 176, 24: 178, 25: 159, 26: 152, 27: 183}
d7pFs4: ranges {0: 154, 1: 167, 2: 163, 3: 170, 4: 173, 5: 183, 6: 191, 7: 189, 8: 171, 9: 168, 10: 173, 11: 160, 12: 154, 13: 159, 14: 147, 15: 155, 16: 157, 17: 174, 18: 170, 19: 192, 20: 180, 21: 172, 22: 187, 23: 177, 24: 175, 25: 153, 26: 152, 27: 183}
d8p0s4: ranges {0: 96, 1: 119, 2: 117, 3: 134, 4: 135, 5: 153, 6: 173, 7: 166, 8: 144, 9: 137, 10: 135, 11: 122, 12: 99, 13: 106, 14: 94, 15: 103, 16: 111, 17: 139, 18: 136, 19: 166, 20: 162, 21: 151, 22: 165, 23: 142, 24: 144, 25: 111, 26: 109, 27: 132}
d8p3s4: ranges {0: 145, 1: 152, 2: 157, 3: 166, 4: 167, 5: 178, 6: 188, 7: 180, 8: 167, 9: 161, 10: 170, 11: 155, 12: 147, 13: 152, 14: 135, 15: 147, 16: 149, 17: 167, 18: 166, 19: 185, 20: 174, 21: 174, 22: 186, 23: 170, 24: 171, 25: 152, 26: 147, 27: 176}
d8p6s4: ranges {0: 150, 1: 163, 2: 161, 3: 166, 4: 170, 5: 181, 6: 190, 7: 185, 8: 169, 9: 166, 10: 174, 11: 156, 12: 150, 13: 154, 14: 141, 15: 150, 16: 151, 17: 170, 18: 166, 19: 190, 20: 175, 21: 171, 22: 185, 23: 176, 24: 171, 25: 154, 26: 148, 27: 175}
d8p9s4: ranges {0: 152, 1: 164, 2: 159, 3: 164, 4: 172, 5: 176, 6: 186, 7: 184, 8: 168, 9: 167, 10: 170, 11: 156, 12: 151, 13: 153, 14: 142, 15: 152, 16: 154, 17: 172, 18: 168, 19: 191, 20: 176, 21: 171, 22: 184, 23: 175, 24: 174, 25: 155, 26: 149, 27: 176}
d8pCs4: ranges {0: 150, 1: 160, 2: 162, 3: 168, 4: 173, 5: 181, 6: 190, 7: 188, 8: 167, 9: 165, 10: 169, 11: 156, 12: 149, 13: 155, 14: 141, 15: 151, 16: 155, 17: 171, 18: 167, 19: 188, 20: 176, 21: 170, 22: 186, 23: 174, 24: 171, 25: 152, 26: 146, 27: 176}
d8pFs4: ranges {0: 152, 1: 165, 2: 161, 3: 168, 4: 171, 5: 181, 6: 187, 7: 185, 8: 168, 9: 165, 10: 169, 11: 152, 12: 150, 13: 151, 14: 143, 15: 151, 16: 154, 17: 167, 18: 167, 19: 187, 20: 176, 21: 172, 22: 185, 23: 175, 24: 172, 25: 153, 26: 149, 27: 180}
d9p0s4: ranges {0: 98, 1: 122, 2: 117, 3: 136, 4: 137, 5: 154, 6: 169, 7: 164, 8: 145, 9: 140, 10: 137, 11: 118, 12: 99, 13: 108, 14: 94, 15: 100, 16: 112, 17: 139, 18: 136, 19: 166, 20: 156, 21: 150, 22: 163, 23: 145, 24: 142, 25: 113, 26: 110, 27: 132}
d9p3s4: ranges {0: 142, 1: 155, 2: 159, 3: 164, 4: 164, 5: 177, 6: 187, 7: 180, 8: 167, 9: 163, 10: 165, 11: 151, 12: 140, 13: 147, 14: 133, 15: 146, 16: 147, 17: 164, 18: 161, 19: 184, 20: 173, 21: 168, 22: 185, 23: 172, 24: 168, 25: 147, 26: 144, 27: 170}
d9p6s4: ranges {0: 143, 1: 160, 2: 157, 3: 164, 4: 168, 5: 177, 6: 189, 7: 183, 8: 165, 9: 165, 10: 169, 11: 154, 12: 146, 13: 144, 14: 137, 15: 140, 16: 148, 17: 170, 18: 163, 19: 188, 20: 172, 21: 174, 22: 185, 23: 175, 24: 172, 25: 150, 26: 145, 27: 171}
d9p9s4: ranges {0: 146, 1: 160, 2: 156, 3: 164, 4: 169, 5: 180, 6: 186, 7: 183, 8: 166, 9: 164, 10: 169, 11: 152, 12: 139, 13: 147, 14: 137, 15: 147, 16: 149, 17: 165, 18: 164, 19: 188, 20: 171, 21: 172, 22: 183, 23: 172, 24: 166, 25: 148, 26: 142, 27: 171}
d9pCs4: ranges {0: 147, 1: 161, 2: 157, 3: 166, 4: 167, 5: 178, 6: 190, 7: 181, 8: 166, 9: 165, 10: 169, 11: 152, 12: 144, 13: 147, 14: 139, 15: 148, 16: 149, 17: 168, 18: 165, 19: 187, 20: 171, 21: 167, 22: 187, 23: 171, 24: 170, 25: 151, 26: 144, 27: 175}
d9pFs4: ranges {0: 140, 1: 158, 2: 160, 3: 163, 4: 168, 5: 180, 6: 187, 7: 185, 8: 168, 9: 162, 10: 167, 11: 149, 12: 143, 13: 144, 14: 132, 15: 148, 16: 148, 17: 168, 18: 163, 19: 188, 20: 176, 21: 171, 22: 183, 23: 172, 24: 169, 25: 147, 26: 145, 27: 174}
dAp0s4: ranges {0: 98, 1: 121, 2: 115, 3: 137, 4: 136, 5: 152, 6: 170, 7: 164, 8: 144, 9: 135, 10: 138, 11: 118, 12: 102, 13: 108, 14: 90, 15: 103, 16: 113, 17: 136, 18: 135, 19: 164, 20: 157, 21: 148, 22: 162, 23: 146, 24: 142, 25: 112, 26: 110, 27: 131}
dAp3s4: ranges {0: 136, 1: 156, 2: 153, 3: 160, 4: 164, 5: 178, 6: 187, 7: 180, 8: 163, 9: 160, 10: 160, 11: 149, 12: 140, 13: 143, 14: 124, 15: 143, 16: 143, 17: 165, 18: 161, 19: 185, 20: 172, 21: 169, 22: 183, 23: 168, 24: 164, 25: 148, 26: 138, 27: 165}
dAp6s4: ranges {0: 142, 1: 152, 2: 154, 3: 159, 4: 166, 5: 179, 6: 185, 7: 183, 8: 164, 9: 165, 10: 166, 11: 149, 12: 143, 13: 148, 14: 130, 15: 144, 16: 145, 17: 162, 18: 161, 19: 185, 20: 173, 21: 169, 22: 184, 23: 174, 24: 169, 25: 146, 26: 139, 27: 169}
dAp9s4: ranges {0: 139, 1: 156, 2: 155, 3: 161, 4: 168, 5: 175, 6: 186, 7: 182, 8: 164, 9: 161, 10: 168, 11: 147, 12: 141, 13: 142, 14: 131, 15: 140, 16: 144, 17: 165, 18: 160, 19: 185, 20: 174, 21: 168, 22: 181, 23: 172, 24: 169, 25: 145, 26: 141, 27: 169}
dApCs4: ranges {0: 134, 1: 155, 2: 154, 3: 158, 4: 168, 5: 176, 6: 185, 7: 182, 8: 162, 9: 162, 10: 162, 11: 149, 12: 143, 13: 141, 14: 131, 15: 144, 16: 145, 17: 163, 18: 163, 19: 183, 20: 171, 21: 168, 22: 182, 23: 172, 24: 169, 25: 143, 26: 141, 27: 167}
dApFs4: ranges {0: 138, 1: 156, 2: 155, 3: 159, 4: 169, 5: 176, 6: 184, 7: 182, 8: 165, 9: 160, 10: 167, 11: 149, 12: 137, 13: 147, 14: 133, 15: 143, 16: 144, 17: 159, 18: 161, 19: 184, 20: 171, 21: 163, 22: 181, 23: 166, 24: 166, 25: 147, 26: 142, 27: 168}
dBp0s4: ranges {0: 99, 1: 123, 2: 121, 3: 136, 4: 136, 5: 150, 6: 165, 7: 164, 8: 143, 9: 140, 10: 136, 11: 120, 12: 104, 13: 110, 14: 92, 15: 105, 16: 113, 17: 136, 18: 137, 19: 162, 20: 156, 21: 151, 22: 161, 23: 144, 24: 140, 25: 112, 26: 108, 27: 132}
dBp3s4: ranges {0: 137, 1: 147, 2: 152, 3: 157, 4: 161, 5: 175, 6: 183, 7: 179, 8: 164, 9: 153, 10: 160, 11: 145, 12: 135, 13: 140, 14: 124, 15: 141, 16: 141, 17: 163, 18: 159, 19: 182, 20: 169, 21: 164, 22: 180, 23: 168, 24: 164, 25: 145, 26: 137, 27: 164}
dBp6s4: ranges {0: 133, 1: 149, 2: 152, 3: 158, 4: 165, 5: 174, 6: 183, 7: 182, 8: 164, 9: 159, 10: 164, 11: 146, 12: 136, 13: 141, 14: 128, 15: 142, 16: 143, 17: 158, 18: 158, 19: 184, 20: 169, 21: 165, 22: 179, 23: 169, 24: 167, 25: 143, 26: 138, 27: 168}
dBp9s4: ranges {0: 136, 1: 151, 2: 151, 3: 159, 4: 163, 5: 178, 6: 184, 7: 181, 8: 164, 9: 160, 10: 163, 11: 147, 12: 135, 13: 141, 14: 126, 15: 144, 16: 142, 17: 159, 18: 159, 19: 182, 20: 170, 21: 165, 22: 181, 23: 169, 24: 166, 25: 145, 26: 135, 27: 167}
dBpCs4: ranges {0: 137, 1: 151, 2: 151, 3: 156, 4: 162, 5: 173, 6: 181, 7: 181, 8: 164, 9: 161, 10: 164, 11: 146, 12: 140, 13: 142, 14: 127, 15: 140, 16: 142, 17: 161, 18: 161, 19: 184, 20: 169, 21: 166, 22: 181, 23: 170, 24: 167, 25: 144, 26: 136, 27: 165}
dBpFs4: ranges {0: 136, 1: 150, 2: 151, 3: 158, 4: 165, 5: 175, 6: 183, 7: 182, 8: 160, 9: 160, 10: 164, 11: 146, 12: 139, 13: 139, 14: 132, 15: 140, 16: 140, 17: 162, 18: 162, 19: 182, 20: 170, 21: 167, 22: 183, 23: 169, 24: 166, 25: 143, 26: 139, 27: 167}
dCp0s4: ranges {0: 100, 1: 121, 2: 119, 3: 131, 4: 137, 5: 151, 6: 168, 7: 163, 8: 143, 9: 136, 10: 134, 11: 119, 12: 103, 13: 110, 14: 96, 15: 107, 16: 115, 17: 135, 18: 135, 19: 160, 20: 157, 21: 149, 22: 161, 23: 147, 24: 144, 25: 112, 26: 106, 27: 132}
dCp3s4: ranges {0: 132, 1: 145, 2: 148, 3: 153, 4: 160, 5: 170, 6: 182, 7: 179, 8: 163, 9: 157, 10: 160, 11: 141, 12: 135, 13: 137, 14: 121, 15: 141, 16: 139, 17: 155, 18: 155, 19: 169, 20: 164, 21: 163, 22: 178, 23: 162, 24: 163, 25: 142, 26: 132, 27: 161}
dCp6s4: ranges {0: 135, 1: 145, 2: 146, 3: 157, 4: 161, 5: 173, 6: 183, 7: 180, 8: 166, 9: 158, 10: 161, 11: 143, 12: 133, 13: 140, 14: 121, 15: 134, 16: 138, 17: 160, 18: 157, 19: 181, 20: 171, 21: 164, 22: 178, 23: 169, 24: 162, 25: 138, 26: 133, 27: 162}
dCp9s4: ranges {0: 134, 1: 149, 2: 149, 3: 158, 4: 161, 5: 175, 6: 182, 7: 182, 8: 162, 9: 160, 10: 157, 11: 144, 12: 134, 13: 140, 14: 126, 15: 133, 16: 141, 17: 160, 18: 158, 19: 177, 20: 170, 21: 165, 22: 181, 23: 167, 24: 164, 25: 143, 26: 135, 27: 163}
dCpCs4: ranges {0: 136, 1: 147, 2: 146, 3: 154, 4: 161, 5: 171, 6: 182, 7: 182, 8: 165, 9: 157, 10: 163, 11: 144, 12: 137, 13: 139, 14: 126, 15: 139, 16: 138, 17: 159, 18: 158, 19: 182, 20: 168, 21: 166, 22: 181, 23: 167, 24: 166, 25: 144, 26: 134, 27: 164}
dCpFs4: ranges {0: 135, 1: 149, 2: 146, 3: 155, 4: 163, 5: 174, 6: 179, 7: 179, 8: 163, 9: 159, 10: 160, 11: 142, 12: 135, 13: 138, 14: 126, 15: 136, 16: 141, 17: 158, 18: 160, 19: 180, 20: 171, 21: 168, 22: 181, 23: 171, 24: 162, 25: 142, 26: 135, 27: 163}
dDp0s4: ranges {0: 102, 1: 122, 2: 119, 3: 134, 4: 135, 5: 147, 6: 161, 7: 161, 8: 145, 9: 139, 10: 135, 11: 120, 12: 104, 13: 111, 14: 96, 15: 109, 16: 111, 17: 132, 18: 135, 19: 161, 20: 152, 21: 145, 22: 160, 23: 146, 24: 146, 25: 110, 26: 109, 27: 130}
dDp3s4: ranges {0: 124, 1: 145, 2: 145, 3: 148, 4: 162, 5: 173, 6: 180, 7: 174, 8: 159, 9: 158, 10: 160, 11: 140, 12: 135, 13: 136, 14: 120, 15: 133, 16: 137, 17: 156, 18: 154, 19: 174, 20: 166, 21: 160, 22: 175, 23: 163, 24: 162, 25: 138, 26: 131, 27: 159}
dDp6s4: ranges {0: 128, 1: 141, 2: 147, 3: 153, 4: 160, 5: 170, 6: 180, 7: 175, 8: 162, 9: 157, 10: 160, 11: 141, 12: 133, 13: 135, 14: 120, 15: 138, 16: 136, 17: 159, 18: 156, 19: 180, 20: 165, 21: 162, 22: 177, 23: 163, 24: 162, 25: 142, 26: 130, 27: 161}
dDp9s4: ranges {0: 130, 1: 147, 2: 144, 3: 154, 4: 159, 5: 173, 6: 180, 7: 178, 8: 162, 9: 158, 10: 162, 11: 143, 12: 133, 13: 138, 14: 121, 15: 136, 16: 138, 17: 157, 18: 155, 19: 177, 20: 166, 21: 161, 22: 174, 23: 167, 24: 162, 25: 142, 26: 131, 27: 160}
dDpCs4: ranges {0: 130, 1: 144, 2: 149, 3: 153, 4: 160, 5: 170, 6: 178, 7: 176, 8: 161, 9: 156, 10: 162, 11: 144, 12: 134, 13: 135, 14: 121, 15: 135, 16: 136, 17: 159, 18: 154, 19: 179, 20: 169, 21: 164, 22: 177, 23: 167, 24: 162, 25: 140, 26: 127, 27: 160}
dDpFs4: ranges {0: 133, 1: 145, 2: 144, 3: 156, 4: 159, 5: 170, 6: 180, 7: 174, 8: 164, 9: 154, 10: 160, 11: 143, 12: 135, 13: 138, 14: 121, 15: 138, 16: 136, 17: 157, 18: 157, 19: 181, 20: 167, 21: 162, 22: 178, 23: 166, 24: 164, 25: 141, 26: 135, 27: 162}
dEp0s4: ranges {0: 98, 1: 122, 2: 120, 3: 132, 4: 136, 5: 149, 6: 163, 7: 159, 8: 145, 9: 136, 10: 137, 11: 120, 12: 103, 13: 111, 14: 92, 15: 109, 16: 110, 17: 135, 18: 132, 19: 160, 20: 155, 21: 149, 22: 160, 23: 139, 24: 141, 25: 112, 26: 107, 27: 129}
dEp3s4: ranges {0: 128, 1: 142, 2: 144, 3: 152, 4: 159, 5: 169, 6: 179, 7: 174, 8: 164, 9: 156, 10: 156, 11: 142, 12: 131, 13: 132, 14: 116, 15: 132, 16: 134, 17: 153, 18: 154, 19: 177, 20: 167, 21: 159, 22: 177, 23: 163, 24: 156, 25: 134, 26: 132, 27: 159}
dEp6s4: ranges {0: 130, 1: 145, 2: 143, 3: 152, 4: 160, 5: 169, 6: 177, 7: 174, 8: 161, 9: 156, 10: 158, 11: 140, 12: 132, 13: 130, 14: 118, 15: 132, 16: 131, 17: 152, 18: 153, 19: 178, 20: 167, 21: 162, 22: 177, 23: 167, 24: 162, 25: 138, 26: 130, 27: 156}
dEp9s4: ranges {0: 125, 1: 143, 2: 146, 3: 152, 4: 161, 5: 168, 6: 177, 7: 174, 8: 161, 9: 154, 10: 162, 11: 143, 12: 132, 13: 132, 14: 120, 15: 137, 16: 136, 17: 156, 18: 154, 19: 178, 20: 167, 21: 161, 22: 177, 23: 163, 24: 161, 25: 140, 26: 129, 27: 157}
dEpCs4: ranges {0: 122, 1: 139, 2: 145, 3: 149, 4: 161, 5: 170, 6: 177, 7: 177, 8: 161, 9: 156, 10: 159, 11: 143, 12: 132, 13: 131, 14: 120, 15: 135, 16: 135, 17: 156, 18: 155, 19: 180, 20: 169, 21: 160, 22: 176, 23: 166, 24: 158, 25: 140, 26: 129, 27: 159}
dEpFs4: ranges {0: 122, 1: 140, 2: 144, 3: 152, 4: 161, 5: 168, 6: 175, 7: 175, 8: 161, 9: 156, 10: 160, 11: 140, 12: 131, 13: 135, 14: 120, 15: 134, 16: 135, 17: 155, 18: 157, 19: 177, 20: 168, 21: 161, 22: 176, 23: 166, 24: 163, 25: 141, 26: 131, 27: 160}
dFp0s4: ranges {0: 102, 1: 123, 2: 123, 3: 136, 4: 136, 5: 153, 6: 164, 7: 160, 8: 145, 9: 137, 10: 140, 11: 120, 12: 104, 13: 110, 14: 95, 15: 110, 16: 115, 17: 135, 18: 134, 19: 160, 20: 149, 21: 151, 22: 161, 23: 142, 24: 140, 25: 115, 26: 109, 27: 131}
dFp3s4: ranges {0: 124, 1: 139, 2: 144, 3: 149, 4: 157, 5: 169, 6: 179, 7: 176, 8: 160, 9: 153, 10: 155, 11: 138, 12: 130, 13: 135, 14: 118, 15: 133, 16: 131, 17: 152, 18: 152, 19: 176, 20: 164, 21: 162, 22: 174, 23: 162, 24: 162, 25: 138, 26: 126, 27: 159}
dFp6s4: ranges {0: 124, 1: 143, 2: 140, 3: 152, 4: 155, 5: 169, 6: 179, 7: 176, 8: 160, 9: 154, 10: 159, 11: 136, 12: 129, 13: 135, 14: 118, 15: 133, 16: 134, 17: 155, 18: 153, 19: 177, 20: 166, 21: 162, 22: 176, 23: 164, 24: 160, 25: 136, 26: 132, 27: 159}
dFp9s4: ranges {0: 124, 1: 141, 2: 142, 3: 152, 4: 159, 5: 170, 6: 178, 7: 173, 8: 159, 9: 150, 10: 158, 11: 138, 12: 132, 13: 134, 14: 117, 15: 133, 16: 135, 17: 154, 18: 153, 19: 177, 20: 164, 21: 161, 22: 175, 23: 161, 24: 160, 25: 139, 26: 130, 27: 154}
dFpCs4: ranges {0: 128, 1: 144, 2: 144, 3: 153, 4: 159, 5: 172, 6: 176, 7: 173, 8: 159, 9: 154, 10: 158, 11: 141, 12: 130, 13: 134, 14: 120, 15: 132, 16: 134, 17: 155, 18: 154, 19: 177, 20: 163, 21: 161, 22: 177, 23: 164, 24: 163, 25: 140, 26: 130, 27: 158}
dFpFs4: ranges {0: 127, 1: 143, 2: 144, 3: 151, 4: 158, 5: 170, 6: 178, 7: 172, 8: 161, 9: 152, 10: 160, 11: 140, 12: 132, 13: 135, 14: 117, 15: 136, 16: 134, 17: 155, 18: 155, 19: 177, 20: 166, 21: 163, 22: 177, 23: 164, 24: 161, 25: 137, 26: 132, 27: 159}
"""

In [ ]:
# select data
data_select = Ds.scan

if data_select is Ds.equalisation_off:
    data = eq0
    figsize = (18,6)
    label_rotation = 65
elif data_select is Ds.equalisation_2:
    data = eq2
    figsize = (18,6)
    label_rotation = 65
elif data_select is Ds.equalisation_4:
    data = eq4
    figsize = (18,6)
    label_rotation = 65
elif data_select is Ds.full_scan:
    data = full
    figsize = (36,12)
    label_rotation = 90
elif data_select is Ds.scan:
    data = undersampled_scan
    figsize = (36,12)
    label_rotation = 65
elif data_select is Ds.scan_172:
    data = undersampled_scan_172
    figsize = (36,12)
    label_rotation = 65
elif data_select is Ds.scan_192:
    data = undersampled_scan_192
    figsize = (36,12)
    label_rotation = 65

else:
    raise NotImplementedError

# parse data
data = data.split("\n")
parsed = {}
all_settings = []
for line in data:
    match = cre.match(line)
    if match:
        dd,pd,ps = int(match.group(Hg.driver_dac),16),int(match.group(Hg.pre_dac),16),int(match.group(Hg.pll_stages),16)
        all_settings.append((dd,pd,ps))
        parsed[(dd,pd,ps)] = {int(match.group(2*i+Hg.pll_stages+Ilg.lane+1)):int(match.group(2*i+Hg.pll_stages+Ilg.width+1)) for i in range(LANES)}

In [ ]:
SHIFT = 4
def get_settings(k,ignore_ps=True):
    dd,pd,ps = k
    if ignore_ps:return dd<<SHIFT|pd
    else:return ps<<2*SHIFT|dd<<SHIFT|pd

def s2str(s,ignore_ps=True):
    if ignore_ps:return f"d{(s>>SHIFT)&0xF:x}p{s&0xF:x}"
    else:return f"s{(s>>2*SHIFT)&0xF}d{(s>>SHIFT)&0xF:x}p{s&0xF:x}"

In [ ]:
#reverse_parsed = {lane: {k: width} for k in parsed.keys() for lane,width in parsed[k].items()}
reverse_parsed = {lane: None for lane in range(LANES)}
reverse_parsed_per_module = {module: {s: [] for s in all_settings} for module in range(1,MODULES+1)}
avg = {module: {} for module in range(1,MODULES+1)}
std = {module: {} for module in range(1,MODULES+1)}
for lane in range(LANES):
    reverse_parsed[lane] = {k: parsed[k][lane] for k in parsed.keys()}

for lane in range(LANES):
    for k in all_settings:
        module = lane2module_lut[lane]
        reverse_parsed_per_module[module][k].append(reverse_parsed[lane][k])

for module in avg.keys():
    for k in all_settings:
        avg[module][get_settings(k)] = st.mean(reverse_parsed_per_module[module][k])
        std[module][get_settings(k)] = st.stdev(reverse_parsed_per_module[module][k])

In [ ]:
marker = {1:'^',2:'*',3:'.',4:'v',5:'1',6:'x',7:'+'}
show = False

fig = figure(num=None, figsize=figsize, dpi=300, facecolor='w', edgecolor='k')

for module in avg.keys():
    x_real = [k for k,_ in avg[module].items()]
    x = [k+.3*module for k,_ in avg[module].items()]
    y = [v for _,v in avg[module].items()]
    e = [v for _,v in std[module].items()]
    plt.errorbar(x, y, e, linestyle='--', marker=marker[module],capsize=3,linewidth=0.95)
plt.legend(avg.keys())
plt.grid()
plt.xticks(x_real,[s2str(i) for i in x_real], rotation=label_rotation)
plt.xlabel('Settings')
plt.ylabel('Idelay scan range')
plt.title(f"Scan for settings {data_select.name.replace('_',' ')}. Pulse 64 pixels at 100 kHz")
if show:
    plt.show()
else:
    idelay_dir = 'idelay'
    if not os.path.exists(idelay_dir):      
        os.makedirs(idelay_dir)
    filename = os.path.join(idelay_dir,f'idelay_scan_{data_select.name}.pdf')
    plt.savefig(filename,bbox_inches='tight')